In [ ]:
%load_ext autoreload
%aimport pyhipp
%aimport elucid
%aimport mahgic
%aimport pyhipp_sims
%aimport dynamic_hotness

import os
print(os.environ.get('MAHGIC_WORK_DIR'))

In [ ]:
from __future__ import annotations
import typing, gc, os, sys, json
from pprint import pprint, pformat
from typing import Self, Callable
from pyhipp.core import DataDict, abc, DataTable
from pyhipp.core.dataproc import Num, Mask, GridMd
from pyhipp.io import h5
from pyhipp.io.excel import Excel
from pyhipp import stats
from pyhipp.stats.random import Rng
from pyhipp import astro
from pyhipp.astro import stats as a_stats
from pyhipp.astro import plot as a_plot
from pyhipp import plot
import numpy as np, pandas as pd
from pyhipp_sims import sims
import Corrfunc
from smbh_model.utils.io import Json

plot.abc.mpl.rc('mathtext', fontset='dejavusans') # to be compatible with Ziwen

from obs import assembly_bias
from obs.assembly_bias import clustering, utils
from obs.assembly_bias.clustering import SubSample, SubSampleManager
from obs.assembly_bias.config import ProjPaths, ColorSets
from obs.assembly_bias import sample, clustering, obs_catalogs, sample_cosmic_web
from scipy.spatial import KDTree
from functools import cached_property
from pathlib import Path
from dataclasses import dataclass, asdict

cs_dark2 = plot.ColorSeq.predefined('dark2').get_rgba()
cs_set1 = plot.ColorSeq.predefined('set1').get_rgba()
cs_set2 = plot.ColorSeq.predefined('set2').get_rgba()
cs_tab10 = plot.ColorSeq.predefined('tab10').get_rgba()

def binned_proj_cross(
    subs_ref: SubSample, 
    s_parent: sample.SubhaloSet,
    key = 'r1_DMO',
    ps = [0.00, 0.02, 0.33, 0.67, 0.98, 1.00],
    cls_kw = {'pimax': 40., 'n_repeat': 5}
):
    '''Binned by a quantity'''
    subs_list = SubSample.list_from_percents(
        s_parent, key=key, ps=ps, add_rsd=True)
    out = {str(i): subs_ref.proj_cross_with(subs, **cls_kw)  
         for i, subs in enumerate(subs_list) }
    return out

def wp_with_ratio1(out: DataDict, out_ref: DataDict, use_median=False):
    lx, y_mean, y_sd = out['lg_rs_c', 'wp', 'wp_sd']
    lx_ref, y_mean_ref, y_sd_ref = out_ref['lg_rs_c', 'wp', 'wp_sd']
    x, x_ref = 10.**lx, 10.**lx_ref
    assert (np.abs(lx-lx_ref) < 1.0e-5).all()
    yr_mean = y_mean / y_mean_ref
    yr_sd = Num.safe_div(y_sd, y_mean_ref)
    
    return x, y_mean, y_sd, yr_mean, yr_sd

def fill_by_hids(n_dst, hids, vals, fill = 0):
    val_dsts = np.full(n_dst, fill_value=fill, 
                       dtype=vals.dtype)    
    val_dsts[hids] = vals
    return val_dsts

In [ ]:
%autoreload

In [ ]:
gc.collect()

# Samples

## HMF and expected number of Coma

In [ ]:
info = sims.predefined['elucid']
cosm = info.cosmology

In [ ]:
mf = cosm.halo_theory.mass_function(np.log10(6.2e4), 7., dlgM=0.05, z=0.)

In [ ]:
dlgM = mf.lgM[1] - mf.lgM[0]
dn = (mf.dn_dlgM * dlgM).sum()

In [ ]:
A_sky = 7047.0 * (np.pi / 180.0)**2
d1, d2 = cosm.distances.comoving_at([0.01, 0.04])
vol = (A_sky / 3.0)*(d2**3 - d1**3)

In [ ]:
vol, dn

In [ ]:
vol * dn

## Group catalogs

In [ ]:
from elucid.geometry.frame import SimFrameSdssL500

info = sims.predefined['elucid']
frame = SimFrameSdssL500()

In [ ]:
def parse_sample(d_grpcat: DataDict):
    lg_m_group = d_grpcat['lg_m_group']
    m_group = 10.0**lg_m_group
    
    x_j2k_cor = d_grpcat['x']
    x_sim_cor = frame.pos_j2k_to_sim(x_j2k_cor)
    ra, dec, z_cor = frame.pos_j2k_to_ra_dec_z(x_j2k_cor)
    
    d_out = DataDict({
        'm_group': m_group,
        'x_j2k_cor': x_j2k_cor,
        'x_sim_cor': x_sim_cor,
        'ra': ra,
        'dec': dec,
        'z_cor': z_cor,
    })
    
    return d_out

In [ ]:
f_in = info.root_dir / 'supplementary/reconstruction_pipeline/raw_files/wang_huiyuan_2016/GroupW5Zc.hdf5'
d_grpcat = h5.File.load_from(f_in, key='SDSS_group_z12Cor_XYZ') 
d_out = parse_sample(d_grpcat)

In [ ]:
f_out = ProjPaths.obs_dir / 'recon_samples' / 'groups.hdf5'
h5.File.dump_to(f_out, d_out, f_flag='w')

In [ ]:
h5.File.ls_from(f_out)

## Observational Dwarfs

### Todal

In [ ]:
s_g = obs_catalogs.tot_dwarfs

z, lm, color, n = s_g.data['z', 'lm_s', 'color_gr', 'n_sersic']
sel = (z < 0.04) & (lm >= 8.5) & (color <= 0.6) & (n <= 1.6)
s_g = s_g.data.subset(sel)
len(s_g['z'])

Found the fraction of dwarfs in Sigma_* bins:

- z004: [0.02467638, 0.23220065, 0.25970874, 0.48341424]
- z003: [0.02324263, 0.22193878, 0.25226757, 0.50255102]

Sigma* medians:
- z003:  [6.212119069661753, 11.406763865855208, 19.36879881129817, 43.12292141750194]

In [ ]:
#s_g = obs_catalogs.tot_dwarfs.s_massive
s_g = obs_catalogs.tot_dwarfs.s_massive_z003

bins = [-1., 7.], [7., 15.], [15., 25.], [25., 1.0e6]
ps, Sigmas = [], []
for l, h in bins:
    S = s_g['Sigma_star']
    sel = (S >= l) & (S < h)
    print(sel.sum())
    p = np.sum(sel)/len(S)
    ps.append(p)
    Sigmas.append(np.median(S[sel]))
ps = np.array(ps)
ps, Sigmas

In [ ]:
s = s_g['Sigma_star']

fig, ax = plot.subplots(1, figsize=4.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

ax.hist(np.log10(s), range=[0.5, 2.35], bins=30)

ax.lim([0.5, 2.5]).label(r'\log(\Sigma_{\rm *})', r'\rm Hist')

In [ ]:
%autoreload
gc.collect()
from elucid.geometry.frame import SimFrameSdssL500
from elucid.geometry.mask import ReconstAreaMaskSdssL500
from scipy.spatial import KDTree

In [ ]:
info = sims.predefined['elucid']
frame = SimFrameSdssL500()
rc_mask = ReconstAreaMaskSdssL500(mask_name='SDSS_reconstruction_area_large')
# rc_mask = ReconstAreaMaskSdssL500()

In [ ]:
def parse_sample(samp: obs_catalogs.Dwarfs):
    ra, dec, z, d_c = samp.data['ra', 'dec', 'z', 'd_c']
    x_j2k = frame.pos_ra_dec_d_to_j2k(ra, dec, d_c)
    x_sim = frame.pos_j2k_to_sim(x_j2k)
    
    l_box = rc_mask.box_size
    in_recon = rc_mask.is_in_reconst_area(x_sim) \
        & (x_sim >= 0.).all(1) \
        & (x_sim < l_box).all(1)
    print(f'{in_recon.sum()}/{len(in_recon)} in reconstruction area')
    
    sel = in_recon
    x_j2k, x_sim = x_j2k[sel], x_sim[sel]
    d_out = DataDict({
        k: v[sel] for k, v in samp.data.items()
    }) | {
        'x_j2k': x_j2k,
        'x_sim': x_sim,
    }

    print('x_range=', x_sim.min(), x_sim.max())
    
    return d_out

def match_RSD_cor_sample(d_out: DataDict):
    
    f_in = info.root_dir / 'supplementary/raw_files/wang_huiyuan_2016/GroupW5Zc.hdf5'
    d_raw = h5.File.load_from(f_in)
    z_obs = d_raw['galaxy_ori_z/z']
    ra, dec = d_raw['SDSS7_z12Cor_pro']['ra', 'dec']
    pos_obs = np.column_stack([ra, dec, z_obs])
    
    pos_out = np.column_stack(d_out['ra', 'dec', 'z']) 
    ds, ids = KDTree(pos_obs).query(pos_out)
    print('Matched. Maximal "distance" =', ds.max())
    
    Mr_n5lgh, x_j2k_cor = d_raw['SDSS7_z12Cor_xyz2']['Mr_n5lgh', 'x']
    Mr_n5lgh, x_j2k_cor = Mr_n5lgh[ids], x_j2k_cor[ids]
    x_sim_cor = frame.pos_j2k_to_sim(x_j2k_cor)
    d_c_cor = np.linalg.norm(x_j2k_cor, axis=1)
    z_cor = info.cosmology.redshifts.at_comoving(d_c_cor)
    d_out |= {
        'Mr_n5lgh': Mr_n5lgh,
        'x_j2k_cor': x_j2k_cor,
        'x_sim_cor': x_sim_cor,
        'd_c_cor': d_c_cor,
        'z_cor': z_cor,
    }

In [ ]:
samp = obs_catalogs.diffuse_dwarfs
d_out = parse_sample(samp)
match_RSD_cor_sample(d_out)
f_out = ProjPaths.obs_dir / 'recon_samples' / 'diffuse_dwarfs.hdf5'
# f_out = ProjPaths.obs_dir / 'recon_samples' / 'diffuse_dwarfs.2.hdf5'
h5.File.dump_to(f_out, d_out, f_flag='w')

In [ ]:
samp = obs_catalogs.compact_dwarfs
d_out = parse_sample(samp)
match_RSD_cor_sample(d_out)
f_out = ProjPaths.obs_dir / 'recon_samples' / 'compact_dwarfs.hdf5'
#f_out = ProjPaths.obs_dir / 'recon_samples' / 'compact_dwarfs.2.hdf5'
h5.File.dump_to(f_out, d_out, f_flag='w')

In [ ]:
h5.File.ls_from(f_out)

In [ ]:
z1, z2, d1, d2 = d_out['z', 'z_cor', 'd_c', 'd_c_cor']

fig, axs = plot.subplots((1,2), share=False, 
                         space=0.25, subsize=(4.5, 4.5), 
                         margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat
ax = axs_f[0]
ax.c('r').hist(z1, bins=12, range=[0., 0.08])
ax.c('k').hist(z2, bins=12, range=[0., 0.08])

ax = axs_f[1]
del_d = d2 - d1
ax.c('r').hist(del_d, bins=12, range=[-5., 5.])

In [ ]:
samp_dif = obs_catalogs.recons_samples.diffuse_dwarfs

In [ ]:
samp_dif

## LGalaxiesELUCID

In [ ]:
%autoreload
gc.collect()
from elucid.geometry.frame import SimFrameSdssL500
from elucid.geometry.mask import ReconstAreaMaskSdssL500

In [ ]:
info = sims.predefined['elucid']
data_dir = ProjPaths.sim_dir_of(info) / 'LGalaxiesGuo13' 

In [ ]:
# Cut mass
f_in = data_dir / 'z0_m1e7p5.hdf5'
d_in = h5.File.load_from(f_in)

u_m = info.cosmology.unit_system.u_m_to_sol
m_s_lb = 1.0e8 / u_m

sel = d_in['objs/m_s'] > m_s_lb
objs = {k : v[sel] for k, v in d_in['objs'].items()} 
d_out = {
    'ctx': d_in['ctx'],
    'header': d_in['header'] | {'m_s_lb': m_s_lb},
    'objs': objs
}
h5.File.dump_to(data_dir / 'z0_m1e8.hdf5', d_out, f_flag='w')

In [ ]:
# Cut reconstruct region
rc_mask = ReconstAreaMaskSdssL500(mask_name='SDSS_reconstruction_area_large'
                                  )

In [ ]:
f_in = data_dir / 'z0_m1e8.hdf5'
d_in = h5.File.load_from(f_in)

sel = rc_mask.is_in_reconst_area(d_in['objs/x'])
n_sel = sel.sum()
frac_sel = n_sel / len(sel)
print(f'Fraction of selected objects: {frac_sel:.2f}')

objs = {k : v[sel] for k, v in d_in['objs'].items()} 
d_out = {
    'ctx': d_in['ctx'],
    'header': d_in['header'],
    'objs': objs
}
h5.File.dump_to(data_dir / 'z0_m1e8_recon.hdf5', d_out, f_flag='w')

## LGalaxies

In [ ]:
sim_info = sims.predefined['millennium_2_scaled_wmap7']
p_dir = ProjPaths.models_dir / 'lgalaxies_guo13' / sim_info.name

In [ ]:
'''
p_in = p_dir / 'MS2_z0.txt'
p_out = p_dir / 'MS2_z0.hdf5'
keys = ['galID', 'mainLeafID', 'fofCentralId', 'redshift', 'bulgeMass', 'stellarMass',
        'bulgeSize', 'stellarDiskRadius', 'gbulge', 'gdust', 'type', 'centralMvir', 'rrvir',
        'x', 'y', 'z', 'len', 'mvir', 'stellarSpinX', 'stellarSpinY', 'stellarSpinZ', 'sfr',
        'massweightedAge', 'infallSnap', 'gnodust', 'rnodust', 'inodust',]
'''
'''
p_in = p_dir / 'MS2_z0_all.txt'
p_out = p_dir / 'MS2_z0_all.hdf5'
keys = ['galaxyID', 'bulgeMass', 'stellarMass', 'bulgeSize', 'stellarDiskRadius',
 'type', 'centralMvir', 'x', 'y', 'z', 'mvir', 'stellarSpinX', 'stellarSpinY',
 'stellarSpinZ', 'sfr', 'massweightedAge', 'infallSnap',]
'''

p_in = p_dir / 'z0_all_withvmax.txt'
p_out = p_dir / 'z0_all_withvmax.hdf5'
dtype = [
    ('galaxyID', 'i8'),
    ('bulgeMass', 'f8'),
    ('stellarMass', 'f8'),
    ('bulgeSize', 'f8'),

    ('stellarDiskRadius', 'f8'),
    ('type', 'i4'),
    ('centralMvir', 'f8'),
    ('x', ('f8', 3)),
    ('mvir', 'f8'),

    ('rvir', 'f8'),
    ('vvir', 'f8'),
    ('vmax', 'f8'),
    ('stellarSpin', ('f8', 3)),

    ('sfr', 'f8'),
    ('massweightedAge', 'f8'),
    ('infallSnap', 'i4'),
]

d_in = np.loadtxt(p_in, comments='#', delimiter=',', dtype=dtype)
dsets = {
    k: d_in[k] for k in d_in.dtype.names
}
#dsets['x'] = np.array([dsets.pop(k) for k in ['x', 'y', 'z'] ]).T
#dsets['stellarSpin'] = np.array([dsets.pop('stellarSpin'+k) for k in ['X', 'Y', 'Z'] ]).T
#dsets['type'] = dsets['type'].astype(int)
dsets['is_c'] = dsets['type'] == 0

In [ ]:
cosm = sim_info.cosmology
ht = cosm.halo_theory
us = cosm.unit_system

vir = ht.vir_props_crit(dsets['centralMvir'], z=0.)
v_h = vir.v * us.u_v_to_kmps

In [ ]:
f_v = dsets['vmax'] / v_h

In [ ]:
f_v.max()

In [ ]:
out = {
    'objs': dsets,
    'header': {
        'n_subhs': len(dsets['x']),
    },
    'ctx': {
        'sim_name': sim_info.name
    }
}

In [ ]:
h5.File.dump_to(p_out, out, f_flag='w')

In [ ]:
h5.File.ls_from(p_out)

In [ ]:
sim_info = sims.predefined['millennium_2_scaled_wmap7']
samp = sample.LGalaxiesGuo13SampleRaw.load(sim_info)

In [ ]:
samp.subset_by({
    'm_s': [1., None]
})

In [ ]:
#p_in = p_dir / 'z0_dwarf_sample.txt'
#p_out = p_dir / 'z0_dwarf_sample.hdf5'

p_in = p_dir / 'z0_UDG_sample.txt'
p_out = p_dir / 'z0_UDG_sample.hdf5'

d_in = np.loadtxt(p_in)
keys = ['x', 'y','z','sfr','logRe','halomass','stellarmass',
        'stellarage','infallage','g_magnitude_with_dust',
        'central_surface_brightness','g_magnitude_no_dust',]

In [ ]:
dsets = {
    k: v for k, v in zip(keys, d_in.T)
}
dsets['x'] = np.array([dsets.pop(k) for k in ['x', 'y', 'z'] ]).T
out = {
    'objs': dsets,
    'header': {
        'n_subhs': len(dsets['x']),
    },
    'ctx': {
        'sim_name': sim_info.name
    }
}

In [ ]:
h5.File.dump_to(p_out, out, f_flag='w')

In [ ]:
h5.File.ls_from(p_out)

In [ ]:
s_udg = sample.LGalaxiesGuo13SampleUdg.load()

In [ ]:
s_udg.subset_by_value('m_s', 0.001)

In [ ]:
cosm = astro.cosmology.model.predefined['millennium_wmap7']
n_ps = 2160
l_box = 100.0
m_p = cosm.rho_matter(0) * l_box**3 / n_ps**3
m_p, m_p * 1.0e4

In [ ]:
cosm.times.lookback_at([2.0, 1.0, 0.0])

# Results

In [ ]:
from obs.assembly_bias.clustering import SubSampleManager

In [ ]:
d_plts = {
    'panel_a': {}, 'panel_b': {}, 'panel_c': {},
    'panel_d': {}, 'panel_e': {}, 'panel_f': {},
}

In [ ]:
sim_info = sims.predefined['tng_dark_300_1']
pdata = ProjPaths.sim_dir_of(sim_info) / 'z0_all_postprocessed.hdf5'
man_kw = {
    'add_rsd': True, 
    'dmo': True,
    'm_h_range_in_sol': [10**10.5, 10**11.],
}
sman = SubSampleManager(sim_info, pdata, **man_kw)
s_dwarf = sman.s_dwarf

In [ ]:
def get_hist(x: np.ndarray, range, bins=17):
    median = np.median(x)
    h, e = np.histogram(x, density=True, bins=bins, range=range)
    c = 0.5*(e[:-1] + e[1:])
    return {
        'median': pd.DataFrame({'median': np.array([median])}),
        'pdf': pd.DataFrame({'bin_center': c, 'pdf': h})
    }

In [ ]:
ps = np.cumsum([0.02467638, 0.23220065, 0.25970874, 0.48341424][::-1])
ps[-1] = 1.
ps = np.concatenate([[0.], ps])
ps_obs = ps
d_plts['percentiles'] = pd.DataFrame({'percentiles': ps_obs}) 

c = s_dwarf['c_h']
d_plts['panel_c']['sample_total'] = get_hist(c, [2.5, 25.])

subs_list = SubSample.list_from_percents(s_dwarf, 
    'z_half', ps_obs,
    #[0.00, 0.02, 0.33, 0.67, 0.98, 1.00
)
for i, subs in enumerate(subs_list):
    c = subs.samp['c_h']
    d_plts['panel_c'][f'sample_{i}'] = get_hist(c, [2.5+.2*i, 25.])
    
x_med = 10.67
x_sd = 3.05
x_rg = [5.89, 18.85]
d_plts['panel_c']['NIHAO'] = pd.DataFrame({
    'median': np.array([x_med]),
    'stddev': np.array([x_sd]),
    'min': np.array([x_rg[0]]),
    'max': np.array([x_rg[1]]),
})

In [ ]:
fig, axs = plot.subplots((1,2), share=(False, False), space=0.15, subsize=(4.5, 4.5), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

ax = axs_f[0]
for i, subs in enumerate(subs_list):
    c = subs.samp['c']
    c_m = np.median(c)
    
    ax.c(cs_g[i])
    ax.hist(c, density=True, bins=15, lw=3, range=[2.5, 25])
    
    ax.plot([c_m, c_m], [.3, .32])
    
ax.text(r'\rm Median', (.96, .78), ha='right')
ax.lim([2.5, 25], [0., .4]).label(r'c', r'\rm PDF')

ax = axs_f[1]
for i, subs in enumerate(subs_list):
    c = np.log10(subs.samp['spin']) 
    c_m = np.median(c)
    
    ax.c(cs_g[i])
    ax.hist(c, density=True, bins=20, lw=3, range=[-2.5, -.5], label=g_labs[i])
    
    ax.plot([c_m, c_m], [3., 3.2])
    
ax.lim([-2.5,-.5], [0., 4.]).label(r'\log\,\lambda').leg(loc='ul', fontsize=12, 
    title=r'$z_{\rm f}\ {\rm percentiles}$', title_fontsize=13)

In [ ]:
def load_data(sim_name: str, gal_name='dwarfs_lm8.5-9_by-S.json', 
              halo_name='halos_by-M.json'):
    info = sims.predefined[sim_name]
    sim_dir = ProjPaths.man_sim_dir(info)
    
    out = DataDict()
    p = sim_dir.base_dir / 'wp' / gal_name
    wps = Json.load_file(p)
    wp_ref = wps.pop('total')
    out['gal'] = [clustering.wp_with_ratio(wps[str(i)], wp_ref) 
           for i in range(len(wps)) ]
    out['gal_tot'] = clustering.wp_with_ratio(wp_ref, wp_ref)
    
    wps = Json.load_file(sim_dir.base_dir / 'wp' / halo_name)
    out['halo'] = [clustering.wp_with_ratio(wps[str(i)], wp_ref) 
            for i in range(len(wps)) ]
    return out

def cvt_wp_data(wp: DataDict):
    x, (y, (el, eh)), (yr, (elr, ehr)) = wp['x', 'y/ebar', 'y_ratio/ebar']
    return pd.DataFrame({
        'r': x, 
        'wp': y, 'wp_err_lo': el, 'wp_err_hi': eh,
        'ratio': yr, 'ratio_err_lo': elr, 'ratio_err_hi': ehr,
    })

## Hydro and SAM

In [ ]:
wps_all = DataDict({
    'tng': load_data('tng'),
    #'lgal': load_data('millennium_2_scaled_wmap7'),
    'lgal': load_data('tng_dark', 'dwarfs_H15_lm8.5-9_by-S.json'),
    'tng50': load_data('tng_50_1', 'dwarfs_lm8-9_by-S.json'),
})

In [ ]:
for ki, ko in [['lgal', 'panel_d'], ['tng', 'panel_e'], ['tng50', 'panel_f']]:
    wps = wps_all[ki]
    out = d_plts[ko]
    for i_sub, wp in enumerate(wps['gal']):
        out[f'galaxy_sample_{i_sub}'] = cvt_wp_data(wp)
    out['galaxy_all'] = cvt_wp_data(wps['gal_tot'])
    for i_sub, wp in enumerate(wps['halo']):
        out[f'halo_sample_{i_sub}'] = cvt_wp_data(wp)

## DMO

In [ ]:
wps_all = DataDict({
    'z_half': load_data('tng_dark_300_1', 'dwarf_halos_lm10p5-11_by-z_half.json'),
    'spin_h': load_data('tng_dark_300_1', 'dwarf_halos_lm10p5-11_by-spin_h.json'),
})

In [ ]:
for ki, ko in [['z_half', 'panel_a'], ['spin_h', 'panel_b']]:
    wps = wps_all[ki]
    out = d_plts[ko]
    for i_sub, wp in enumerate(wps['gal']):
        out[f'galaxy_sample_{i_sub}'] = cvt_wp_data(wp)
    out['galaxy_all'] = cvt_wp_data(wps['gal_tot'])
    for i_sub, wp in enumerate(wps['halo']):
        out[f'halo_sample_{i_sub}'] = cvt_wp_data(wp)

In [ ]:
Excel.dump_file(ProjPaths.proj_dir/'data_for_publication/Wang_EDfig8.xlsx', d_plts)

In [ ]:
sim_info = sims.predefined['tng_dark_300_1']
s_dir = ProjPaths.sim_dir_of(sim_info) / 'wp'
f_pref = 'wp-result-pi40-resamp50'

path = s_dir / f'{f_pref}.hdf5'
# path = ProjPaths.sim_dir_of(sim_info) / 'wp-result-pi40-resamp50-addej.hdf5'
res_wp = h5.File.load_from(path)

path = s_dir / f'{f_pref}-r1-bins.hdf5'
res_wp['dwarfs'] |= h5.File.load_from(path)['dwarfs']

path = s_dir / f'{f_pref}-r1-list.hdf5'
res_wp['dwarfs'] |= h5.File.load_from(path)['dwarfs']

for path in [
    f'r1_binned_1.hdf5', f'r1_binned_2.hdf5', f'rho_0_binned.hdf5',
    f'rho_0_binned_1.hdf5', f'rho_0_binned_2.hdf5', 
]:
    res_wp['dwarfs'] |= h5.File.load_from(s_dir/path)['dwarfs']

res_wp['meta'] = {
    'source': 'TNG300-1-Dark',
    'selection': 'central & m_mean200 in 10**{10.5-11.0} & last_sat_z >= 15',
    'subsample selection': 'non-overlapping breaking by percentiles 0.02, 0.33, 0.67, 0.98',
    'author': 'Yangyao Chen, yangyaochen.astro@foxmail.com',
    'usage': 'see load-example.py',
}

In [ ]:
out = {
    k: res_wp[k] for k in ['meta', 'dwarfs', 'halos']
}
path = s_dir / 'wp-tng300-1-dmo.json'
with open(path, 'w') as f:
    json.dump(out, f, default=json_default, indent=2)

In [ ]:
res_wp.keys()

In [ ]:
res_wp['halos/0'].keys()

In [ ]:
res_wp['dwarfs'].keys()

In [ ]:
res_wp['dwarfs']['r1'].keys()

In [ ]:
sim_info = sims.predefined['tng_dark_300_1']
path = ProjPaths.sim_dir_of(sim_info) / 'wp/wp-tng300-1-dmo.json'
with open(path) as f:
    wp_out = parse_json_input(json.load(f))

In [ ]:
sim_info = sims.predefined['tng_dark_300_1']
path = ProjPaths.sim_dir_of(sim_info) / 'wp-tng300-1-dark.json'
with open(path) as f:
    wp_out = parse_json_input(json.load(f))

In [ ]:
cs_g = ['b', 'g', cs_set1[4], 'r', 'purple'][::-1]
g_labs = [
    r'$0^{\rm th}\,$-$\,2^{\rm nd}$',
    r'$2^{\rm nd}\,$-$\,33^{\rm rd}$',
    r'$33^{\rm rd}\,$-$\,67^{\rm th}$',
    r'$67^{\rm th}\,$-$\,98^{\rm th}$',
    r'$98^{\rm th}\,$-$\,100^{\rm th}$',
]
h_labs = [
    '?', ####
    r'$11.0 \leqslant M_{\rm h} < 12.0$',
    r'$12.0 \leqslant M_{\rm h} < 13.0$',
    r'$13.0 \leqslant M_{\rm h}$',
]
lws_h = np.linspace(1., 3.5, 4)
key_labs = r'z_{\rm f}', r'\lambda'

In [ ]:
wp_out['dwarfs'].keys()

In [ ]:
fig, axs = plot.subplots((2,2), share=(True, 'row'),
    space=0.055, subsize=(5., 3.75), 
    margin=[0.02, 0.02, 0.08, 0.08], layout='none',
    ratios=(None, [2, 1.]))
axs_f = axs.flat

leg_kw = dict(title_fontsize=13., labelspacing=.2, numpoints=1, 
              fontsize=13, labelcolor='linecolor')
fn = wp_with_ratio1 # wp_with_ratio

# for i_res, key_prop in enumerate(['z_f', 'spin']):
for i_res, key_prop in enumerate(['r1', 'rho_0_0.1']):

    axs_c = axs[:,i_res]
    ax, ax_r = axs_c

    wp_ref = wp_out['dwarfs/tot']
    wps_g = wp_out[f'dwarfs/{key_prop}']
    wps_h = wp_out['halos']

    x, y, y_e, yr, yr_e = fn(wp_ref, wp_ref)
    axs_c.c('k')
    ax.errorbar(x, y, yerr=y_e, zorder=20, lw=2, label=r'$\rm Total$')
    ax_r.errorbar([1.0e-10, 1.0e10], [1., 1.], yerr=[0., 0.],
                    zorder=20, lw=2.)

    for i, i_sub in enumerate(range(len(wps_g))):
        axs_c.c(cs_g[i])
        x, y, y_e, yr, yr_e = fn(wps_g[str(i_sub)], wp_ref)
        
        ax.errorbar(x, y, yerr=y_e, zorder=10-i_sub, lw=4, label=g_labs[i_sub])
        ax_r.errorbar(x, yr, yerr=yr_e, zorder=10-i_sub)
        
    for i, i_sub in enumerate(range(len(wps_h))):
        axs_c.c('grey')
        
        x, y, y_e, yr, yr_e = fn(wps_h[str(i_sub)], wp_ref)
        
        ax.errorbar(x, y, yerr=y_e, lw=lws_h[i])
        ax_r.errorbar(x, yr, yerr=yr_e, label=h_labs[i], lw=lws_h[i])

    ax.leg(loc='ll', 
             title=r'${\rm Dwarfs},\,%s\ {\rm percentiles}$'%(key_labs[i_res]), 
             **leg_kw, ncol=2)    

axs[0].scale('log', 'log').lim([1.00e-2, 12], [0.5, 5.0e3])
axs[1].lim(y=[0.2, 2.75])


axs[0,0].label(y=r'w_{\rm p}(r_{\rm p})\,[h^{-1}{\rm Mpc}]')
axs[1,0].label(y=r'\rm Ratio')\
    .leg(loc=(.5, 2.6), title=r'${\rm Halos},\,M_{\rm h} / {\rm M}_\odot$', **leg_kw)
axs[1].label(r'r_{\rm p}\,[h^{-1}{\rm Mpc}]')
axs.label_outer()

#ProjPaths.save_fig('fig_2pccf_dmo.pdf')
# ProjPaths.save_fig('fig_2pccf_dmo_with_ej.pdf')
#ProjPaths.save_fig('fig_2pccf_dmo_rho_0.pdf')

## Test for quenching 

In [ ]:
# load wp_vs_props2.py

key_sels = 'by_ssfr', 'by_noej', 'all'
S_ranges = [-1., 7.], [7., 15.], [15., 100000.]

# sim_name = 'tng'
# model_suf = ''

# sim_name = 'tng_dark'
# model_suf = '.LGalaxies-H15'

sim_name = 'tng_dark_300_1'
model_suf = '.LGalaxies-H15'

sim_info = sims.predefined[sim_name]
sim_dir = ProjPaths.man_sim_dir(sim_info)

pdata = ProjPaths.sim_dir_of(sim_info) / f'z0_all_postprocessed.hdf5{model_suf}'
m_s_range = [10**8.5, 10**9.]
kw = {'sim_info': sim_info, 'data_file': pdata,
      'm_s_range_in_sol': m_s_range, 'add_rsd': False, 'dmo': False}
smans = {
    'by_ssfr': SubSampleManager(**kw),
    'by_noej': SubSampleManager(**kw, ssfr_lb_in_sol=-1.0, baryon_excl_ej=True),
    'all': SubSampleManager(**kw, ssfr_lb_in_sol=-1.0),
}

p = sim_dir.base_dir / 'wp_vs_prop' / f'z_half.json{model_suf}'
d_in = Json.load_file(p)
d_plt = DataDict()
for i, k in enumerate(key_sels):
    for i_s, (S_lo, S_hi) in enumerate(S_ranges):
        wps = d_in[f'{k}.S{i_s}']
        wp_ref = d_in[f'{k}.S{2}.ref']
        rel_bias = utils.stats.RelativeBias(wps, wp_ref).data

        s = smans[k].s_dwarf.subset_by_value('Sigma_star', S_lo, S_hi)
        z_f = s.objs['z_half']
        h, e = np.histogram(z_f, range=[0., 5.], bins=32, density=True)
        c = 0.5 * (e[:-1] + e[1:])
        z_half_pdf = DataDict({'h': h, 'e': e, 'c': c})
        d_plt[f'{k}.S{i_s}'] = {'rel_bias': rel_bias,
                                'z_half_pdf': z_half_pdf}

Json.dump_file(d_plt, str(p)+'.plt', indent=2)

In [ ]:
d_plts = [
    Json.load_file(ProjPaths.sim_dir_of(sims.predefined['tng']) / 'wp_vs_prop' / 'z_half.json.plt'),
    Json.load_file(ProjPaths.sim_dir_of(sims.predefined['tng_dark']) / 'wp_vs_prop' / 'z_half.json.LGalaxies-H15.plt'),
    Json.load_file(ProjPaths.sim_dir_of(sims.predefined['tng_dark_300_1']) / 'wp_vs_prop' / 'z_half.json.LGalaxies-H15.plt'),
]

In [ ]:
fig, axs_all = plot.subplots((6,3), share=(True, 'row'),
    space=0.03, subsize=(5., 2.55), 
    margin=[0.02, 0.01, 0.05, 0.065], layout='none',
    ratios=(None, [2, 1.,2,1,2,1]))

sim_names = r'\rm TNG', r'\rm L$-$\rm Galaxies\,(100)', r'\rm L$-$\rm Galaxies\,(300)'

labs_samp = r'\rm Star$-$\rm forming\,(main\ texts)', r'\rm Backsplash$-$\rm excluded', r'\rm All'

cs = 'b', 'g', 'r'
labels = r'$\Sigma_* < 7\,{\rm M}_\odot\,{\rm pc}^{-2}$', \
    r'$\Sigma_* = 7$-$15\,{\rm M}_\odot\,{\rm pc}^{-2}$',\
    r'$\Sigma_*\geqslant 15\,{\rm M}_\odot\,{\rm pc}^{-2}$'

labs_samp = r'\rm Star$-$\rm forming\,(main\ texts)', r'\rm Backsplash$-$\rm excluded', r'\rm All'
axs_all.fmt_errorbar(capsize=0)
for i_mod, d_plt in enumerate(d_plts):
    axs = axs_all[[2*i_mod, 2*i_mod+1]]
    for i_k, k in enumerate(key_sels):
        axs_c = axs[:, i_k]
        for i_s in range(3):
            d = d_plt[f'{k}.S{i_s}']
            
            ax = axs_c[0]
            x, y, ye = d['rel_bias']['x', 'y', 'ye']
            ax.c(cs[i_s]).errorbar(x, y, yerr=ye, label=labels[i_s], 
                                lw=1, marker='s', ms=5.5, mew=2.)

            ax = axs_c[1]
            c, h = d['z_half_pdf']['c', 'h']
            ax.c(cs[i_s]).plot(c, h, lw=1.5)
        
    axs[0].scale().lim([0.,5.5], [.1, 3.75])\
        .label(y=r'{\rm Relative\ bias}')
    axs[1].scale(y='log').lim(y=[0.005, 3.])\
        .label(r'z_{\rm f}', r'\rm PDF')
    axs[0,0].text(sim_names[i_mod], (.075, .9))
    if i_mod == 0:
        axs[0,2].leg(loc='lr', numpoints=1, fontsize=13, labelspacing=0)
        for i_k in range(3):
            axs[0,i_k].text(labs_samp[i_k], (.92, .9), ha='right')
    
axs_all.label_outer();

ProjPaths.save_fig('fig_rel_bias_vs_zf_sub_binned.pdf')

# ELUCID-LGalaxiesGuo13

In [ ]:
%autoreload
gc.collect()

In [ ]:
sim_info = sims.predefined['elucid']
s_all = sample.ElucidLGalaxiesGuo13Sample.load(sim_info)

In [ ]:
s_all

In [ ]:
x = s_all['x']
print(x.min(), x.max())

In [ ]:
x, y, z = s_all['x'].T
sel = (z >= 250.) & (z <= 251.)
x, y = x[sel], y[sel]


In [ ]:
fig, ax = plot.subplots(1, figsize=4.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

ax.scatter(x, y)

# ELUCID

In [ ]:
%autoreload

In [ ]:
from pyhipp.field import cubic_box
from elucid.geometry.frame import SimFrameSdssL500
from elucid.geometry.mask import ReconstAreaMaskSdssL500, reconst_area_masks
from obs.assembly_bias import elucid_cv, utils, obs_catalogs
from obs.assembly_bias.utils.gal_web_cor import GalEnv, GalEnvSimed
from obs.assembly_bias.utils import stats as u_stats
from obs.assembly_bias.elucid_cv import ReconSample, ObsProjCross
from obs.assembly_bias.clustering import SubSampleManager, SimCross
from sklearn.neighbors import KDTree as SkKDTree
from scipy.interpolate import interp1d
from obs.assembly_bias.sidm.drivers import SIDMDriver
from obs.assembly_bias import sidm

# sim_info = sims.predefined['elucid']
sim_info = sims.predefined['elucid_ext_v2']
sim_dir = ProjPaths.man_sim_dir(sim_info)

def take_wps_rel(d_in: DataDict, d_rel: DataDict):
    wp_rel = d_in['wp_samples'] / d_rel['wp_samples']
    assert wp_rel.shape[1] == 1
    wp_rel = wp_rel[:, 0]
    y, yl, yh = np.median(wp_rel), np.quantile(wp_rel, 0.16), np.quantile(wp_rel, 0.84)
    yel, yeh = y - yl, yh - y
    x, (xl, xh) = d_in['sub_sample']['median', '1sigma']
    xel, xeh = x - xl, xh - x
    return x, y, yel, yeh

def take_wps_rels(d_ins: DataDict, d_rel: DataDict = None):
    n_ds = len(d_ins)
    if d_rel is None:
        d_rel = d_ins[str(n_ds-1)]
    wp_rel = []
    for i in range(n_ds):
        wp_rel.append(take_wps_rel(d_ins[str(i)], d_rel))
    wp_rel = np.array(wp_rel)
    return wp_rel

dps = [0.02467638, 0.23220065, 0.25970874, 0.48341424]
ps = np.cumsum(dps)
ps[-1] = 1.
ps = np.concatenate([[0.], ps])
ps_obs = ps

dps = [0.02324263, 0.22193878, 0.25226757, 0.50255102]
ps = np.cumsum(dps)
ps[-1] = 1.
ps = np.concatenate([[0.], ps])
ps_obs_z003 = ps

ps = np.cumsum(dps[::-1])
ps[-1] = 1. 
ps = np.concatenate([[0.], ps])
ps_obs_inv = ps

ps_fine = np.array([0.0, 0.02, 0.04, 0.06, 0.08, 0.14, 0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9, 1.0])

ps_list = {
    'obs_bin': ps_obs,
    'fine_bin': ps_fine,
}

## Datasets

In [ ]:
# extract sub sample

sim_info = sims.predefined['elucid_ext_v2']
sim_dir = ProjPaths.man_sim_dir(sim_info)
p_data = ProjPaths.sim_dir_of(sim_info) / 'z0_all.hdf5'

u_m = u_m = sim_info.cosmology.unit_system.u_m_to_sol
m_lb, m_ub = 10**10.5 / u_m, 10**11. / u_m
with h5.File(p_data) as f:
    pprint(tuple(f['objs'].keys()), compact=True)
    d_in = f['objs'].datasets.load(keys=[
        'c_mean200_AM_lm10p5_to_11', 'd_c',
        'dec', 'is_c', 'is_recon', 'is_recon_large', 'last_sat_z', 'm_mean200',
        'm_peak', 'ra', 'spin', 'v_max', 'x', 'z', 'z_half',
        'Sigma_star_AM_lm10p5_to_11_rho0.85'
    ])
    sel = d_in['is_c'] & (d_in['m_mean200'] >= m_lb) & (d_in['m_mean200'] < m_ub) \
        & (d_in['last_sat_z'] >= 15.) & d_in['is_recon_large'] \
        & (d_in['z'] >= 0.01) & (d_in['z'] < 0.04)    
    d_in = {k: v[sel] for k, v in d_in.items()}
    ctx = f['ctx'].load()
    header = f['header'].load()

In [ ]:
d_in |= {
    'r_mean200': sim_info.cosmology.halo_theory.vir_props_mean(d_in['m_mean200']).r,
    'c_mean200': d_in['c_mean200_AM_lm10p5_to_11'].copy(),
}

In [ ]:
d_out = {
    'objs': d_in,
    'ctx': ctx, 
    'header': header,
}
h5.File.dump_to(
    str(p_data) + '.c.dwarf.noBS.recon.z0p01-0p04',
    d_out, f_flag='w')

## 2PCF with abundance matching

### 2PCCF results

#### Meausing 2PCCF

In [ ]:
p_data = ProjPaths.sim_dir_of(sim_info) / 'z0_all.hdf5'
man_kw = {'add_rsd': False, 'dmo': True, 
          'm_h_range_in_sol': (10**10.5, 10**11.0),
          'm_h_ref_lb': 10**10.5,
          'extra_mask_dwarfs': {
              'is_recon_large': {'eq': True},
              'z': {'lo': 0.01, 'hi': 0.04},
              #'z': {'lo': 0.01, 'hi': 0.03}
              #'z': {'lo': 0.01, 'hi': 0.06},
              #'z': {'lo': 0.04, 'hi': 0.06},
              #'z': {'lo': 0.06, 'hi': 0.07},
          }
}
sman = SubSampleManager(sim_info, p_data, **man_kw)
sman.s_dwarf

In [ ]:
wp_kw = {'pimax': 40., 'n_repeat': 4, 'rs': np.array([2., 10.]),
         'bootstrap_kw': {'keep_samples': True}}
sim_cross = SimCross(sman)

In [ ]:
prop_pref = 'Sigma_star_AM_lm10p5_to_11_SIDM_r1'
keys = 'sigma0.25', 'sigma0.30', 'sigma0.40' #'sigma0.10', 'sigma0.50', 'sigma1.00'
for k in keys:
    prop_key = f'{prop_pref}_{k}'
    vals = sman.s_dwarf.objs[prop_key]
    print(prop_key, vals.min(), vals.max())
    
    o_data = {
        k: sim_cross.wps_of('s_dwarf', prop_key, ps = ps, **wp_kw) 
        for k, ps in [('obs_bin', ps_obs),]
    }
    o_path = ProjPaths.sim_dir_of(sim_info) / f'wp/am/{prop_key}.hdf5'
    Json.dump_file(o_data, o_path, indent=2)

In [ ]:
prop_key = 'Sigma_star_SIDM_lm10p5_to_11_sigma0.40'

vals = sman.s_dwarf.objs[prop_key]
print(prop_key, vals.min(), vals.max())

o_data = {
    k: sim_cross.wps_of('s_dwarf', prop_key, ps = ps_list[k], **wp_kw) 
    for k in ['obs_bin']
}
o_path = ProjPaths.sim_dir_of(sim_info) / f'wp/am/{prop_key}.hdf5'
Json.dump_file(o_data, o_path, indent=2)

In [ ]:
# prop_key = 'Sigma_star_AM_lm10p5_to_11_z004_to_006'
# prop_key = 'Sigma_star_AM_lm10p5_to_11_z006_to_007'

# prop_pref = 'Sigma_star_AM_lm10p5_to_11_z001_to_006'
prop_pref = 'Sigma_star_AM_lm10p5_to_11_z001_to_003'

#[1., .8, .6]:  #1., 0.99, 0.95, 0.9, 0.8, 0.7, 0.6, 0.5:      # 
for rho in [1., 0.9, 0.85, 0.8, 0.6, 0.5]: 
    prop_key = f'{prop_pref}_rho{rho:.2f}'

    vals = sman.s_dwarf.objs[prop_key]
    print(prop_key, vals.min(), vals.max())

    o_data = {
        k: sim_cross.wps_of('s_dwarf', prop_key, ps = ps, **wp_kw) 
        for k, ps in ps_list.items()
    }
    o_path = ProjPaths.sim_dir_of(sim_info) / f'wp/am/{prop_key}.hdf5'
    Json.dump_file(o_data, o_path, indent=2)

In [ ]:
prop_pref = 'SIDM_r1_iterFixAr'
#prop_pref = 'SIDM_rh4'
#keys = 'sigma0.25', 'sigma0.30', 'sigma0.40' # 'sigma0.10', 'sigma0.50', 'sigma1.00'
#keys = 'sigma3.00',
keys = 'sigma0.10', 'sigma0.30', 'sigma1.00'
ps_list = {
    'obs_inv_bin': ps_obs_inv,
}
for k in keys:
    prop_key = f'{prop_pref}_{k}'
    vals = sman.s_dwarf.objs[prop_key]
    print(prop_key, vals.min(), vals.max())
    
    o_data = {
        k: sim_cross.wps_of('s_dwarf', prop_key, ps = ps, **wp_kw) 
        for k, ps in ps_list.items()
    }
    o_path = ProjPaths.sim_dir_of(sim_info) / f'wp/am/{prop_key}.hdf5'
    Json.dump_file(o_data, o_path, indent=2)

#### Show 2PCCF

In [ ]:
wps = DataDict()

#for i_k, prop_key in enumerate(('Sigma_star_AM_lm10p5_to_11', 
#                                'Sigma_star_AM_lm10p5_to_11_z004_to_006', 
#                                'Sigma_star_AM_lm10p5_to_11_z006_to_007')):
#    d_in = Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/am/{prop_key}.hdf5')
#    wps[f'z_f{i_k}'] = {
#        'fine_bin': take_wps_rels(d_in['fine_bin'], d_in['obs_bin/3']),
#        'obs_bin': take_wps_rels(d_in['obs_bin'])
#    }

prop_pref = 'Sigma_star_AM_lm10p5_to_11'
# prop_pref = 'Sigma_star_AM_lm10p5_to_11_z001_to_003'
# for rho in [1., 0.99, 0.95, 0.9, 0.85, 0.8, 0.7, 0.6, 0.5]:
for rho in [1., 0.9, 0.85, 0.8, 0.5]:
    prop_key = f'{prop_pref}_rho{rho:.2f}'
    out_key = f'z_f_rho{rho:.2f}'
    d_in = Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/am/{prop_key}.hdf5')
    wps[out_key] = {
        'fine_bin': take_wps_rels(d_in['fine_bin'], d_in['obs_bin/3']),
        'obs_bin': take_wps_rels(d_in['obs_bin']),
        'data': DataDict({
            'z_half': sman.s_dwarf.objs['z_half'],
            'S': sman.s_dwarf.objs[prop_key],
        })
    }
    
#prop_pref = 'Sigma_star_AM_lm10p5_to_11_z001_to_006'
#for rho in [1., 0.8, 0.6]:
#    prop_key = f'{prop_pref}_rho{rho:.2f}'
#    out_key = f'z_f_z001_to_006_rho{rho:.2f}'
#    d_in = Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/am/{prop_key}.hdf5')
#    wps[out_key] = {
#        'fine_bin': take_wps_rels(d_in['fine_bin'], d_in['obs_bin/3']),
#        'obs_bin': take_wps_rels(d_in['obs_bin']),
#        'data': DataDict({
#            'z_half': sman.s_dwarf.objs['z_half'],
#            'S': sman.s_dwarf.objs[prop_key],
#        })
#    }

wps['tng'] = take_wps_rels(Json.load_file(ProjPaths.sim_dir_of(sims.predefined['tng']) / 'wp/ms8.5-9_r2-10.json')['dwarfs/S'])
#wps['lgal'] = take_wps_rels(Json.load_file(ProjPaths.sim_dir_of(sims.predefined['millennium_2_scaled_wmap7']) / 'wp/ms8.5-9_r2-10.json')['dwarfs/S'])
wps['lgal'] = take_wps_rels(
    Json.load_file(ProjPaths.sim_dir_of(sims.predefined['tng_dark']) / 'wp/dwarfs_H15_lm8.5-9_r2-10_by-S.json')
)

# z004, massive sample, z-weighted
wps['obs'] = np.array([
    [6.2,    2.4, 0.28, 0.28,],
    [11.37,  1.44, 0.1, 0.1,],
    [19.33,  1.29, 0.09, 0.09,],
    [42.37, 1.0, 0.0, 0.0,],
])
# z003, ...
#wps['obs'] = np.array([
#    [6.212119069661753, 11.406763865855208, 19.36879881129817, 43.12292141750194],
#    [2.96, 1.74, 1.46, 1.00],
#    [0.37, 0.15, 0.13, 0.0],
#    [0.37, 0.15, 0.13, 0.0],
#]).T

In [ ]:
def load_prop_rel(sim_name, file_name):
    info = sims.predefined[sim_name]
    file_path = ProjPaths.man_sim_dir(info).base_dir / 'stats' / file_name
    return Json.load_file(file_path)
prop_rels = {
    'lgal': load_prop_rel('tng_dark', 'z_half-Sigma_star.json'),
    'tng': load_prop_rel('tng', 'z_half-Sigma_star.json')
}

In [ ]:
def cvt_bias_curve(bias_curve: np.ndarray):
    x, y, el, eh = bias_curve.T
    return pd.DataFrame({
        'Sigma': x, 
        'bias': y,
        'err_lo': el, 'err_up': eh
    })
    
d_plts = {
    'panel_a': {
        'rho1p00_fine_bin': cvt_bias_curve(wps['z_f_rho1.00/fine_bin']),
    } | {
        k.replace('.', 'p'): cvt_bias_curve(wps[f'z_f_{k}/obs_bin']) for k in [
            'rho1.00', 'rho0.90', 'rho0.85', 'rho0.80', 'rho0.50']
    } | {
        k: cvt_bias_curve(wps[k]) for k in ['obs', 'tng', 'lgal']
    },
    'panel_b': {}
}
y, x = wps[f'z_f_rho0.85/data']['z_half', 'S']
yp = np.linspace(0., 4.5, 128)
xp, el, eh = stats.KernelRegression1D.by_local_kernel(y, x, yp, reduce='errorbar', max_dx=.25)['y'][0].T
d_plts['panel_b']['rho0p85'] = pd.DataFrame({
    'Sigma': xp, 'z_f': yp, 'xerr_lo': el, 'xerr_up': eh})

y, x = wps[f'z_f_rho1.00/data']['z_half', 'S']
xp = interp1d(y, x, kind='linear')(yp)   
d_plts['panel_b']['rho1p00'] = pd.DataFrame({
    'Sigma': xp, 'z_f': yp
})
d_plts['panel_b']['rho1p00_sample'] = pd.DataFrame({
    'Sigma': x, 'z_f': y
})

for i_k, key in enumerate(['tng', 'lgal']):
    y, x, (el, eh) = prop_rels[key]['z_half', 'Sigma_star', 'Sigma_star_err']
    x, y, el, eh = x[::4], y[::4], el[::4], eh[::4]
    d_plts['panel_b'][key] = pd.DataFrame({
        'Sigma': x, 'z_f': y, 'xerr_lo': el, 'xerr_up': eh
    })

In [ ]:
#Excel.load_file(ProjPaths.proj_dir/'data_for_publication/Fig3.xlsx')
#
#for sig in 0.3, 0.4, 0.5:
#    prop_key = f'Sigma_star_SIDM_lm10p5_to_11_sigma{sig:.2f}'
#    d_in = Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/am/{prop_key}.hdf5')
#    wp = take_wps_rels(d_in['obs_bin'])
#    d_plts[f'test_sigma{sig:.2f}'] = cvt_bias_curve(wp)

In [ ]:
#Excel.load_file(ProjPaths.proj_dir/'data_for_publication/Fig3.xlsx')

def get_r1_pred(sigma = 0.30, M_s_in_sol = 10**8.8, 
                A_r_scale = 1., A_r_fixed = None, suf=''):
    
    u_l = sman.sim_info.cosmology.unit_system.u_l_to_pc * 1.0e-3
    r_key = f'SIDM_r1{suf}_sigma{sigma:.2f}'
    r1 = sman.s_dwarf[r_key] * u_l                                # kpc
    z_f = sman.s_dwarf['z_half']
    
    if A_r_fixed is None:
        r50_ref = obs_catalogs.tot_dwarfs.s_massive['r_50']           # kpc
        A_r = np.median(r50_ref) / np.median(r1) * A_r_scale
        print(f'A_r scaled as {A_r}')
    else:
        A_r = A_r_fixed
        print(f'A_r fixed as {A_r}')
    r50 = r1 * A_r
    Sigma = M_s_in_sol / (2. * np.pi * (r50*1.0e3) **2)
    S_outs = []
    for i, p in enumerate(zip(ps_obs[:-1], ps_obs[1:])):
        ql, qh = np.quantile(Sigma, p)
        sel = (Sigma >= ql) & (Sigma < qh)
        S_out = np.median(Sigma[sel])
        S_outs.append(S_out)
    S_outs = np.array(S_outs)
        
    d_in = Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/am/{r_key}.hdf5')['obs_inv_bin']
    _, y, el, eh = take_wps_rels(d_in, d_in['0'])[::-1].T
    
    bias_curve = pd.DataFrame({
        'Sigma': S_outs, 'bias': y, 'err_lo': el, 'err_up': eh
    })
    
    y, x = z_f, Sigma
    yp = np.linspace(0., 4.5, 128)
    xp, el, eh = stats.KernelRegression1D.by_local_kernel(y, x, yp, reduce='errorbar', max_dx=.25)['y'][0].T
    zfsig_rel = pd.DataFrame({
        'Sigma': xp, 'z_f': yp, 'xerr_lo': el, 'xerr_up': eh})
    
    return bias_curve, zfsig_rel, A_r

d_plts |= {'panel_c': {}, 'panel_d': {}}
A_rs, sigmas = [], []
for sig_val, sig_key, A_r_scale, A_r_fixed in [
    (0.10, '0p1', 1.05, 2.2595430708452,), 
    (0.30, '0p3', 1.1,  1.1550470595722,), 
    (1.00, '1p0', 1.05, 0.5739794429262,)
]:
    c, d, A_r = get_r1_pred(sig_val, A_r_scale=A_r_scale, 
                            A_r_fixed=A_r_fixed, suf='_iterFixAr')
    d_plts['panel_c'][f'sigma_m_{sig_key}'] = c
    d_plts['panel_d'][f'sigma_m_{sig_key}'] = d
    A_rs.append(A_r)
    sigmas.append(sig_val)
    
d_plts['panel_c_inset'] = pd.DataFrame({
    'sigma': sigmas, 'A_r': A_rs
})

In [ ]:
Excel.dump_file(ProjPaths.proj_dir/'data_for_publication/Fig3.xlsx', d_plts)

### Abundance modeling 

For Sigma_* and c, and SIDM. 

In z = 0.01 - 0.03 (0.04), no of dwarfs is about 27000 (68000)

In [ ]:
sim_info = sims.predefined['elucid_ext_v2']
p_data = ProjPaths.sim_dir_of(sim_info) / 'z0_all.hdf5'
samp = sample.SimSample.load(sim_info, p_data)
samp.objs['hid'] = np.arange(samp.n_objs, dtype=np.int64)

In [ ]:
u_m = sim_info.cosmology.unit_system.u_m_to_sol
# m_lb, m_ub = 10**10.7 / u_m, 10**10.9 / u_m
m_lb, m_ub = 10**10.5 / u_m, 10**11.0 / u_m

z_lb, z_ub = 0.01, 0.04
# z_lb, z_ub = 0.01, 0.03
#z_lb, z_ub = 0.04, 0.06
#z_lb, z_ub = 0.06, 0.07
#z_lb, z_ub = 0.01, 0.06

s_h = samp.subset_by_value('is_c', eq=True)\
    .subset_by_value('m_mean200', lo=m_lb, hi=m_ub)\
    .subset_by_value('last_sat_z', lo=15.)\
    .subset_by_value('is_recon_large', eq=True)\
    .subset_by_value('z', lo=z_lb, hi=z_ub)
s_h.objs['r_mean200'] = sim_info.cosmology.halo_theory.vir_props_mean(s_h['m_mean200']).r
s_h.objs['c_mean200'] = s_h['c_mean200_AM_lm10p5_to_11'].copy()
s_h

#### Effects of baryon on SIDM

In [ ]:
# n_objs = s_h.n_objs
# ids = Rng(0).choice(n_objs, 4096)
# s_h_sel = s_h.subset(ids)

s_h_sel = s_h

In [ ]:
sigma = .3
S_key = 'Sigma_star_AM_lm10p5_to_11_rho0.85'
kw = dict(samp=s_h_sel, S_key=S_key, sigma_in_cm2perg=sigma)
sidm_drs = {
    'default': SIDMDriver(**kw),
    'no_b': SIDMDriver(**kw, ada_model=None, sidm_with_b=False),
    'ada': SIDMDriver(**kw, sidm_with_b=False),
    'sidm_b': SIDMDriver(**kw, ada_model=None),
}

In [ ]:
for v in sidm_drs.values():
    v.require_r1()

In [ ]:
def stats_zf_rc(sidm_dr: SIDMDriver):
    zf = s_h_sel.objs['z_half']
    r1 = sidm_dr.data['r1'] * sidm_dr.u_l * 1.0e-3  # in [pc]
    lr1 = Num.safe_lg(r1)
    rg = np.quantile(lr1, [0.005, 0.995])
    lr1_p = np.linspace(*rg, 64)
    dx = (rg[1] - rg[0]) / 24
    zf_p, el, eh = stats.KernelRegression1D.by_local_kernel(lr1, zf, lr1_p, 
                              reduce='errorbar', max_dx=dx)['y'][0].T
    h, e = np.histogram(lr1, range= (-1., 1.5), bins=32, density=True)
    c = .5 * (e[:-1] + e[1:])
    return DataDict({
        'lr1': lr1_p, 'zf': zf_p, 'zf_err': (el, eh),
        'hist': DataDict({'h': h, 'e': e, 'c': c}),
    })

In [ ]:
d_plt = {
    k: stats_zf_rc(v) for k, v in sidm_drs.items()
}

In [ ]:
fig, axs = plot.subplots((2,1), share=(True, False), 
                         space=0.02, subsize=(5.75, 3.75), 
                         margin=[0.02, 0.02, 0.075, 0.12], layout='none')
axs_f = axs.flat
cs = cs_dark2[0], cs_dark2[4], cs_dark2[2], 'k'
labs = r'$\rm Baryon\ effects\ disabled$', r'$\rm +Adiabatic\ contraction$', \
    r'$\rm +Poisson$-$\rm Jeans$', r'$\rm +Both\ (main\ texts)$'
for i_k, k in enumerate(['no_b', 'ada', 'sidm_b', 'default']):
    d = d_plt[k]
    axs.c(cs[i_k])
    
    ax = axs_f[0]
    h, e, c = d['hist']['h', 'e', 'c']
    ax.plot(c, h, lw=2.5, label=labs[i_k])
    
    ax = axs_f[1]
    x, y, ye = d['lr1', 'zf', 'zf_err']
    ax.errorfill(x, y, yerr=ye, lw=2.5, label=k, 
                 fill_between_kw={'lw': .5})

ax = axs_f[0]    
r_e = obs_catalogs.tot_dwarfs.s_massive['r_50']
lr_e = Num.safe_lg(r_e)
h, e = np.histogram(lr_e, range=(-0.75, 1.), bins=32, density=True)
c = .5 * (e[:-1] + e[1:])
ax.plot(c, h, lw=3, c=cs_dark2[1], label=r'$\rm Dwarfs\,(R_{50})$')

axs_f[0].lim([-0.99, 0.75], [0., 3.5]).label(y=r'{\rm PDF}')\
    .leg(loc='ul', fontsize=13, labelspacing=0)
axs_f[1].lim(y=[-0.1, 3.25]).label(r'\log\,r_{\rm c}\,[{\rm kpc}]', r'z_{\rm f}')
axs.label_outer()

ProjPaths.save_fig('fig_sidm_baryon_effects.pdf')

#### r1 x rho0 in the c-zf plane

Asked by Yu Rong 2025-03. 

$r_{\rm core} \times \rho_0$ seems to be an invariant in MOND.

In [ ]:
class SIDMMap:
    def __init__(self, s_h_ref: sample.SimSample, n_grid=32, sidm_sigma=0.3):
        info = s_h_ref.sim_info
        us = info.cosmology.unit_system
        u_l = us.u_l_to_pc
        u_m = us.u_m_to_sol

        z_fs = np.linspace(0., 4., n_grid)
        cs = np.linspace(2.5, 40., n_grid-2)

        z_fs_m, cs_m = np.meshgrid(z_fs, cs, indexing='ij')
        z_fs_f, cs_f = z_fs_m.ravel(), cs_m.ravel()

        m_mean200 = 10**10.77 / u_m
        M_s_in_sol = 10**8.8
        r_mean200 = info.cosmology.halo_theory.vir_props_mean(m_mean200).r
        m_mean200 = np.full_like(z_fs_f, m_mean200)
        r_mean200 = np.full_like(z_fs_f, r_mean200)

        z_f0, Sigma0 = s_h_ref.objs['z_half',
                                    'Sigma_star_AM_lm10p5_to_11_rho0.85']
        Sigmas_f = stats.KernelRegression1D.by_knn(
            z_f0, Sigma0, z_fs_f, k=64)['y'][0]
        # Sigmas_f[...] = 1.0e-8
        s_h = sample.SimSample.from_data(DataTable({
            'm_mean200': m_mean200, 'r_mean200': r_mean200,
            'c_mean200': cs_f, 'z_half': z_fs_f,
            'S': Sigmas_f,
        }), sim_info=info, header=DataDict())
        driver = SIDMDriver(s_h, M_s_in_sol, S_key='S', sigma_in_cm2perg=sidm_sigma)
        driver.require_rho0()

        r1s, rho0s = driver.data['r1', 'rho0']
        r1s: np.ndarray = r1s * u_l * 1.0e-3
        rho0s = rho0s * (u_m / u_l**3) * 1.0e9
        r1s = r1s.reshape(z_fs_m.shape)
        rho0s = rho0s.reshape(z_fs_m.shape)

        self.driver = driver
        self.s_h = s_h
        self.z_fs = z_fs
        self.cs = cs
        self.r1s = r1s
        self.rho0s = rho0s

In [ ]:
s_h_sel = s_h
sidm_map = SIDMMap(s_h_sel, sidm_sigma=1.)

In [ ]:
rg = [0., 4.], [2.5, 39.]

ps = np.linspace(0.2, 1., 5)
ps[-1] = 0.99
ps = ps[::-1]

fig, axs = plot.subplots((2,2), 
    share=False, space=0.25, subsize=(5.5, 4.5), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

sig = 1.
cmap = 'gnuplot2'
cb_labs = r'$r_{\rm c}\,[{\rm kpc}]$'

m = sidm_map
z_fs, cs, r1s, rho0s = m.z_fs, m.cs, m.r1s, m.rho0s
lrho0s = Num.safe_lg(rho0s)
inds = .5, .75, 1., 1.5
for i_y, ind in enumerate(inds):
    ax = axs_f[i_y]
    y = Num.safe_lg(r1s * rho0s**ind) 
    #for i_y, y in enumerate([r1s, lrho0s]):
    cb = ax._raw.contourf(z_fs, cs, y.T, levels=32,
                            cmap=cmap, zorder=1)
    fig.colorbar(cb, ax=ax, location='right', aspect=30,
                fraction=0.1, pad=0.02, label=r'$\log\,r_{\rm c}\times \rho_0^{%.2f}$'%ind)

z_f, c = s_h_sel.objs['z_half', 'c_mean200']
for i_ax, ax in enumerate(axs_f):
    s2d = ax.scatter_2d(z_f, c, range=rg, n_bins=32)
    s2d.contour(ps, zorder=4, colors='k', ls='-', lw=1.5)
    ax.label(r'z_{\rm f}', r'c')

lab = r'\sigma_{\rm m} = %.1f\,{\rm cm^2/g}'%(sig)
ax.text(lab, (.92, .08), ha='right', color='w')

# ProjPaths.save_fig('???.pdf')

#### r1 and rho0 in the c-zf plane

In [ ]:
class SIDMMap:
    def __init__(self, s_h_ref: sample.SimSample, n_grid=32, sidm_sigma=0.3):
        info = s_h_ref.sim_info
        us = info.cosmology.unit_system
        u_l = us.u_l_to_pc
        u_m = us.u_m_to_sol

        z_fs = np.linspace(0., 4., n_grid)
        cs = np.linspace(2.5, 40., n_grid-2)

        z_fs_m, cs_m = np.meshgrid(z_fs, cs, indexing='ij')
        z_fs_f, cs_f = z_fs_m.ravel(), cs_m.ravel()

        m_mean200 = 10**10.77 / u_m
        M_s_in_sol = 10**8.8
        r_mean200 = info.cosmology.halo_theory.vir_props_mean(m_mean200).r
        m_mean200 = np.full_like(z_fs_f, m_mean200)
        r_mean200 = np.full_like(z_fs_f, r_mean200)

        z_f0, Sigma0 = s_h_ref.objs['z_half',
                                    'Sigma_star_AM_lm10p5_to_11_rho0.85']
        Sigmas_f = stats.KernelRegression1D.by_knn(
            z_f0, Sigma0, z_fs_f, k=64)['y'][0]
        # Sigmas_f[...] = 1.0e-8
        s_h = sample.SimSample.from_data(DataTable({
            'm_mean200': m_mean200, 'r_mean200': r_mean200,
            'c_mean200': cs_f, 'z_half': z_fs_f,
            'S': Sigmas_f,
        }), sim_info=info, header=DataDict())
        driver = SIDMDriver(s_h, M_s_in_sol, S_key='S', sigma_in_cm2perg=sidm_sigma)
        driver.require_rho0()

        r1s, rho0s = driver.data['r1', 'rho0']
        r1s: np.ndarray = r1s * u_l * 1.0e-3
        rho0s = rho0s * (u_m / u_l**3) * 1.0e9
        r1s = r1s.reshape(z_fs_m.shape)
        rho0s = rho0s.reshape(z_fs_m.shape)

        self.driver = driver
        self.s_h = s_h
        self.z_fs = z_fs
        self.cs = cs
        self.r1s = r1s
        self.rho0s = rho0s

In [ ]:
sidm_maps = [SIDMMap(s_h_sel, sidm_sigma=s) for s in [0.1, 0.3, 1.]] 

In [ ]:
rg = [0., 4.], [2.5, 39.]

ps = np.linspace(0.2, 1., 5)
ps[-1] = 0.99
ps = ps[::-1]

In [ ]:
fig, axs = plot.subplots((2, 3), share=False, space=(0.15, .02), subsize=(5.25, 3.8),
                         margin=[0.02, 0.02, 0.075, 0.035], layout='none')
axs_f = axs.flat
sigs = 0.1, 0.3, 1.0
cmaps = ['gnuplot2', 'gnuplot2_r']
cb_labs = r'$r_{\rm c}\,[{\rm kpc}]$', r'$\log\,\rho_0\,[{\rm M}_\odot\,{\rm kpc}^{-3}]$'

for i_map, m in enumerate(sidm_maps):
    z_fs, cs, r1s, rho0s = m.z_fs, m.cs, m.r1s, m.rho0s
    lrho0s = Num.safe_lg(rho0s)
    for i_y, y in enumerate([r1s, lrho0s]):
        ax = axs[i_y, i_map]
        cb = ax._raw.contourf(z_fs, cs, y.T, levels=24,
                              cmap=cmaps[i_y],
                              zorder=1)
        lab = cb_labs[i_y] if i_map == 2 else None
        fig.colorbar(cb, ax=ax, location='right', 
                     fraction=0.2, pad=0.02, label=lab)
    
    lab = r'\sigma_{\rm m} = %.1f\,{\rm cm^2/g}'%(sigs[i_map])
    axs[0, i_map].text(lab, (.92, .08), ha='right', color='w')

z_f, c = s_h_sel.objs['z_half', 'c_mean200']
cs = ('k',)*3 + ('k',)*3
for i_ax, ax in enumerate(axs_f):
    s2d = ax.scatter_2d(z_f, c, range=rg, n_bins=32)
    s2d.contour(ps, zorder=4, colors=cs[i_ax], ls='-', lw=1.5)

axs.label(r'z_{\rm f}', r'c').label_outer()
for ax in axs[0]:
    ax._raw.set_xticklabels([])

ProjPaths.save_fig('fig_sidm_on_c_zf_plane.pdf')

#### Find r1, rho0

In [ ]:
%autoreload

Run 

In [ ]:
for i, sig in enumerate((0.1, 0.25, 0.3, 0.4, 0.5, 1.)):
    sidm = SIDMDriver(
        s_h, S_key='Sigma_star_AM_lm10p5_to_11_rho0.85', sigma_in_cm2perg=sig)
    sidm.require_rho0()
    r1_all = fill_by_hids(samp.n_objs, s_h['hid'], sidm.data['r1'])
    rho0_all = fill_by_hids(samp.n_objs, s_h['hid'], sidm.data['rho0'])

    h5.File.dump_to(p_data, {
        f'SIDM_r1_sigma{sig:.2f}': r1_all,
        f'SIDM_rho0_sigma{sig:.2f}': rho0_all,
    }, f'objs', dump_flag='ac', g_flag='ac')

In [ ]:
for i, sig in enumerate((3.,4.,5.,10.)):
    sidm = SIDMDriver(
        s_h, S_key='Sigma_star_AM_lm10p5_to_11_rho0.85', sigma_in_cm2perg=sig)
    sidm.require_rho0()
    rh4 = fill_by_hids(samp.n_objs, s_h['hid'], sidm.data['rh4'])
    rhalf = fill_by_hids(samp.n_objs, s_h['hid'], sidm.data['rhalf'])

    h5.File.dump_to(p_data, {
        f'SIDM_rh4_sigma{sig:.2f}': rh4,
        f'SIDM_rhalf_sigma{sig:.2f}': rhalf,
    }, f'objs', dump_flag='ac', g_flag='ac')

Iterative run

In [ ]:
def itered_r1(sig: float, A_r: float, rho = 1., max_n_iters=5, suf='iter'):
    # take AM R_eff as initial guess
    dr = SIDMDriver(s_h, S_key=f'Sigma_star_AM_lm10p5_to_11_rho{rho:.2f}', 
                    sigma_in_cm2perg=sig)
    s_h.objs['R_eff_iter_0'] = dr.data['R_eff']
    
    for i in range(max_n_iters):
        dr = SIDMDriver(s_h, S_key=None, R_eff_key=f'R_eff_iter_{i}',
                        sigma_in_cm2perg=sig)
        dr.require_r1()
        s_h.objs[f'R_eff_iter_{i+1}'] = dr.data['r1'] * A_r
        
    r1 = dr.data['r1']
    r1_all = fill_by_hids(samp.n_objs, s_h['hid'], r1)
    h5.File.dump_to(p_data, {
        f'SIDM_r1{suf}_sigma{sig:.2f}': r1_all,
    }, f'objs', dump_flag='ac', g_flag='ac')
    
itered_r1(sig=.1, A_r = 2.2595430708452, suf='_iterFixAr')
itered_r1(sig=.3, A_r = 1.15504705957222, suf='_iterFixAr')
itered_r1(sig=1., A_r = 0.573979442926262, suf='_iterFixAr')

In [ ]:
u_l = sim_info.cosmology.unit_system.u_l_to_pc * 1.0e-3

In [ ]:
sig = .1 #.4 #.5  #.3
rho = 1.
dr = SIDMDriver(s_h, S_key=f'Sigma_star_AM_lm10p5_to_11_rho{rho:.2f}', 
                sigma_in_cm2perg=sig)
s_h.objs['R_eff_iter_0'] = dr.data['R_eff']

In [ ]:
dr = SIDMDriver(s_h, S_key=f'Sigma_star_AM_lm10p5_to_11_rho0.85', 
                sigma_in_cm2perg=sig)
s_h.objs['R_eff_rho0.85'] = dr.data['R_eff']

In [ ]:
for i in range(5):
    dr = SIDMDriver(s_h, S_key=None, R_eff_key=f'R_eff_iter_{i}',
                    sigma_in_cm2perg=sig)
    dr.require_r1()
    s_h.objs[f'R_eff_iter_{i+1}'] = dr.data['r1']

In [ ]:
dr = SIDMDriver(s_h, S_key=None, R_eff_key=f'R_eff_iter_4',
                sigma_in_cm2perg=sig)

S_all = fill_by_hids(samp.n_objs, s_h['hid'], dr.data['S'])
h5.File.dump_to(p_data, {
    f'Sigma_star_SIDM_lm10p5_to_11_sigma{sig:.2f}': S_all,
}, f'objs', dump_flag='ac', g_flag='ac')

In [ ]:
fig, axs = plot.subplots((1,2), share=False, space=0.1, subsize=(4.5, 4.5), 
                         margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

AM_keys = 'iter_0', 'rho0.85'
SIDM_key = 'iter_4'
for i, AM_key in enumerate(AM_keys):
    ax = axs_f[i]
    r_AM, r_SIDM = s_h[f'R_eff_{AM_key}']*u_l, s_h[f'R_eff_{SIDM_key}']*u_l
    ax.fmt_marker(elw=0, s=3).scatter(r_AM, r_SIDM, rasterized=True)
    ax.scale('log', 'log').lim([.1, 10.], [.1, 9.])
    ax.plot([1.0e-2, 1.0e2], [1.0e-2, 1.0e2], c='grey', lw=1.5, ls='--')
    ax.label(y=r'R_{50}^{({\rm SIDM})}')

axs_f[0].label(r'R_{50}^{({\rm AM},\rho=1)}\,[{\rm kpc}]')
axs_f[1].label(r'R_{50}^{({\rm AM},\rho=0.85)}\,[{\rm kpc}]')
axs.label_outer()

In [ ]:
fig, axs = plot.subplots((2,2), share=False, space=0.3, subsize=(4.5, 4.5), 
                         margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

for i in range(4):
    ax = axs_f[i]
    r0, r1 = s_h[f'R_eff_iter_{i}']*u_l, s_h[f'R_eff_iter_{i+1}']*u_l
    ax.fmt_marker(elw=0, s=3).scatter(r0, r1, rasterized=True)
    ax.label(r'R_{50}^{(%s)}\,[{\rm kpc}]'%(i), 
             r'R_{50}^{(%s)}\,[{\rm kpc}]'%(i+1))
    ax.scale('log', 'log').lim([.1, 10.], [.1, 9.])
    ax.plot([1.0e-2, 1.0e2], [1.0e-2, 1.0e2], c='grey', lw=1.5, ls='--')

Check impl

In [ ]:
args = np.arange(0, s_h['hid'].size, s_h.n_objs // 10) 
s_h_test = s_h.subset(args)

In [ ]:
driver = SIDMDriver(s_h_test, S_key='Sigma_star_AM_lm10p5_to_11_rho0.85',
                  sigma_in_cm2perg=3., keep_halo=True)
driver.require_rho0()

In [ ]:
driver.data.keys()

In [ ]:
fig, ax = plot.subplots(1, figsize=4.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

hid = 9
for k, v in driver.data.items():
    if isinstance(v, np.ndarray):
        print(k, v[hid])
h = driver.data['sidm_halos'][hid]
rh4, rc, rh = driver.data['rh4'][hid], driver.data['r1'][hid], driver.data['rhalf'][hid]

h: sidm.models.StitchedHalo
pf = h.profile._impl
x, y = pf._lr_cents, pf._rho_cents
ax.plot(x, y, label=r'$\rm SIDM$', c='grey')
y0 = y[0]

pf = h.halo.profile._impl
x, y = pf._lr_cents, pf._rho_cents
ax.plot(x, y, label=r'$\rm CDM$')

cs = cs_dark2
labs = r'$r_{\rho_0/2}$', r'$r_{\rho_0/4}$', r'$r_1$'
for i, r in enumerate([rh, rh4, rc]):
    lr = np.log10(r)
    ax.plot([lr,]*2, [1., 1.0e10], c=cs[i], label=labs[i])
    
ax.scale(y='log').lim([-5., -1.],[1.0e4, y0*100.]).leg()\
    .label(r'\log\,r\,[h^{-1}{\rm Mpc}]', 
           r'\rho\,[{\rm intern.\ U.}]')

In [ ]:
args = np.arange(0, s_h['hid'].size, 30) 
s_h_test = s_h.subset(args)

In [ ]:
sigmas = 1.,3.,5.,7.,9.,10.,20.
rh4s = []
for sigma in sigmas:
    sidm = SIDMDriver(
    s_h_test, S_key='Sigma_star_AM_lm10p5_to_11_rho0.85', sigma_in_cm2perg=sigma)
    sidm.require_rho0()
    rh4 = sidm.data['rh4']
    rh4s.append(rh4)

In [ ]:
d_obs = DataDict()

x_lim = [-.425, 0.875]

lr_e = np.log10(obs_catalogs.tot_dwarfs.s_massive['r_50'])
lS = Num.safe_lg(obs_catalogs.tot_dwarfs.s_massive['Sigma_star']) 

h, e = np.histogram(lr_e, range=x_lim, bins=32, density=True)
c = .5 * (e[:-1] + e[1:])
d_obs['hist'] = DataDict({
    'h': h, 'e': e, 'c': c
})

x_p = np.linspace(*np.quantile(lr_e, [0.001, 0.999]), 16)
y_p, el, eh = stats.KernelRegression1D.by_local_kernel(lr_e, lS, x_p, reduce='errorbar', max_dx=.15)['y'][0].T
d_obs['lr2lS'] = DataDict({
    'x': x_p, 'y': y_p, 'err_lo': el, 'err_up': eh
})

In [ ]:
us = sim_info.cosmology.unit_system
u_l = us.u_l_to_pc * 1.0e-3
u_m = us.u_m_to_sol

S, z_f = s_h_test['Sigma_star_AM_lm10p5_to_11_rho0.85', 'z_half']
lS = Num.safe_lg(S)

d_sidm = DataDict()
for i, rh4 in enumerate(rh4s):
    lr = Num.safe_lg(rh4 * u_l)
    h, e = np.histogram(lr, bins=32, range=x_lim, density=True)
    c = 0.5 * (e[:-1] + e[1:])
    d_sidm[str(i)] = DataDict({
      'hist': DataDict({'h': h, 'e': e, 'c': c})
    })
    
    x_p = np.linspace(*np.quantile(lr, [0.001, 0.999]), 128)
    dx = (x_p[-1] - x_p[0])/24
    y_p, el, eh = stats.KernelRegression1D.by_local_kernel(
        lr, lS, x_p, reduce='errorbar', max_dx=dx)['y'][0].T
    d_sidm[str(i)]['lr2lS'] = DataDict({
        'x': x_p, 'y': y_p, 'e': (el, eh),
    }) 

In [ ]:
fig, axs = plot.subplots((1,2), share=False, space=0.3, 
                         subsize=(5.5, 4.5), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

ax = axs_f[0]
x, y = d_obs['hist']['c', 'h']
ax.plot(x, y, lw=2, c='k', zorder=21,
        label=r'${\rm Dwarfs},\,R_{\rm 50}$')

for i, sigma in enumerate(sigmas):
    d = d_sidm[str(i)]
    x, y = d['hist']['c', 'h']
    ax.plot(x, y, lw=2, c=cs_set1[i], label=r'$\sigma_{\rm m}=%.1f\,{\rm cm^2/g}$'%(sigma))

ax = axs_f[1]
x, y, el, eh = d_obs['lr2lS']['x', 'y', 'err_lo', 'err_up']
ax.c('k').fmt_marker(fa=.3, elw=1)\
    .errorbar(x, y, yerr=(el, eh), marker='s', mew=1.5, ms=5.5, capsize=0, lw=1,
              label=r'${\rm Dwarfs}\,(\Sigma_*)$', zorder=21)
    
for i, sigma in enumerate(sigmas):
    d = d_sidm[str(i)]
    x, y, e = d['lr2lS']['x', 'y', 'e']
    ax.plot(x, y, lw=2.5, c=cs_set1[i])
    
axs[0].lim(x_lim, [0., 5.85]).label(r'\log\,r_{\rho_0/4}\,({\rm kpc})', r'\rm PDF')\
    .leg(loc='ul', 
         #labelcolor='linecolor', 
         fontsize=13.5,
         handlelength=1.75, labelspacing=0., ncol=1, columnspacing=0.5)
axs[1].lim(
    x_lim, [.415, 2.76]).label(
    r'\log\,r_{\rho_0/4}\,({\rm kpc})', r'\log\,\Sigma_*\,({\rm M_\odot pc^{-2}})')

In [ ]:
sidm = SIDMDriver(
    s_h_test, S_key='Sigma_star_AM_lm10p5_to_11_rho0.85', sigma_in_cm2perg=3.)
sidm.require_rho0()

In [ ]:
sidm2 = SIDMDriver(
    s_h_test, S_key='Sigma_star_AM_lm10p5_to_11_rho0.85', sigma_in_cm2perg=.3)
sidm2.require_rho0()

In [ ]:
sidm3 = SIDMDriver(
    s_h_test, S_key='Sigma_star_AM_lm10p5_to_11_rho0.85', sigma_in_cm2perg=30.)
sidm3.require_rho0()

In [ ]:
r1, rhalf, rh4, rho0, rho1 = sidm.data['r1', 'rhalf', 'rh4', 'rho0', 'rho1']
r1_2 = sidm2.data['r1']
r1_3 = sidm3.data['rhalf']

In [ ]:
fig, axs = plot.subplots((1,2), share=False)
axs_f = axs.flat

ax = axs_f[0]
x1, x2, x3 = Num.safe_lg(r1), Num.safe_lg(rhalf), Num.safe_lg(rh4)
x4 = Num.safe_lg(r1_2)
x5 = Num.safe_lg(r1_3)
ax.hist(x1, bins=32, density=True)
ax.hist(x2, bins=32, density=True)
ax.c('b').hist(x3, bins=32, density=True)
ax.c('r').hist(x4, bins=32, density=True)
ax.c('g').hist(x5, bins=32, density=True)
ax.lim([-4., -2.])

x1, x2 = Num.safe_lg(rho0), Num.safe_lg(rho1)
ax = axs_f[1]
ax.c('k').hist(x1)
ax.c('r').hist(x2)

In [ ]:
fig, ax = plot.subplots(1, figsize=4.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

ax.plot(lrs, rhos, lw=2.5)

ax.scale(y='log')

In [ ]:
sidm.data

#### Randomly draw c

In [ ]:
info_ref = sims.predefined['tng_dark_300_1']
sman_ref = SubSampleManager(
    info_ref, 
    ProjPaths.sim_dir_of(info_ref) / 'z0_all_postprocessed.hdf5', 
    add_rsd= False, 
    dmo= True,
    m_h_range_in_sol= (10**10.5, 10**11.),
)
s_ref_all = sman_ref.s_all
s_ref_all.objs['hid'] = np.arange(s_ref_all.n_objs, dtype=np.int64)

s_ref = sman_ref.s_dwarf

In [ ]:
def find_b(s: sample.SimSample, sman: SubSampleManager, r_range=(2., 10)):
    x = s['x']
    print(x.shape[0], 'objs, range=', x.min(), x.max()) 
    
    s_clsref = sman.s_ref
    x_clsref = s_clsref['x']
    print(x_clsref.shape[0], 'clsref objs, range=', x_clsref.min(), x_clsref.max())
    print()
    
    l_box = sman.sim_info.full_box_size
    kdt_clsref = KDTree(x_clsref, boxsize=l_box)
    
    r1, r2 = r_range
    n1 = kdt_clsref.query_ball_point(x, r1, return_length=True)
    n2 = kdt_clsref.query_ball_point(x, r2, return_length=True)
    dn = n2 - n2
    print('dn found, size=', dn.size)
    
    dV = 4.0/3.0 * np.pi * (r2**3 - r1**3)
    rho_n = len(x_clsref) / l_box**3
    dn_exp = dV * rho_n
    xi = dn / dn_exp - 1.0
    
    return xi

xi = find_b(s_ref, sman_ref)
xi_all = fill_by_hids(s_ref_all.n_objs, s_ref['hid'], xi)

In [ ]:
h5.File.dump_to(p_data, c_all,
                f'objs/c_mean200_AM_lm10p5_to_11',
                dump_flag='ac')

Use $p(c | z_{\rm f})$

In [ ]:
def cond_match(z_fs, cs, z_f_dsts, rng = 2):
    assert len(z_fs) == len(cs)
    kdt = KDTree(z_fs[:, None])
    inds = kdt.query(z_f_dsts[:, None], k=128)[1]
    
    rng = Rng(rng)
    c_dsts = np.zeros_like(z_f_dsts)
    for i, ind in enumerate(inds):
        ind_sel = rng.choice(ind)
        c_dsts[i] = cs[ind_sel]
    return c_dsts

In [ ]:
z_fs, cs = s_ref.objs['z_half', 'c_mean200']
z_f_dsts = s_h.objs['z_half']
c_dsts = cond_match(z_fs, cs, z_f_dsts)

In [ ]:
c_all = np.zeros(len(samp['hid']), dtype=c_dsts.dtype)
c_all[ s_h['hid'] ] = c_dsts
h5.File.dump_to(p_data, c_all,
                f'objs/c_mean200_AM_lm10p5_to_11',
                dump_flag='ac')

In [ ]:
fig, ax = plot.subplots(1, figsize=4.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

ax.scatter_2d(z_f_dsts, c_dsts, range=[[0,4.], [2., 45.]], n_bins=32).contour()
ax.scatter_2d(z_fs, cs, range=[[0,4.], [2., 45.]], n_bins=32).contour()

#### Match $z_{\rm f}$ or $r_1$ to $\Sigma_*$

In [ ]:
# s_g = obs_catalogs.tot_dwarfs.s_massive
s_g = obs_catalogs.tot_dwarfs.s_massive_z003
s_g, len(s_g['z'])

In [ ]:
# match z_f and r_1
rng = Rng(3)
rn = stats.RandomNoise(.001, rng=rng)
lS =  rn.add_to(np.log10(s_g['Sigma_star'])) 
rho = 1.
for sig in [.1, .5, 1.]: 
    key = f'SIDM_r1_sigma{sig:.2f}'
    lr1 = rn.add_to(np.log10(s_h[key]))
    
    lS_pred = u_stats.abudance_match_with_corr_coef(lr1, lS, rho=-rho, rng=rng)
    S_all = fill_by_hids(samp.n_objs, s_h['hid'], 10**lS_pred)
    h5.File.dump_to(p_data, S_all, 
                    f'objs/Sigma_star_AM_lm10p5_to_11_{key}', dump_flag='ac')

In [ ]:
# match z_f and Sigma_star

rng = Rng(10086)
rn = stats.RandomNoise(.001, rng=rng)
lS =  rn.add_to(np.log10(s_g['Sigma_star'])) 
lz_f = rn.add_to(np.log10(s_h['z_half'] + 1.))

# old impl
# lS_pred = -u_stats.abundance_match(lz_f, -lS)
# S_pred = 10**lS_pred

# 0.99, 0.95, 0.7
for rho in [1., 0.9, 0.85, 0.8, 0.6, 0.5]:  
    lS_pred = u_stats.abudance_match_with_corr_coef(lz_f, lS, rho=-rho, rng=rng)
    S_pred = 10**lS_pred
    
    S_all = np.zeros(len(samp['hid']), dtype=S_pred.dtype)
    S_all[ s_h['hid'] ] = S_pred
    h5.File.dump_to(p_data, S_all, 
                    #'objs/Sigma_star_AM_lm10p8', 
                    #f'objs/Sigma_star_AM_lm10p5_to_11_rho{rho:.2f}',
                    f'objs/Sigma_star_AM_lm10p5_to_11_z001_to_003_rho{rho:.2f}',
                    #'objs/Sigma_star_AM_lm10p5_to_11_z004_to_006' ,
                    #'objs/Sigma_star_AM_lm10p5_to_11_z006_to_007' ,
                    #f'objs/Sigma_star_AM_lm10p5_to_11_z001_to_006_rho{rho:.2f}',
                    dump_flag='ac')

In [ ]:
h5.File.ls_from(p_data)

### SIDM results

In [ ]:
sim_info = sims.predefined['elucid_ext_v2']
p_data = ProjPaths.sim_dir_of(sim_info) / 'z0_all.hdf5'
samp = sample.SimSample.load(sim_info, p_data)
samp.objs['hid'] = np.arange(samp.n_objs, dtype=np.int64)

In [ ]:
u_m = sim_info.cosmology.unit_system.u_m_to_sol
m_lb, m_ub = 10**10.5 / u_m, 10**11.0 / u_m
z_lb, z_ub = 0.01, 0.04

s_h = samp.subset_by_value('is_c', eq=True)\
    .subset_by_value('m_mean200', lo=m_lb, hi=m_ub)\
    .subset_by_value('last_sat_z', lo=15.)\
    .subset_by_value('is_recon_large', eq=True)\
    .subset_by_value('z', lo=z_lb, hi=z_ub)
s_h

In [ ]:
s_h = sman.s_dwarf

In [ ]:
with h5.File(sim_dir.base_dir / 'samples/lm10p5-11.hdf5', 'w') as f:
    s_h.to_h5_group(f)

In [ ]:
def f_line(P, x):
    return P[0] * x + P[1]

def odr_fit(x, y):
    X = np.column_stack([x, y])
    X_mean, X_sd = X.mean(), X.std()
    X_scl = (X - X_mean) / X_sd
    
    x, y = X_scl.T
    #from sklearn.linear_model import LinearRegression
    from scipy.odr import ODR, Model, Data
    model = Model(f_line)
    data = Data(x, y)
    odr = ODR(data, model, beta0=[1, 0])
    out = odr.run()
    k, b = out.beta
    #lr = LinearRegression().fit(x[:, None], y)
    #k, b = lr.coef_, lr.intercept_
    
    x_lo, x_hi = np.quantile(x, [0.01, 0.99])
    x_p = np.linspace(x_lo, x_hi, 16)
    y_p = k * x_p + b
    X_p = np.column_stack([x_p, y_p])
    X_p = X_p * X_sd + X_mean
    x_p, y_p = X_p.T
    
    return DataDict({
        'x_p': x_p, 'y_p': y_p, 'k': k, 'b': b,
    })

#### Sigma_* predicted analytically

In [ ]:
us = sim_info.cosmology.unit_system
u_l = us.u_l_to_pc * 1.0e-3
u_m = us.u_m_to_sol

keys = 'sigma0.10', 'sigma0.30', 'sigma1.00'
outs = []
for i_k, k in enumerate(keys):
    r1, rho0 = s_h[f'SIDM_r1_{k}'] * u_l, s_h[f'SIDM_rho0_{k}'] * u_m / u_l**3
    S_true = s_h['Sigma_star_AM_lm10p5_to_11_rho0.85'] 
    S_mean = S_true.mean()
    z_f = s_h['z_half']
    S_1 = (r1 / r1.mean()) **(-2 ) * S_mean
    S_2 = (rho0 / rho0.mean()) **(2./3.) * S_mean
    out = DataDict({
        'z_f': z_f, 'S_1': S_1, 'S_2': S_2,
    })
    outs.append(out)

In [ ]:
fig, axs = plot.subplots((1,2), share=True, 
                         space=(0.03, 0), 
                         subsize=(5.5, 4.5), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

ax = axs_f[0]
lws = 2, 2, 2
lss = (0, (3,1.5)), '-', (0,(1,1))
_as = .075, .2, .075
sigs = 0.1, 0.3, 1.0
for i_k, out in enumerate(outs):
    z_f, S_1, S_2 = out['z_f', 'S_1', 'S_2']
    for i_S, S in enumerate((S_1, S_2)):
        ax = axs_f[i_S]
        x, y = S, z_f  
        y_p = np.linspace(0., 4.5, 128)
        x_p, el, eh = stats.KernelRegression1D.by_local_kernel(
            y, x, y_p, reduce='errorbar', max_dx=.125)['y'][0].T
        #ax.scatter(S, z_f, label=f'S_{i_S+1}', alpha=.5)
        lab = r'$\sigma_{\rm m} = %.1f\,{\rm cm^2/g}$'%(sigs[i_k])
        ax.plot(x_p, y_p, lw=lws[i_k], ls=lss[i_k], label=lab)
        ax._raw.fill_betweenx(y_p, x_p-el, x_p+eh, color='k', 
                              alpha=_as[i_k], lw=0)
   
axs.scale('log').lim([2., 10**2.55], [0., 3.99])
axs[0].label(r'\Sigma_* \propto r_{\rm c}^{-2}', r'z_{\rm f}')
axs[1].label(r'\Sigma_*\propto \rho_{\rm 0}^{2/3}')
axs.label_outer()
axs[0].leg(loc='ll')

#### r_c matched Sigma*

In [ ]:
us = sim_info.cosmology.unit_system
u_l = us.u_l_to_pc * 1.0e-3
u_m = us.u_m_to_sol

d_plts = DataDict({
    'bias': [],
    'scat': []
})
prop_pref = 'Sigma_star_AM_lm10p5_to_11'
keys = 'sigma0.10', 'sigma0.50', 'sigma1.00'
for i_k, k in enumerate(keys):
    x_key = f'SIDM_r1_{k}'
    prop_key = f'{prop_pref}_{x_key}'
    
    d_in = Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/am/{prop_key}.hdf5')
    wps = take_wps_rels(d_in['obs_bin'])
    d_plts['bias'].append(wps)
    
    r1 = s_h[x_key] * u_l
    lr1 = Num.safe_lg(r1)
    S = s_h[prop_key]
    lS = Num.safe_lg(S)
    x, y = S, r1
    y_p = np.linspace(0., 7., 128)
    x_p, el, eh = stats.KernelRegression1D.by_knn(
        y, x, y_p, reduce='errorbar', k=4)['y'][0].T
    
    d_plts['scat'].append(DataDict({
        'r1': r1, 'lr1': lr1, 'S': S, 'lS': lS,
        'x_p': x_p, 'y_p': y_p, 'el': el, 'eh': eh,
    }))
    
d_plts['bias_obs'] = np.array([
    [6.2,    2.4, 0.28, 0.28,],
    [11.37,  1.44, 0.1, 0.1,],
    [19.33,  1.29, 0.09, 0.09,],
    [42.37, 1.0, 0.0, 0.0,],
])

In [ ]:
fig, axs = plot.subplots((2, 1), share=(True, False),
                         space=0.03, subsize=(6.5, 3.5),
                         margin=[0.02, 0.125, 0.1, 0.135], layout='none',
                         ratios=(None, [2, 1.2]))
axs_f = axs.flat

ax = axs_f[0]
sigs = 0.1, 0.5, 1.0
lws = 2, 2, 2
lss = '-', (0, (3, 1.5)), (0, (1, 1))
for i in range(3):
    ax = axs_f[0]
    ax.c('k')
    x, y, el, eh = d_plts['bias'][i].T
    lab = r'$\sigma_{\rm m} = %.1f\,{\rm cm^2/g}$' % (sigs[i])
    ax.errorbar(x, y, yerr=(el, eh), lw=lws[i], ls=lss[i],
                marker='s',
                mew=2, ms=5.5, capsize=0, elinewidth=1.5,
                label=lab, zorder=10)
    
    ax = axs_f[1]
    ax.c('k')
    x_p, y_p, el, eh = d_plts['scat'][i]['x_p', 'y_p', 'el', 'eh']
    ax.plot(x_p, y_p, lw=lws[i], ls=lss[i], zorder=3)
    #ax.errorfill(x_p, y_p, yerr=(el, eh), lw=1., ls='-',
    #             fill_between_kw={'lw': 0., 'alpha': .2}, zorder=9)
    
    y, x = d_plts['scat'][i]['lr1', 'S']
    #ax.plot(x, y)

ax = axs_f[0]
x, y, el, eh = d_plts['bias_obs'].T
ax.c(cs[i]).errorbar(x, y, yerr=(el, eh), lw=.75, marker='o',
                     mew=2, ms=8, capsize=0, elinewidth=1.5,
                     label=r'$\rm Observation$', zorder=10)

ax.scale(x='log').lim([10**0.45, 1.0e2], [0.85,  2.75])
ax.label(y=r'{\rm Relative\ bias}')
ax.leg(loc='ur', labelcolor='linecolor',
       handlelength=1.75, numpoints=1, fontsize=14.5)

ax = axs_f[1]
ax.label(r'\Sigma_*\,[{\rm M}_\odot\,{\rm pc}^{-2}]', r'r_{\rm c}\,[{\rm kpc}]')\
    .lim(y=[0., 6.5])

#### r_c predicted

In [ ]:
x_lim = [-.425, 0.875]
us = sim_info.cosmology.unit_system
u_l = us.u_l_to_pc * 1.0e-3
u_m = us.u_m_to_sol
dn_pts = 10

d_plts = {
    'panel_a': {},
    'panel_b': {},
    'panel_c': {'curve': {}, 'scatter': {}},
    'panel_d': {'curve': {}, 'scatter': {}},
}
keys = 'sigma0.10', 'sigma0.30', 'sigma1.00', 'sigma3.00'
key_rs = 'SIDM_r1', 'SIDM_r1', 'SIDM_r1', 'SIDM_rh4'
key_outs = 'sigma_m_0p1', 'sigma_m_0p3', 'sigma_m_1p0', 'sigma_m_3p0'
for i_k, (k, key_r, key_out) in enumerate(zip(keys, key_rs, key_outs)):    
    r1 = s_h[f'{key_r}_{k}'] * u_l
    lr1 = Num.safe_lg(r1)
    S, z_f = s_h['Sigma_star_AM_lm10p5_to_11_rho0.85', 'z_half']
    lS = Num.safe_lg(S)
    if i_k == 1:
        lrho0 = Num.safe_lg(s_h[f'SIDM_rho0_{k}'] * u_m / u_l**3)
    
    h, e = np.histogram(lr1, bins=32, range=x_lim, density=True)
    c = 0.5 * (e[:-1] + e[1:])
    d_plts['panel_a'][key_out] = pd.DataFrame({
        'log_r': c, 'pdf': h,
    })
    
    d_in = Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/am/{key_r}_{k}.hdf5')['obs_inv_bin']
    x, y, el, eh = take_wps_rels(d_in, d_in['0']).T
    x = np.log10(x * u_l)
    d_plts['panel_b'][key_out] = pd.DataFrame({
        'log_r': x, 'bias': y, 'err_lo': el, 'err_up': eh,
    })
    
    
    x, ys, y_keys, o_pans = lr1, [z_f, lS], ['z_f', 'log_Sigma'], ['panel_c', 'panel_d']
    x_p = np.linspace(*np.quantile(x, [0.001, 0.999]), 128)
    dx = (x_p[-1] - x_p[0])/24
    for y, y_key, o_pan in zip(ys, y_keys, o_pans):
        y_p, el, eh = stats.KernelRegression1D.by_local_kernel(
            x, y, x_p, reduce='errorbar', max_dx=dx)['y'][0].T
        d_plts[o_pan]['curve'][key_out] = pd.DataFrame({
            'log_r': x_p, y_key: y_p, 'err_lo': el, 'err_up': eh,  
        })
        
        if i_k != 1:
            continue
        
        z = lrho0
        X = np.column_stack([x, y])
        X_mean, X_sd = X.mean(0), np.std(X, axis=0)
        X_sc = (X - X_mean) / X_sd
        z_p = stats.KernelRegressionND.by_knn(X_sc, z, X_sc, k=128)['y'][0]
        d_plts[o_pan]['scatter'][key_out] = pd.DataFrame({
            'log_r': x[::dn_pts], y_key: y[::dn_pts], 'log_rho_0': z_p[::dn_pts],
        })
        
    '''
    d_plt |= {
        'r1': r1, 'lr1': lr1,
        'S': S, 'lS': lS,
        'z_f': z_f,
    }
    d_plt |= {'rho0': rho0, 'lrho0': lrho0,}
    '''

lr_e = np.log10(obs_catalogs.tot_dwarfs.s_massive['r_50'])
lS = Num.safe_lg(obs_catalogs.tot_dwarfs.s_massive['Sigma_star']) 

h, e = np.histogram(lr_e, range=x_lim, bins=32, density=True)
c = .5 * (e[:-1] + e[1:])
d_plts['panel_a']['obs'] = pd.DataFrame({'log_r': c, 'pdf': h})

x_p = np.linspace(*np.quantile(lr_e, [0.001, 0.999]), 16)
y_p, el, eh = stats.KernelRegression1D.by_local_kernel(lr_e, lS, x_p, reduce='errorbar', max_dx=.15)['y'][0].T
d_plts['panel_d']['obs'] = pd.DataFrame({'log_r': x_p, 'log_Sigma': y_p,  'err_lo': el, 'err_up': eh})

In [ ]:
# Excel.dump_file(ProjPaths.proj_dir/'data_for_publication/Fig4.xlsx', d_plts)

In [ ]:
keys = 'sigma0.10', 'sigma0.30', 'sigma1.00'
key_rs = ('SIDM_r1_iterFixAr', )*3
key_outs = 'iter_sigma_m_0p1', 'iter_sigma_m_0p3', 'iter_sigma_m_1p0'
for i_k, (k, key_r, key_out) in enumerate(zip(keys, key_rs, key_outs)):    
    r1 = s_h[f'{key_r}_{k}'] * u_l
    lr1 = Num.safe_lg(r1)
    S, z_f = s_h[f'Sigma_star_AM_lm10p5_to_11_rho0.85', 'z_half']
    lS = Num.safe_lg(S)
    
    h, e = np.histogram(lr1, bins=32, range=x_lim, density=True)
    c = 0.5 * (e[:-1] + e[1:])
    d_plts['panel_a'][key_out] = pd.DataFrame({
        'log_r': c, 'pdf': h,
    })
    
    d_in = Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/am/{key_r}_{k}.hdf5')['obs_inv_bin']
    x, y, el, eh = take_wps_rels(d_in, d_in['0']).T
    x = np.log10(x * u_l)
    d_plts['panel_b'][key_out] = pd.DataFrame({
        'log_r': x, 'bias': y, 'err_lo': el, 'err_up': eh,
    })
    
    
    x, ys, y_keys, o_pans = lr1, [z_f, lS], ['z_f', 'log_Sigma'], ['panel_c', 'panel_d']
    x_p = np.linspace(*np.quantile(x, [0.001, 0.999]), 128)
    dx = (x_p[-1] - x_p[0])/24
    for y, y_key, o_pan in zip(ys, y_keys, o_pans):
        y_p, el, eh = stats.KernelRegression1D.by_local_kernel(
            x, y, x_p, reduce='errorbar', max_dx=dx)['y'][0].T
        d_plts[o_pan]['curve'][key_out] = pd.DataFrame({
            'log_r': x_p, y_key: y_p, 'err_lo': el, 'err_up': eh,  
        })

In [ ]:
fig, axs = plot.subplots(
    (2, 2),
    share=(False, False),
    space=(0.2, .02),
    figsize=(11., 6.5),
    margin=[0.01, 0.01, 0.08, 0.065],
    layout='none')
axs = axs.flat[[0, 2, 1, 3]]

x_lim = [-.425, 0.875]

lws = 2.5, 2.5, 2.5, 2.5
lss = (0, (3, 1.5)), '-', (0, (1, 1)), '-'
sigs = 0.1, 0.3, 1.0, 3.0
c_outs = 'k', 'k', 'k', 'grey'
mss = 6., 6., 6., 5.
_as = .1, .2, .1, 0.
key_outs = 'sigma_m_0p1', 'sigma_m_0p3', 'sigma_m_1p0', 'sigma_m_3p0'
zorders = 20, 22, 18, 17

for i_k, key_out in enumerate(key_outs):
    c_out = c_outs[i_k]

    ax = axs[0]
    if i_k < 3:
        lab = r'$\sigma_{\rm m} = %.1f\,{\rm cm^2/g}$' % (sigs[i_k])
    else:
        lab = r'$\sigma_{\rm m} = %.1f\,{\rm cm^2/g},\,r_{\rho_{\rm 0}/4}$' % (
            sigs[i_k])
    x, y = d_plts['panel_a'][key_out][['log_r', 'pdf']].to_numpy().T
    ax.plot(
        x, y, lw=lws[i_k],
        ls=lss[i_k],
        c=c_out, label=lab, zorder=zorders[i_k])
    if i_k < 3:
        x, y = d_plts['panel_a']['iter_'+key_out][['log_r', 'pdf']].to_numpy().T
        ax.plot(
            x, y, lw=lws[i_k],
            ls=lss[i_k],
            c='b', zorder=zorders[i_k]+1)
    

    ax = axs[1]
    x, y, el, eh = d_plts['panel_b'][key_out][
        ['log_r', 'bias', 'err_lo', 'err_up']].to_numpy().T
    ax.c(c_out).errorbar(
        x, y, yerr=(el, eh),
        lw=lws[i_k],
        marker='s', ms=mss[i_k],
        mew=1., ls=lss[i_k],
        capsize=2.5, elinewidth=1.5, zorder=zorders[i_k])
    if i_k < 3:
        x, y, el, eh = d_plts['panel_b']['iter_'+ key_out][
            ['log_r', 'bias', 'err_lo', 'err_up']].to_numpy().T
        ax.c('b').errorbar(
            x, y, yerr=(el, eh),
            lw=lws[i_k],
            marker='s', ms=mss[i_k],
            mew=1., ls=lss[i_k],
            capsize=2.5, elinewidth=1.5, zorder=zorders[i_k]+1)

    for i_y, (y_k, pan) in enumerate([
        ('z_f', 'panel_c'), ('log_Sigma', 'panel_d')]):
        ax = axs[i_y+2]
        ax.c(c_out)
        x, y, el, eh = d_plts[pan]['curve'][key_out][
            ['log_r', y_k, 'err_lo', 'err_up']].to_numpy().T
        kw = {'zorder': zorders[i_k]}
        ax.errorfill(x, y, yerr=(el, eh), lw=lws[i_k], ls=lss[i_k],
                     fill_between_kw={'lw': 0., 'fc': (0, 0, 0, _as[i_k]),
                                      'ec': 'gray',
                                      'zorder': 8}, **kw)
        if i_k == 3:
            ax.plot(x, y, lw=3., c='w', zorder=zorders[i_k]-1)
            
        if i_k < 3:
            ax.c('b')
            x, y, el, eh = d_plts[pan]['curve']['iter_'+key_out][
                ['log_r', y_k, 'err_lo', 'err_up']].to_numpy().T
            kw = {'zorder': zorders[i_k]+1}
            ax.plot(x, y, lw=lws[i_k], ls=lss[i_k],**kw)
            
        if i_k != 1:
            continue

        norm = plot.abc.mpl.colors.Normalize(7.8, 8.95)
        x, y, z = d_plts[pan]['scatter'][key_out][
            ['log_r', y_k, 'log_rho_0']].to_numpy().T
        ax.fmt_marker(elw=0, fa=1)
        ax.scatter(x, y, rasterized=True, zorder=1, c='grey', s=40)
        cb = ax.scatter(x, y, rasterized=True, zorder=2, c=z,
                        s=25, norm=norm, cmap='Greens').last_draw

fig.colorbar(cb, ax=axs, location='right', pad=0.01, aspect=40.,
             label=r'$\log\, \rho_0\,({\rm M_\odot kpc^{-3}})$')

ax = axs[0]
x, y = d_plts['panel_a']['obs'][['log_r', 'pdf']].to_numpy().T
ax.plot(x, y, lw=2, c=cs_dark2[1], zorder=21,
        label=r'${\rm Dwarfs},\,R_{\rm 50}$')

ax = axs[3]
x, y, el, eh = d_plts['panel_d']['obs'][
    ['log_r', 'log_Sigma', 'err_lo', 'err_up']].to_numpy().T
ax.c(cs_dark2[1]).fmt_marker(fa=.3, elw=1)\
    .errorbar(x, y, yerr=(el, eh), marker='s', mew=1.5, ms=5.5, capsize=0, lw=1,
              label=r'${\rm Dwarfs}\,(\Sigma_*)$', zorder=21)

axs[0].lim(x_lim, [0., 5.85]).label(y=r'\rm PDF')\
    .leg(loc='ul', 
         #labelcolor='linecolor', 
         fontsize=13.5,
         handlelength=1.75, labelspacing=0., ncol=1, columnspacing=0.5)
axs[1].scale().lim(x_lim, [0.7, 2.8])\
    .label(r'\log\,r_{\rm c}\,({\rm kpc})', r'{\rm Relative\ bias}')\
    .plot([-10., 10.], [1., 1.], c='grey', lw=1, ls=(0,(1,2)))
axs[2].lim(x_lim, [-.2, 3.99]).label(y=r'z_{\rm f}')
axs[3].lim(
    x_lim, [.415, 2.76]).label(
    r'\log\,r_{\rm c}\,({\rm kpc})', r'\log\,\Sigma_*\,({\rm M_\odot pc^{-2}})')
axs[0]._raw.set_xticklabels([])
axs[2]._raw.set_xticklabels([])

ProjPaths.save_fig('sidm_results_with_iter.pdf')

### Halo domain fields

In [ ]:
info = sims.predefined['elucid']

In [ ]:
f_in = info.root_dir / 'supplementary' / 'reconstruction_pipeline' / 'fields.hdf5'
field_raw = h5.File.load_from(f_in)['HaloDomain/RSDCorrected/CICCorrected/smoothed_r0']
rc_mask = ReconstAreaMaskSdssL500(mask_name='SDSS_reconstruction_area_large')
field_raw[~rc_mask.mask_reconst_area] = 1.
field_raw = np.array(field_raw, dtype=np.float64)

In [ ]:
l_box = info.full_box_size
n_grids = field_raw.shape[0]
mesh = cubic_box.Mesh.new(n_grids, l_box)
df = cubic_box.Field.new_by_data(field_raw, mesh=mesh)

In [ ]:
for r_sm in 1,2,4:
    suf = f'sm{r_sm}'
    tf = cubic_box.TidalField(n_workers=8, r_sm=float(r_sm), correct_shape=False)
    res = tf.run(df)
    f_out = ProjPaths.sim_dir_of(info) / 'fields' / f'domain.tidal.s99.{suf}.hdf5'
    with h5.File(f_out, 'w') as f:
        res.dump(f)

In [ ]:
fig, ax = plot.subplots(1, figsize=4.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

im = rc_mask.mask_reconst_area[:, :, 250:251].max(axis=-1)
ax._raw.imshow(im)

In [ ]:
fig, ax = plot.subplots(1, figsize=10, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

im = f[:, :, 250:251].mean(axis=-1)
ax._raw.imshow(im, norm='log', vmin=1.0e-2, vmax=1.0e3)

### Add_masks

In [ ]:
sim_info = sims.predefined['elucid_ext_v2']
p_data = ProjPaths.sim_dir_of(sim_info) / 'z0_all.hdf5'

In [ ]:
rc_mask = ReconstAreaMaskSdssL500(mask_name='SDSS_reconstruction_area_large')

In [ ]:
with h5.File(p_data, 'ac') as f:
    dsets = f['objs'].datasets
    is_recon = rc_mask.is_in_reconst_area(dsets['x'])
    dsets.dump({
        'is_recon_large': is_recon    
    }, flag='ac')

In [ ]:
h5.File.ls_from(p_data)

### Cosmic Variance

In [ ]:
lm_h_edges = np.array([
    [11.1, 11.11],
    [12.1, 12.2], 
    [13.0, 16.0]
])

pdata = ProjPaths.sim_dir_of(sim_info) / 'z0_all.hdf5'
man_kw = {'add_rsd': False, 'dmo': True}
wp_kw = {'pimax': 10., 'n_repeat': 10, 
         'bootstrap_kw': {'keep_samples': True}
         }

sman = SubSampleManager(sim_info, pdata, **man_kw)
sim_cross = SimCross(sman)

u_m = 1.0e10 / sman.sim_info.cosmology.hubble
m_h_edges = 10.0**np.array(lm_h_edges) / u_m

wp_h = {
    str(i): sim_cross.wp_of('s_halo', sub_key='m_mean200', 
        qs=m_h_edge, **wp_kw)
    for i, m_h_edge in enumerate(m_h_edges)
}
data_allvol = DataDict({
    #'dwarfs': {
    #    'z_f': wp_z_f, 'spin': wp_spin, 'tot': wp_tot,    
    #},
    'halos': wp_h,
})

In [ ]:
wp_h

In [ ]:
#Projected, in redshift space

p_data = ProjPaths.sim_dir_of(sim_info) / 'z0_all.hdf5'
p_wc = ProjPaths.obs_dir / 'weight_curve/diffuse_dwarf_z_dis'
rc_samp = ReconSample(sim_info, p_data, p_wc, 
    m_h_range_in_sol=(10**10.5, 10**11.),
    rc_mask='is_recon_large')

In [ ]:
obspc = ObsProjCross(rc_samp)

In [ ]:
u_m = 1.0e10 / sim_info.cosmology.hubble
s_h = obspc.sub_sample_of('s_halo', 'm_mean200', qs=[1.0e13/u_m, 1.0e16/u_m])

In [ ]:
fig, axs = plot.subplots((2,2), share=False, space=0.25, subsize=(5.0, 5.0), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

ax = axs_f[0]
for i in range(10):
    re_s_h = obspc.resample_match_z(s_h)
    z = re_s_h['z']

    ax.hist(z, range=(0.0, 0.12), bins=32, fc='none')
    
re_s_h = obspc.resample_match_z(s_h)
ra, dec, z = re_s_h['ra', 'dec', 'z']
ax = axs_f[1]
ax.scatter(ra, dec, c=z, s=1.0, cmap='viridis')

ax = axs_f[2]
ax.scatter(ra, z, c=z, s=1.0, cmap='viridis')

ax = axs_f[3]
ax.scatter(dec, z, c=z, s=1.0, cmap='viridis')

In [ ]:
u_m = 1.0e10 / sim_info.cosmology.hubble
s_h = rc_samp.s_halo.subset_by_value('m_mean200', lo=10**12.0/u_m)\
    .subset_by_value('z', lo=0.02, hi=0.03)

In [ ]:
rc_samp

In [ ]:
ps_list = [0.98, 1.00], [0.49, 0.51], [0.00, 0.02]
s_zs = [
    obspc.sub_sample_of('s_dwarf', sub_key='z_half', ps=ps).subset_by_value('z', 0.02, 0.03)
    for ps in ps_list
]

In [ ]:
fig, axs = plot.subplots((1,3), share=True, space=0, subsize=(4.5, 4.5), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

deg2rad = np.pi / 180.
cs = 'b', 'purple', 'r'
g_labs = [
    r'$98^{\rm th}\,$-$\,100^{\rm th}$',
    r'$49^{\rm th}\,$-$\,51^{\rm st}$',
    r'$0^{\rm th}\,$-$\,2^{\rm nd}$',
]

for i_ax, i_z in enumerate([0,1,2]):
    ax = axs_f[i_ax]
    
    ra, dec = s_zs[i_z]['ra', 'dec']
    x, y = ra, np.sin(dec * deg2rad)
    
    lab = g_labs[i_ax]
    if i_ax == 0:
        lab = r'$z_{\rm f}\,{\rm percentiles}=$' + lab
    ax.scatter(x, y, s=30, c=cs[i_ax], edgecolors='none', marker='*', alpha=1, 
               label=lab)
    
    ra, dec, m = s_h['ra', 'dec', 'm_mean200']
    x, y = ra, np.sin(dec * deg2rad)
    s = (m)**(1./3.) * 1.5
    ax.scatter(x, y, s=s, c='grey', edgecolors='none', zorder=-10, alpha=.8)
    
axs.label(r'\rm RA', r'\sin({\rm Dec})').label_outer()\
    .lim([101., 269], [-0.15, 1.1]).leg(numpoints=1, markerscale=0, labelcolor='linecolor')

In [ ]:
wp_kw = {'n_z_match': 5000, 'n_repeat': 5, 'pimax': 10.}

In [ ]:
u_m = 1.0e10 / rc_samp.sim_info.cosmology.hubble
m_h_edges = 10.0**np.array([11.0, 12.0, 13.0, 16.0]) / u_m
wp_h2 = obspc.wps_of('s_halo', 'm_mean200', ps=None, qs=m_h_edges, in_simframe=True,
    **wp_kw)

In [ ]:
wp_kw = {'n_z_match': 5000, 'n_repeat': 3, 'pimax': 40.}

In [ ]:
wp_tot = obspc.wp_of('s_dwarf', **wp_kw)

In [ ]:
wp_z = obspc.wps_of('s_dwarf', 'z_half', **wp_kw)

In [ ]:
wp_spin = obspc.wps_of('s_dwarf', 'spin', **wp_kw)

In [ ]:
u_m = 1.0e10 / rc_samp.sim_info.cosmology.hubble
m_h_edges = 10.0**np.array([11.0, 12.0, 13.0, 16.0]) / u_m
wp_h = obspc.wps_of('s_halo', 'm_mean200', ps=None, qs=m_h_edges,
    **wp_kw)

In [ ]:
data = DataDict({
    'dwarfs': {
        'z_f': wp_z, 
        'spin': wp_spin, 
        'tot': wp_tot,
    },
    'halos': wp_h,
})

In [ ]:
'''
with open(p_out, 'w') as f:
    json.dump(data, f, default=json_default, indent=2)
'''

In [ ]:
p_out = ProjPaths.sim_dir_of(sim_info) / 'wp-elucid-large-simframe.json'
with open(p_out) as f:
    data = json.load(f)
    data = parse_json_input(data)

In [ ]:
wp_out = data

cs_g = ['b', 'g', cs_set1[4], 'r', 'purple'][::-1]
g_labs = [
    r'$0^{\rm th}\,$-$\,2^{\rm nd}$',
    r'$2^{\rm nd}\,$-$\,33^{\rm rd}$',
    r'$33^{\rm rd}\,$-$\,67^{\rm th}$',
    r'$67^{\rm th}\,$-$\,98^{\rm th}$',
    r'$98^{\rm th}\,$-$\,100^{\rm th}$',
]
h_labs = [
    r'$11.0 \leqslant M_{\rm h} < 12.0$',
    r'$12.0 \leqslant M_{\rm h} < 13.0$',
    r'$13.0 \leqslant M_{\rm h}$',
]
lws_h = np.linspace(1., 3.5, 4)
key_labs = r'z_{\rm f}', r'\lambda'

fig, axs = plot.subplots((2,2), share=(True, 'row'),
    space=0.055, subsize=(5., 3.75), 
    margin=[0.02, 0.02, 0.08, 0.08], layout='none',
    ratios=(None, [2, 1.]))
axs_f = axs.flat

leg_kw = dict(title_fontsize=13., labelspacing=.2, numpoints=1, 
              fontsize=13, labelcolor='linecolor')
for i_res, key_prop in enumerate(['z_f', 'spin']):

    axs_c = axs[:,i_res]
    ax, ax_r = axs_c

    wp_ref = wp_out['dwarfs/tot']
    wps_g = wp_out[f'dwarfs/{key_prop}']
    wps_h = wp_out['halos']
    #wps_h = wp_h2
    #wps_h = data_allvol['halos']

    x, y, y_e, yr, yr_e = wp_with_ratio(wp_ref, wp_ref,)
    axs_c.c('k')
    ax.errorbar(x, y, yerr=y_e, zorder=20, lw=2, label=r'$\rm Total$')
    ax_r.errorbar([1.0e-10, 1.0e10], [1., 1.], yerr=[0., 0.],
                    zorder=20, lw=2.)

    for i, i_sub in enumerate(range(len(wps_g))):
        axs_c.c(cs_g[i])
        x, y, y_e, yr, yr_e = wp_with_ratio(wps_g[str(i_sub)], wp_ref,)
        
        ax.errorbar(x, y, yerr=y_e, zorder=10-i_sub, lw=4, label=g_labs[i_sub])
        ax_r.errorbar(x, yr, yerr=yr_e, zorder=10-i_sub)
        
    for i, i_sub in enumerate(range(len(wps_h))):
        axs_c.c('grey')
        
        x, y, y_e, yr, yr_e = wp_with_ratio(wps_h[str(i_sub)], wp_ref,)
        
        ax.errorbar(x, y, yerr=y_e, lw=lws_h[i])
        ax_r.errorbar(x, yr, yerr=yr_e, label=h_labs[i], lw=lws_h[i])

    ax.leg(loc='ll', 
             title=r'${\rm Dwarfs},\,%s\ {\rm percentiles}$'%(key_labs[i_res]), 
             **leg_kw, ncol=2)    

axs[0].scale('log', 'log').lim([2.00e-2, 15], [0.1, 2.0e3])
axs[1].lim(y=[-0.15, 4.75])

axs[0,0].label(y=r'w_{\rm p}(r_{\rm p})\,[h^{-1}{\rm Mpc}]')
axs[1,0].label(y=r'\rm Ratio')\
    .leg(loc=(.5, 2.6), title=r'${\rm Halos},\,M_{\rm h} / {\rm M}_\odot$', **leg_kw)
axs[1].label(r'r_{\rm p}\,[h^{-1}{\rm Mpc}]')
axs.label_outer()

# ProjPaths.save_fig('fig_2pccf_dmo.pdf')

In [ ]:
p_wc = ProjPaths.obs_dir / 'weight_curve/z004-005'
rc_samp04 = ReconSample(sim_info, p_data, p_wc, 
    m_h_range_in_sol=(10**10.5, 10**11.))

In [ ]:
p_wc = ProjPaths.obs_dir / 'weight_curve/z005-006'
rc_samp05 = ReconSample(sim_info, p_data, p_wc, 
    m_h_range_in_sol=(10**10.5, 10**11.))

In [ ]:
obspc04 = ObsProjCross(rc_samp04)

In [ ]:
obspc05 = ObsProjCross(rc_samp05)

In [ ]:
wp_tot05 = obspc05.wp_of('s_dwarf', **wp_kw)

In [ ]:
wp_tot04 = obspc04.wp_of('s_dwarf', **wp_kw)

In [ ]:
data = DataDict({
    'dwarfs': {
        'z': wp_z, 'spin': wp_spin, 'tot': wp_tot,
    },
    'halos': wp_h,
})

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'wp-elucid.json'
with open(path, 'w') as f:
    json.dump(data, f, default=json_default, indent=2)

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'wp-elucid.json'
with open(path) as f:
    data = json.load(f)
    data = parse_json_input(data)

In [ ]:
cs_g = ['b', 'g', 'orange', 'r', 'purple'][::-1]
g_labs = [
    r'$0^{\rm th}\,$-$\,2^{\rm nd}$',
    r'$2^{\rm nd}\,$-$\,33^{\rm rd}$',
    r'$33^{\rm rd}\,$-$\,67^{\rm th}$',
    r'$67^{\rm th}\,$-$\,98^{\rm th}$',
    r'$98^{\rm th}\,$-$\,100^{\rm th}$',
]
h_labs = [
    r'$11.5 \leqslant M_{\rm h} < 12.0$',
    r'$12.0 \leqslant M_{\rm h} < 12.5$',
    r'$12.5 \leqslant M_{\rm h} < 13.0$',
    r'$13.0 \leqslant M_{\rm h}$',
]
lws_h = np.linspace(1., 3.5, 4)

In [ ]:
fig, axs = plot.subplots((2,2), share=(True, 'row'),
    space=0.03, subsize=(5., 3.75), 
    margin=[0.02, 0.02, 0.08, 0.08], layout='none',
    ratios=(None, [2, 1.]))
axs_f = axs.flat

ax = axs_f[0]
x1, y1, y1_e = wp_tot04['lg_rs_c', 'wp', 'wp_sd']
ax.errorbar(x, y1, yerr=y1_e, zorder=10-i_sub, lw=10, c='k')

ax = axs_f[0]
x1, y1, y1_e = wp_tot05['lg_rs_c', 'wp', 'wp_sd']
ax.errorbar(x, y1, yerr=y1_e, zorder=10-i_sub, lw=10, c='r')

for i_res, key_prop in enumerate(['z', 'spin']):

    axs_c = axs[:,i_res]
    ax, ax_r = axs_c

    xis_g = data['dwarfs'][key_prop]
    xi_ref = data['dwarfs/tot']

    x0, y0, y0_e = xi_ref['lg_rs_c', 'wp', 'wp_sd']
    x = 10.0**x0
    axs_c.c('k')
    ax.errorbar(x, y0, yerr=y0_e, zorder=20, lw=2)
    ax_r.errorbar(x, np.ones_like(x), yerr=np.zeros_like(x), 
                    zorder=20, lw=2., label=r'$\rm Total$')

    for i, i_sub in enumerate(range(len(xis_g))):
        axs_c.c(cs_g[i])
        x1, y1, y1_e = xis_g[str(i_sub)]['lg_rs_c', 'wp', 'wp_sd']
        
        y_r = Num.safe_div(y1, y0)
        y_r_e = Num.safe_div(y1_e, y0)
        
        ax.errorbar(x, y1, yerr=y1_e, zorder=10-i_sub, lw=4)
        ax_r.errorbar(x, y_r, yerr=y_r_e, zorder=10-i_sub, label=g_labs[i_sub])
        
    xis_h = data['halos']
    for i, i_sub in enumerate([0,1,2,3]):
        axs_c.c('grey')
        
        x1, y1, y1_e = xis_h[str(i_sub)]['lg_rs_c', 'wp', 'wp_sd']
        
        y_r = Num.safe_div(y1, y0)
        y_r_e = Num.safe_div(y1_e, y0)
        
        ax.errorbar(x, y1, yerr=y1_e, lw=lws_h[i], label=h_labs[i])
        ax_r.errorbar(x, y_r, yerr=y_r_e, lw=lws_h[i])


axs[0].scale('log', 'log').lim([1.00e-2, 18], [0.2, 2.0e3])
axs[1].lim(y=[0.2, 4.75])

leg_kw = dict(title_fontsize=13., labelspacing=.2, numpoints=1, fontsize=13)
axs[0,0].label(y=r'w_{\rm p}(r_{\rm p})\,[h^{-1}{\rm Mpc}]')\
    .leg(loc='ur', title=r'${\rm Halos},\,M_{\rm h} / {\rm M}_\odot$', **leg_kw)
axs[1,0].leg(loc=(0.03, 1.1), title=r'${\rm Dwarfs},\,z_{\rm f}\ {\rm percentiles}$',**leg_kw, ncol=2)\
    .label(y=r'\rm Ratio')
axs[1,1].leg(loc=(0.03, 1.1), title=r'${\rm Dwarfs},\ \lambda\ {\rm percentiles}$',**leg_kw, ncol=2)
axs[1].label(r'r_{\rm p}\,[h^{-1}{\rm Mpc}]')
axs.label_outer()

ax = axs_f[1]
ax.text(r'\rm ELUCID', (.94, .94), ha='right')

# ProjPaths.save_fig('fig_2pccf_dmo.pdf')
# ProjPaths.save_fig('fig_2pccf_dmo_with_ej.pdf')
#ProjPaths.save_fig('fig_2pccf_dmo_rho_0.pdf')

In [ ]:
fig, axs = plot.subplots((2,2), share=(True, 'row'),
    space=0.03, subsize=(5., 3.75), 
    margin=[0.02, 0.02, 0.08, 0.08], layout='none',
    ratios=(None, [2, 1.]))
axs_f = axs.flat

ax = axs_f[0]
x1, y1, y1_e = wp_tot04['lg_rs_c', 'wp', 'wp_sd']
ax.errorbar(x, y1, yerr=y1_e, zorder=10-i_sub, lw=10, c='k')

ax = axs_f[0]
x1, y1, y1_e = wp_tot05['lg_rs_c', 'wp', 'wp_sd']
ax.errorbar(x, y1, yerr=y1_e, zorder=10-i_sub, lw=10, c='r')

for i_res, key_prop in enumerate(['z', 'spin']):

    axs_c = axs[:,i_res]
    ax, ax_r = axs_c

    xis_g = data['dwarfs'][key_prop]
    xi_ref = data['dwarfs/tot']

    x0, y0, y0_e = xi_ref['lg_rs_c', 'wp', 'wp_sd']
    x = 10.0**x0
    axs_c.c('k')
    ax.errorbar(x, y0, yerr=y0_e, zorder=20, lw=2)
    ax_r.errorbar(x, np.ones_like(x), yerr=np.zeros_like(x), 
                    zorder=20, lw=2., label=r'$\rm Total$')

    for i, i_sub in enumerate(range(len(xis_g))):
        axs_c.c(cs_g[i])
        x1, y1, y1_e = xis_g[str(i_sub)]['lg_rs_c', 'wp', 'wp_sd']
        
        y_r = Num.safe_div(y1, y0)
        y_r_e = Num.safe_div(y1_e, y0)
        
        ax.errorbar(x, y1, yerr=y1_e, zorder=10-i_sub, lw=4)
        ax_r.errorbar(x, y_r, yerr=y_r_e, zorder=10-i_sub, label=g_labs[i_sub])
        
    xis_h = data['halos']
    for i, i_sub in enumerate([0,1,2,3]):
        axs_c.c('grey')
        
        x1, y1, y1_e = xis_h[str(i_sub)]['lg_rs_c', 'wp', 'wp_sd']
        
        y_r = Num.safe_div(y1, y0)
        y_r_e = Num.safe_div(y1_e, y0)
        
        ax.errorbar(x, y1, yerr=y1_e, lw=lws_h[i], label=h_labs[i])
        ax_r.errorbar(x, y_r, yerr=y_r_e, lw=lws_h[i])


axs[0].scale('log', 'log').lim([1.00e-2, 18], [0.2, 2.0e3])
axs[1].lim(y=[0.2, 4.75])

leg_kw = dict(title_fontsize=13., labelspacing=.2, numpoints=1, fontsize=13)
axs[0,0].label(y=r'w_{\rm p}(r_{\rm p})\,[h^{-1}{\rm Mpc}]')\
    .leg(loc='ur', title=r'${\rm Halos},\,M_{\rm h} / {\rm M}_\odot$', **leg_kw)
axs[1,0].leg(loc=(0.03, 1.1), title=r'${\rm Dwarfs},\,z_{\rm f}\ {\rm percentiles}$',**leg_kw, ncol=2)\
    .label(y=r'\rm Ratio')
axs[1,1].leg(loc=(0.03, 1.1), title=r'${\rm Dwarfs},\ \lambda\ {\rm percentiles}$',**leg_kw, ncol=2)
axs[1].label(r'r_{\rm p}\,[h^{-1}{\rm Mpc}]')
axs.label_outer()

ax = axs_f[1]
ax.text(r'\rm ELUCID', (.94, .94), ha='right')

# ProjPaths.save_fig('fig_2pccf_dmo.pdf')
# ProjPaths.save_fig('fig_2pccf_dmo_with_ej.pdf')
#ProjPaths.save_fig('fig_2pccf_dmo_rho_0.pdf')

### Weight curve

In [ ]:
p_wc = ProjPaths.obs_dir / 'weight_curve//diffuse_dwarf_z_dis'

z = np.linspace(0., .12, 128)
f = rc_samp.f_at_z(z)
f = f / f.sum()

fig, ax = plot.subplots(1, figsize=4.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

ax.plot(z, f)

s = rc_samp.ramdom_subs_dwarf()
w = rc_samp.find_weight(s['z'])

h, e = np.histogram(s['z'], range=(0., .12), bins=128, weights=w)
x = .5 * (e[:-1] + e[1:])
ax.plot(x, h, c='r')

### Tidal Field

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'fields' / 'tidal.s99.sm1.hdf5'
h5.File.ls_from(path)

In [ ]:
# generate tidal field
path = ProjPaths.sim_dir_of(sim_info) / 'density_field.99.hdf5'
with h5.File(path) as f:
    rho_x = cubic_box.DensityField.load(f)

tf = cubic_box.TidalField(n_workers=8, r_sm=1.0)
res = tf.run(rho_x)

path = ProjPaths.sim_dir_of(sim_info) / 'tidal_field.99.sm1.hdf5'
with h5.File(path, 'w') as f:
    res.dump(f)

In [ ]:
# generate tidal field
path = ProjPaths.sim_dir_of(sim_info) / 'density_field.99.hdf5'
with h5.File(path) as f:
    rho_x = cubic_box.DensityField.load(f)

tf = cubic_box.TidalField(n_workers=8, r_sm=2.0)
res = tf.run(rho_x)

path = ProjPaths.sim_dir_of(sim_info) / 'tidal_field.99.sm2.hdf5'
with h5.File(path, 'w') as f:
    res.dump(f)

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'tidal_field.99.sm2.hdf5'
delta_sm_k = h5.File.load_from(path, 'delta_sm_k')
fft = cubic_box.fft.NdRealFFT(n_workers=8, norm='ortho')
delta_sm_x = fft.backward(delta_sm_k)
h5.File.dump_to(path, delta_sm_x, key='delta_sm_x')

In [ ]:
fig, axs = plot.subplots((3,2), share=True, space=0, 
    subsize=(6.25, 6.25), margin=[.03, .01, .04, .06], layout='none')
axs_f = axs.flat

l_grid = clsf.mesh.l_grid
l_box = sim_info.box_size
    
z0s = [20., 100., 180.]
cs = 'b', 'r'
labs = r'$z_{\rm h,1/2} > 2.75$', r'$ 0.9 \leqslant z_{\rm h,1/2} < 1$'
for i_z, z0 in enumerate(z0s):
    z1 = z0 + 10.0
    dz = 0.    
    zi0, zi1 = int((z0-dz) / l_grid), int((z1+dz) / l_grid)
    im = clsf.is_filament[:, :, zi0:zi1].sum(-1) >= 1

    for i, s_g in enumerate([1,2]):
        ax = axs[i_z, i]
        ax._raw.imshow(im.T, origin='lower', 
                    extent=[0., l_box, 0., l_box], 
                    cmap='Greys', vmin=0., vmax=2., aspect='auto')
        
        if i_z == 0:
            ax._raw.set_title(labs[i], color=cs[i])        

axs.lim([0., l_box], [0., l_box])
axs.label(x=r'x\,[h^{-1}{\rm Mpc}]', y=r'y\,[h^{-1}{\rm Mpc}]').label_outer();

# ProjPaths.save_fig('tng_dark_300_map_with_cosmic_web.pdf')

### Observation samples

In [ ]:
obs_diffuse = obs_catalogs.diffuse_dwarfs

In [ ]:
obs_diffuse.data.keys()

In [ ]:
np.zeros((4,4,4,3))[[1,2],[2,1],[2,1]]

In [ ]:
obs_diffuse = obs_catalogs.diffuse_dwarfs
obs_compact = obs_catalogs.compact_dwarfs

frame = SimFrameSdssL500(sim_info)
mask = ReconstAreaMaskSdssL500(sim_info)

path = ProjPaths.sim_dir_of(sim_info) / 'tidal_field.99.sm1.hdf5'
with h5.File(path) as f:
    clsf = cubic_box.TidalClassifier.new_from_file(f, lam_th=0.2)
    
path = ProjPaths.sim_dir_of(sim_info) / 'tidal_field.99.sm2.hdf5'
with h5.File(path) as f:
    f_interp = cubic_box.cosmic_web.FieldInterpolator.new_density_field_from_file(
        f, key='delta_x')
f_interp

In [ ]:
fig, ax = plot.subplots(1, figsize=4.5, extent=[1., 1., 0., 0.], layout='none')

kw = dict(bins=30, density=True, range=[0.0, 0.08])

ra, dec, z = obs_diffuse.data['ra', 'dec', 'redshift']
ax.hist(z, **kw)

ra, dec, z = obs_compact.data['ra', 'dec', 'redshift']
ax.hist(z, **kw)

## Environment of UDG Candidates

In [ ]:
sim_info = sims.predefined['elucid_ext_v2']
p_data = ProjPaths.sim_dir_of(sim_info) / 'z0_all.hdf5'

In [ ]:
man_kw = {'add_rsd': False, 'dmo': True, 
          #'m_h_range_in_sol': (10**10.79, 10**10.81)
          'm_h_range_in_sol': (10**10.5, 10**11)
}
sman = SubSampleManager(sim_info, p_data, **man_kw)

In [ ]:
s_dwarf = sman.s_dwarf
s_dwarf = s_dwarf.subset_by_value('z', lo=0.02, hi=0.03).subset_by_value('is_recon_large', eq=True)
s_dwarf

In [ ]:
tf = obs_catalogs.tidal_fields.resim_z0_sm1
rng = Rng(0)

### $z_{\rm f}$ vs web type

In [ ]:
pz_ranges = [0.0, 0.02], [0.49, 0.51], [0.98, 1.00]
pfs = {}
for i_pz, pz_range in enumerate(pz_ranges):
    s = s_dwarf.subset_by_percentile('z_half', *pz_range)
    env = GalEnvSimed(s, tf, rng=rng)
    pf = env.profiles(n_bootstrap=5)
    pfs[str(i_pz)] = pf

In [ ]:
fig, ax = plot.subplots(1, figsize=5.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

cs = 'r', 'g', 'b'
lab_subs = r'$\rm 0\%$-$2\%$', r'$\rm 49\%$-$51\%$', r'$\rm 98\%$-$100\%$' 

lws = 1, 1, 3, 3
lss = (0,(1,2)), (0,(3,1)), (0, (2,1)), '-'

for i_samp in 0, 1, 2:
    pf = pfs[str(i_samp)]
    ax.c(cs[i_samp])
    for i_t in range(4):
        x = pf['lr']
        (y, e) = pf['f_webs'][i_t]
        lab = None 
        if i_z == 0 and i_t == 3:
            lab = lab_subs[i_samp]
        ax.errorbar(x, y, yerr=e, lw=lws[i_t], ls=lss[i_t], 
                    capsize=1.5, elinewidth=1.5, marker='s', ms=6., label=lab)

ax.lim(y=[-0.05, 0.85])
ax.leg(
    numpoints=1, labelspacing=0, loc='ur',
    labelcolor='linecolor', handlelength=0, title=r'$z_{\rm f}\ {\rm percents}$')
ax.label(r'\log\,r\ [h^{-1}{\rm Mpc}]', r'\rm Fraction').label_outer()

#ProjPaths.save_fig('fig_gal_env_web_frac_by-zf.pdf')

In [ ]:
def web_tp_profile(gal_env: GalEnvSimed):
    samp, tf, rng = gal_env.samp, gal_env.tf, gal_env.rng
    web_t = tf.wet_t_at(samp['x'])
    out = {}
    for i_t in 0, 1, 2, 3:
        sel = web_t == i_t
        if sel.sum() == 0:
            continue
        samp_t = samp.subset(sel)
        pf = GalEnvSimed(samp_t, tf, rng=rng).profiles(n_bootstrap=5)
        out[str(i_t)] = pf
    out['all'] = gal_env.profiles(n_bootstrap=5)
    return out

In [ ]:
pz_ranges = [0.0, 0.02], [0.49, 0.51], [0.98, 1.00]
pfs_t = {}
for i_pz, pz_range in enumerate(pz_ranges):
    s = s_dwarf.subset_by_percentile('z_half', *pz_range)
    env = GalEnvSimed(s, tf, rng=rng)
    pf = web_tp_profile(env)
    pfs_t[str(i_pz)] = pf

In [ ]:
fig, axs = plot.subplots((3,1), share=(True, False), space=0.03, 
    subsize=(5.5, 3.), margin=[0.02, 0.02, 0.06, 0.145], layout='none')
axs_f = axs.flat

lws = 1, 1, 3, 3
lss = (0,(1,2)), (0,(3,1)), (0, (2,1)), '-'
cs = 'r', 'g', 'b'
lab_subs = r'$\rm 0\%$-$2\%$', r'$\rm 49\%$-$51\%$', r'$\rm 98\%$-$100\%$' 

eb_kw = dict(lw=2.5, capsize=1.5, elinewidth=1.5, marker='s', ms=7.)
ef_kw = dict(lw=1.5, fill_between_kw={'lw': 0, 'alpha': .4})

for i_sub in range(3):
    axs.c(cs[i_sub])
    pf = pfs_t[f'{i_sub}']
    
    ax = axs[0]
    _pf = pf['all']
    x, (y, e) = _pf['lr'], _pf['den_prof']
    ax.errorfill(x, y, yerr=e, **ef_kw, label=lab_subs[i_sub])

    ax = axs_f[1]
    _pf = pf['2']
    x, (y, e) = _pf['lr'], _pf['den_prof']
    ax.errorfill(x, y, yerr=e, **ef_kw)
        
    ax = axs_f[2]
    _pf = pf['1']
    x, (y, e) = _pf['lr'], _pf['den_prof']
    ax.errorfill(x, y, yerr=e, **ef_kw)

txs = r'\rm In\ all\ web\ types', r'\rm In\ filament', r'\rm In\ sheet'
for i, tx in enumerate(txs):
    axs[i].text(tx, (.94, .9), ha='right')
        
axs.label(r'\log\,r\ [h^{-1}{\rm Mpc}]', r'\rho/\bar{\rho}')\
    .label_outer()

ax = axs_f[0]        
ax.lim([-0.05, 1.15],[0.85, 5.25]).leg(
    numpoints=1, labelspacing=0, loc=(.62, .52), 
    labelcolor='linecolor', handlelength=0, title=r'$z_{\rm f}\ {\rm percents}$')

ax = axs_f[1]
ax.lim(y=[0.95, 5.25])

ax = axs_f[2]
ax.lim(y=[0.55, 1.7])

#ProjPaths.save_fig('fig_gal_env_profiles-by-zf.pdf')

## Galaxy-Web Correlation

In [ ]:
info = sims.predefined['elucid']
sim_dir = ProjPaths.man_sim_dir(info)
g_rng = Rng(0)
frame = SimFrameSdssL500(info)

tf = obs_catalogs.tidal_fields.dom_z0_sm1
#tf = obs_catalogs.tidal_fields.resim_z0_sm1

In [ ]:
rc_samps = obs_catalogs.recons_samples

dwarf_samples: DataDict[str, obs_catalogs.ReconSample] = DataDict({
    'diffuse': DataDict({
        'all': rc_samps.diffuse_dwarfs,
    }),
    'compact': DataDict({
        'all': rc_samps.compact_dwarfs,
    }),
    'group': DataDict({
        'all': rc_samps.groups,
    })
})

for i_k, k in enumerate(('diffuse', 'compact', 'group')):
    s_list = dwarf_samples[k]
    s_all = s_list['all']
    
    z_cor = s_all.data['z_cor']
    if i_k != 2:
        m_s = 10.0 ** s_all.data['log_m_s'] * (info.cosmology.hubble**-2.)
        print(np.log10(m_s).min())
        sel = (m_s > 10**8.5)&(z_cor < 0.04)
        s_complete = s_all.subset_by(sel)
        s_list['complete'] = s_complete
    
    s_z23 = s_all.subset_by_val('z_cor', lo=0.02, hi=0.03)
    s_z34 = s_all.subset_by_val('z_cor', lo=0.03, hi=0.04)
    s_z45 = s_all.subset_by_val('z_cor', lo=0.04, hi=0.05)
    s_z56 = s_all.subset_by_val('z_cor', lo=0.05, hi=0.06)
    s_z2 = s_all.subset_by_val('z_cor', hi=0.02)
    sel = (z_cor <= 0.02) | (z_cor >= 0.03)
    s_zexcl23 = s_all.subset_by(sel)
    
    s_list |= {
        'z23': s_z23,
        'z34': s_z34,
        'z45': s_z45,
        'z56': s_z56,
        'z2': s_z2,
        'zexcl23': s_zexcl23,
    }

In [ ]:
dwarf_samples

In [ ]:
# old
s_dwarf
samps = rc_samps.diffuse_dwarfs, rc_samps.compact_dwarfs

samps_z23 = [s.subset_by_val('z_cor', 0.02, 0.03) for s in samps]
samps_z2 = [s.subset_by_val('z_cor', hi=0.02) for s in samps]

samp_grp = rc_samps.groups
samp_grp_z23 = samp_grp.subset_by_val('z_cor', 0.02, 0.03)

### Field images

In [ ]:
f = cubic_box.Field.new_by_data(tf.delta + 1., tf.mesh)
f_interp = cubic_box.cosmic_web.FieldInterpolator(f)
f_interp_ts = []
for i in range(4): 
    f = (tf.n_lams == i)
    f = cubic_box.Field.new_by_data(f.astype(float), tf.mesh)
    f = cubic_box.cosmic_web.FieldInterpolator(f)
    f_interp_ts.append(f)

In [ ]:
def img_get_group(samp: obs_catalogs.ReconSample):
    hubble = info.cosmology.hubble
    m_h_lb = 10**12.0 * hubble
    samp_grp_slice = samp.subset_by_val('m_group', lo=m_h_lb)
    
    ra, dec, m = samp_grp_slice.data['ra', 'dec', 'm_group']
    x, y = ra, np.sin(dec * np.pi / 180.)
    s = m**(1./3.)
    s = s / s.max() * 100
    return pd.DataFrame({'RA': x, 'sin_Dec': y, 'marker_size': s})

def img_get_dwarf(samp: obs_catalogs.ReconSample, n_out=None, rng=g_rng):
    ra, dec, x_sim_cor = samp.data['ra', 'dec', 'x_sim_cor']
    x, y = ra, np.sin(dec * np.pi / 180.)
    web_t = tf.wet_t_at(x_sim_cor)
    if n_out is not None:
        ids = rng.choice(len(x), n_out, replace=False)
        x, y = x[ids], y[ids]
        web_t = web_t[ids]
    return pd.DataFrame({'RA': x, 'sin_Dec': y, 'web_type': web_t})

def img_get_density(n_ra=64, n_dec=64, n_z=10, z_rg=[0.02, 0.03]):
    
    assert n_ra == n_dec
    
    ras1 = np.linspace(100., 275., n_ra)
    sin_decs1 = np.linspace(-0.15, 0.95, n_dec)
    decs1 = np.arcsin(sin_decs1) * 180.0 / np.pi 
    d_lo, d_hi = info.cosmology.distances.comoving_at(z_rg)
    ds1 = np.linspace(d_lo, d_hi, n_z)
    
    ras, decs, ds = np.meshgrid(ras1, decs1, ds1, indexing='ij')
    dims = ras.shape
    ras, decs, ds = ras.ravel(), decs.ravel(), ds.ravel()
    x_j2k = frame.pos_ra_dec_d_to_j2k(ras, decs, ds)
    x_sim = frame.pos_j2k_to_sim(x_j2k)
    
    rho = f_interp.value_at(x_sim).reshape(dims).mean(-1)
    web_ts = [
        f.value_at(x_sim).reshape(dims).mean(-1)
        for f in f_interp_ts
    ]
    sel = reconst_area_masks.large.is_in_reconst_area(x_sim).reshape(dims).sum(-1) > 1

    return {
        'grids': pd.DataFrame({
            'RA': ras1, 'sin_Dec': sin_decs1
        }),
        'fields': pd.DataFrame({
            'frac_filament': web_ts[2].ravel(),
            'reconstruction_mask': sel.ravel(),
        })
    }
    
    #return DataDict({
    #    'rho': rho, 'web_ts': web_ts, 'sel': sel, 
    #    'ra': ras1, 'dec': decs1, 'sin_dec': sin_decs1
    #})

In [ ]:
d_plts = {}
_rng = Rng(0)
key = 'z23'
z_rg = [0.02, 0.03]
#key = 'z34'
#z_rg = [0.03, 0.04]
#key = 'z2'
#z_rg = [0.01, 0.02]
#key = 'z56'
#z_rg = [0.05, 0.06]

d_plts['group'] = img_get_group(dwarf_samples[f'group/{key}'])
d_plts['diffuse'] = img_get_dwarf(dwarf_samples[f'diffuse/{key}'], rng=_rng)
d_plts['compact'] = img_get_dwarf(dwarf_samples[f'compact/{key}'], n_out=len(d_plts['diffuse']['RA']), rng=_rng)
#d_plts |= {
#    'diffuse': img_get_dwarf(samps_z23[0], rng=_rng),
#    'compact': img_get_dwarf(samps_z23[1], n_out=samps_z23[0].n_objs, rng=_rng),
#}
#
d_plts |= img_get_density(n_ra=256, n_dec=256, n_z=20, z_rg=z_rg)

In [ ]:
z_wgt = utils.weights.ZWeighting(dwarf_samples['diffuse/all'].data['z_cor'], bins=24)

rng = Rng(0)
keys = 'all', 'z23', 'complete'
wgts = z_wgt, None, None

gal_envs = {}
for k, wgt in zip(keys, wgts):
    for k2 in 'diffuse', 'compact':
        gal_envs[f'{k2}.{k}'] = GalEnv(dwarf_samples[k2][k], tf, zwgt=wgt, 
                                       rng=rng, lr_range=[0.1, 1.2], n_bins=8)
        
def find_corr(ge: GalEnv, mass_weighted=True, n = 100):
    outs = ge.corrs(mass_weighted=mass_weighted)
    out = [asdict(ge.bootstrap(out, n)) for out in outs['web_typed']]
    return DataDict({
        'lrs': ge.lrs,
        'web_typed': out
    })

corrs_mwgt = DataDict({
    k: find_corr(ge, mass_weighted=True) for k, ge in gal_envs.items()
})
corrs_nowgt = DataDict({
    k: find_corr(ge) for k, ge in gal_envs.items()
})

d_corrs = DataDict({
    'nowgt': corrs_nowgt,
    'mwgt': corrs_mwgt
})
Json.dump_file(d_corrs, sim_dir.base_dir/'wp/gal_web_cross.json')

In [ ]:
d_corrs = Json.load_file(sim_dir.base_dir/'wp/gal_web_cross.json')

corrs = d_corrs['mwgt']
z_key = 'all'

web_types = ['void', 'sheet', 'filament', 'knot']
d_plts['2pccfs'] = {}
for i_t, web_type in enumerate(web_types):
    for i_sub, sub_key in enumerate(['diffuse', 'compact']):
        corr = corrs[f'{sub_key}.{z_key}']
        y, x = corr['web_typed'][i_t], corr['lrs']
        y, (yl, yh) = np.array(y['median']), np.array(y['sigma_1'])
        el, eh = y-yl, yh-y
        d_plts['2pccfs'][f'{sub_key}_and_{web_type}'] = pd.DataFrame({
            'log_r': x, 'xi': y, 'err_lo': el, 'err_up': eh
        })

In [ ]:
Excel.dump_file(ProjPaths.proj_dir/'data_for_publication/Fig2.xlsx', d_plts)

In [ ]:
fig, axs = plot.subplots((4,2), share=True, space=0, subsize=(5.5, 4.), 
                         margin=[0.1, 0.02, 0.1, 0.1], layout='none')
axs_f = axs.flat
vmaxs = [1., 1., 1., .5]
lab_ts = 'Void', 'Sheet', 'Filament', 'Knot'
for i_t in range(4):
    rho, sel, ra, dec = d_plt['density']['web_ts', 'sel', 'ra', 'sin_dec']
    rho = rho[i_t]
    rho = np.ma.array(rho, mask=~sel)
    for i_c in range(2):
        ax = axs[i_t, i_c]
        ax._raw.contourf(ra, dec, rho.T, cmap='Greys', vmin=-.1, 
                         vmax=vmaxs[i_t], alpha=.9, rasterized=True)
    axs[i_t, 1].text(lab_ts[i_t], (.92, .92), ha='right')
        
cs_samp = 'b', 'r'
labs = r'$\rm Diffuse\ dwarf$', r'$\rm Compact\ dwarf$'
for i_r in range(4):
    for i_samp, samp in enumerate(['diffuse', 'compact']):
        ax = axs[i_r, i_samp]
        x, y = d_plt[samp]['x', 'y']
        lab = labs[i_samp] if i_r == 0 else None
        ax.fmt_marker(c=cs_samp[i_samp], ea=1, fa=0.2, elw=1.5)\
            .scatter(x, y, s=75, marker='*', label=lab, zorder=100)
        
axs.lim([105, 270], [-0.15, 1.1]).label(r'\rm RA', r'\sin(\rm Dec)').label_outer()
kw=dict(scatterpoints=1, ncol=3, handletextpad=0, loc='ul', handlelength=1, handleheight=1)
axs_f[:2].leg(**kw, labelcolor='mec')
axs_f[2].leg(**kw, markerscale=3)

#ProjPaths.save_fig('fig_spatial_distribution.pdf')

### Typed density profiles around dwarfs

In [ ]:
z_zwgts = {
    'all': utils.weights.ZWeighting(dwarf_samples['diffuse/all'].data['z_cor'], bins=24), #samps[0].data['z_cor'], bins=24
    'zexcl23': utils.weights.ZWeighting(dwarf_samples['diffuse/zexcl23'].data['z_cor'], bins=24)
}

def web_tp_profile(gal_env: GalEnv):
    samp, tf, zwgt, rng = gal_env.samp, gal_env.tf, gal_env.zwgt, gal_env.rng
    web_t = tf.wet_t_at(samp.data['x_sim_cor'])
    out = {}
    for i_t in 0, 1, 2, 3:
        sel = web_t == i_t
        samp_t = samp.subset_by(sel)
        pf = GalEnv(samp_t, tf, zwgt=zwgt, rng=rng).profiles()
        out[str(i_t)] = pf
    out['all'] = gal_env.profiles()
    return out

In [ ]:
pfs = {
    '0': web_tp_profile(samps_z23[0]),
    '1': web_tp_profile(samps_z23[1]),
    '0-w': web_tp_profile(samps[0], zwgt=zwgt),
    '1-w': web_tp_profile(samps[1], zwgt=zwgt),
}

In [ ]:
fig, axs = plot.subplots((2,2), share=False, space=0.13, 
                         subsize=(4.5, 4.5), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

subs = '0', '1'
lab_subs = r'$Diffuse$', r'$Compact$'
lab_ws = r'$\,\rm (z$-$\rm weighted)$', r'$\,(0.02 < z < 0.03)$'
cs = 'b', 'r'

lab_ts = 'Void', 'Sheet', 'Filament', 'Knot'
for i_t in range(4):
    ax = axs_f[i_t]    
    ax.text(lab_ts[i_t], (.94, .92), ha='right')
    for i_sub, sub in enumerate(subs):
        ax.c(cs[i_sub])
        
        pf = pfs[sub+'-w'][str(i_t)]
        x, (y, e) = pf['lr', 'den_prof']
        ax.errorbar(x, y, yerr=e, lw=2, label=lab_subs[i_sub] + lab_ws[0])
        
        pf = pfs[sub][str(i_t)]
        x, (y, e) = pf['lr', 'den_prof']
        ax.fill_between(x, y-e, y+e, ec=cs[i_sub], 
                        lw=0, label=lab_subs[i_sub] + lab_ws[1])
        #ax.plot(x, y, lw=2)
        
#axs.lim(y=[1.,10.1])
kw = dict(numpoints=1, fontsize=13, title_fontsize=12, 
          labelspacing=.2, labelcolor='mec', loc='ul')
axs_f[0].leg(**kw, title=r'$Sample$')
axs[1].label(r'\log\,r\,[h^{-1}{\rm Mpc}]')
axs[:, 0].label(y=r'\rho/\bar{\rho}')

### Web fractions around dwarfs

In [ ]:
rng = Rng(0)
gal_envs = [GalEnv(samp, tf, zwgt=zwgt, rng=rng) for samp in samps] \
    + [GalEnv(samp, tf, zwgt=None, rng=rng) for samp in samps_z23]
pfs = [ge.profiles() for ge in gal_envs]
pfs = {
    '0': pfs[0], '1': pfs[1],
    '0-z23': pfs[2], '1-z23': pfs[3],
}

In [ ]:
fig, axs = plot.subplots((1,2), share=True, space=0, 
    subsize=(4.5, 4.5), margin=[0.02, 0.02, 0.12, 0.08], layout='none')
axs_f = axs.flat

lab_samps = r'\rm All\ redshifts\, (z$-$\rm weighted)', r'0.02 \leqslant z < 0.03'

cs = 'b', 'r'
lab_subs = r'$\rm Diffuse\ dwarfs$', r'$\rm Compact\ dwarf$'

lws = 1, 1, 3, 3
lss = (0,(1,2)), (0,(3,1)), (0, (2,1)), '-'

for i_z, z_key in enumerate(['', '-z23']):
    ax = axs_f[i_z]
    ax.text(lab_samps[i_z], (.05, .92))
    for i_samp in 0, 1:
        pf = pfs[f'{i_samp}{z_key}']
        ax.c(cs[i_samp])
        for i_t in range(4):
            x = pf['lr']
            (y, e) = pf['f_webs'][i_t]
            lab = None 
            if i_z == 0 and i_t == 3:
                lab = lab_subs[i_samp]
            ax.errorbar(x, y, yerr=e, lw=lws[i_t], ls=lss[i_t], 
                        capsize=1.5, elinewidth=1.5, marker='s', ms=6., label=lab)

axs.lim(y=[-0.05, 0.85])
axs[0].leg(loc=(0.05, 0.77),
    numpoints=1, fontsize=13, labelspacing=0, 
    labelcolor='linecolor', handlelength=0)
axs.label(r'\log\,r\ [h^{-1}{\rm Mpc}]', r'\rm Fraction').label_outer()

ProjPaths.save_fig('fig_gal_env_web_frac.pdf')

### z-weight

In [ ]:
zwgt = utils.weights.ZWeighting(samps[0].data['z_cor'], bins=24)

fig, ax = plot.subplots(1, figsize=4.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

cs = 'b', 'r'
for i, samp in enumerate(samps):
    z = samp.data['z_cor']
    ax.c(cs[i]).hist(z, bins=12, range=[0., 0.06], density=True, fc='none', lw=3)
    w = zwgt.weights_of(z)
    ax.hist(z, bins=12, range=[0., 0.06], weights=w, density=True, lw=0)

### Minimal distances

In [ ]:
gwcs = [utils.gal_web_cor.GalWebCor(s, td_s2, mask=rc_mask, 
    zweighting=zweighting) for s in df_samps]

In [ ]:
o_ds = {}
for n_lam in range(4):
    for i_samp, gwc in enumerate(gwcs):
        o_ds[n_lam, i_samp] = gwc.d_to_webt(n_lam)

In [ ]:
fig, axs = plot.subplots((2,2), share=True, space=0.03, subsize=(5., 5.), 
                         margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

lab_ts = 'WebType=Void', 'Sheet', 'Filament', 'Knot'

cs = 'b', 'r'
labs = r'$\rm Diffuse$', r'$\rm Compact$'

kw = dict(range=(-1.2, 1.2), bins=25, fc='none', density=True)
for i_lam in range(4):
    ax = axs_f[i_lam]
    ax.text(lab_ts[i_lam], (.03, .9), fontsize=13)
    for i_samp in range(2):
        ax.c(cs[i_samp])
        d, w = o_ds[i_lam, i_samp]['d', 'w']
        ax.hist(np.log10(d), **kw, label=labs[i_samp])
        if i_samp == 1:
            ax.hist(np.log10(d), weights=w, **kw, ls=(0,(2,2)), 
                    label=r'$\rm Compact\, (z$-$\rm weighted)$')
            
axs_f[0].leg(title=r'$\rm Sample$', fontsize=12)
axs.label(r'd_{\rm min}(Galaxy, WebType)', r'\rm PDF').label_outer()

### Fraction of web types

In [ ]:
lam_th = np.linspace(-1, 1., 128)
frac_webt = {}
for i_td, _td in enumerate([td_resim, td_s1, td_s2, td_s4]):
    for i_samp, samp in enumerate(df_samps):
        gwc = utils.gal_web_cor.GalWebCor(samp, _td
                                          #, zweighting=zweighting
                                          )
        frac_webt[i_td, i_samp] = gwc.frac_webt(lam_th)

In [ ]:
fig, axs = plot.subplots((2,2), share=True, space=0.1, subsize=(5., 5.), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

lab_tds = r'ReSimulated\,(R_{\rm sm}=2)', r'Domain\,(R_{\rm sm}=1)', r'Domain\,(R_{\rm sm}=2)', r'Domain\,(R_{\rm sm}=4)'

lss = '-', (0,(2,2))
lab_ss = 'Diffuse', 'Compact'

lws = 4,3,2,1
cs = 'b', 'g', 'orange', 'r' 
lab_ts = 'Void', 'Sheet', 'Filament', 'Knot'
for i_td in range(4):
    ax = axs_f[i_td]
    ax.text(lab_tds[i_td], (.03, .9), fontsize=13)
    for i_samp in range(2):
        ax.fmt_line(ls=lss[i_samp])    
        for i_wt, f in enumerate(frac_webt[i_td, i_samp]):
            ax.fmt_line(lw=lws[i_wt], c=cs[i_wt])
            lab = None
            if i_samp == 0 and i_td == 0:
                lab = lab_ts[i_wt]
            if i_wt == 0 and i_td == 1:
                lab = lab_ss[i_samp] 
            ax.plot(lam_th, f, label=lab)
            
axs.label(r'\lambda_{\rm th}', r'N_{WebType}/N_{\rm Total}')\
    .lim([-1., 1.], [0., 1.]).label_outer()
axs_f[0].leg(title=r'WebType')
axs_f[1].leg(title=r'Sample')

### Field G-Index

Looks like a bad definition

In [ ]:
lam_off = 0.
norm = True
G_f1, G_f2 = \
    gwc_diffuse.G_s(lam_off, norm), gwc_compact.G_s(lam_off, norm)

In [ ]:
fig, ax = plot.subplots(1, figsize=4.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

ax.c('b').hist(G_f1, density=True, bins=20, range=[0., 1.])
ax.c('r').hist(G_f2, density=True, bins=20, range=[0., 1.])

ax.lim([0., 1.02], [0., 10.])

## Data Navigation

In [ ]:
u_m = sim_info.cosmology.unit_system.u_m_to_sol
m_lb, m_ub = 10**10.5 / u_m, 10**11.0 / u_m

s_h_parent = samp.subset_by_value('is_c', eq=True)\
    .subset_by_value('m_mean200', lo=m_lb, hi=m_ub)\
    .subset_by_value('last_sat_z', lo=15.)\
    .subset_by_value('is_recon_large', eq=True)
    
z_rgs = [0.01, 0.04], [0.04, 0.06], [0.06, 0.07]

z_fs = []
for i_z, (z_lb, z_ub) in enumerate(z_rgs):
    s_h = s_h_parent.subset_by_value('z', lo=z_lb, hi=z_ub)
    z_f = s_h['z_half']
    z_fs.append(z_f)
    
    
fig, ax = plot.subplots(1, figsize=5.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

cs = cs_dark2
for i_z, (z_lb, z_ub) in enumerate(z_rgs):
    z_f = z_fs[i_z]
    
    ax.hist(z_f, range=[0., 5.5], bins=32, 
            ec=cs[i_z], label=f'{z_lb} < z < {z_ub}', histtype='step', density=True)


ax.scale(y='log').leg(loc='ur')

# LGalaxies

In [ ]:
from obs.assembly_bias.clustering import SubSampleManager, SimCross

sim_info = sims.predefined['millennium_2_scaled_wmap7']
sim_dir = ProjPaths.man_sim_dir(sim_info)

## Datasets

In [ ]:
s_all = sample.LGalaxiesGuo13SampleRaw.load(in_path='MS2_z0_all.hdf5')
s_all.objs['m_star'] = s_all.objs['m_s']
s_all.objs['Sigma_star'] = s_all.objs['S']

In [ ]:
# store the result for simplicity
p = ProjPaths.sim_dir_of(sim_info) / 'G13_z0_all_postprocessed.hdf5'
assembly_bias.sample_dumper.dump_sample_to(p, 
    s_all.objs, sim_info, sim_info.n_snapshots-1)

In [ ]:
p = ProjPaths.sim_dir_of(sim_info) / 'G13_z0_all_postprocessed.hdf5'

m_200c = h5.File.load_from(p, 'objs/m_h')
lm_200c = np.log10(m_200c)
lm_200m = 1.0122627*lm_200c + 0.1165777     # for z=0
m_200m = 10.**lm_200m
h5.File.dump_to(p, {
    'm_mean200': m_200m, 'm_crit200': m_200c,
}, key='objs', dump_flag='ac', g_flag='ac')
h5.File.ls_from(p)

## Relative bias

In [ ]:
pdata = sim_dir.base_dir / 'G13_z0_all_postprocessed.hdf5'
man_kw = {
    'add_rsd': False,
    'dmo': False,
    'm_s_range_in_sol': [10**8.5, 10**9.],
}
sman = SubSampleManager(sim_info, pdata, **man_kw)
sman.s_dwarf

wp_kw = {
    'pimax': 10., 'n_repeat': 100,
    'bootstrap_kw': {'keep_samples': True}
}
sim_cross = SimCross(sman)

In [ ]:
o_path = sim_dir.base_dir / 'wp/dwarfs_lm8.5-9_by-S.json'

Sigma_star_lims = [-0.1, 7.,15., 25., 1.0e6]
wps = sim_cross.wps_of('s_dwarf', 'Sigma_star', ps=None, 
                       qs=Sigma_star_lims, **wp_kw)
wps['total'] = sim_cross.wp_of('s_dwarf', **wp_kw)
Json.dump_file(wps, o_path, indent=2)

In [ ]:
lm_h_edges = [11., 12., 13., 16.]
o_path = sim_dir.base_dir / 'wp/halos_by-M.json'

u_m = 1.0e10 / sman.sim_info.cosmology.hubble
m_h_edges = 10.0**np.array(lm_h_edges) / u_m
wps = sim_cross.wps_of('s_halo', 'm_mean200', ps=None, qs=m_h_edges, **wp_kw)

Json.dump_file(wps, o_path, indent=2)

In [ ]:
m_s_ranges = (
    #[10**7.5, 10**9.],
    #[10**8., 10**9.],
    [10**8.5, 10**9.],)
ofiles = (
    #'wp-lg-ms7.5-9.json',
    #'wp-lg-ms8-9.json',
    #'wp-lg-ms8.5-9.json',
    'wp/ms8.5-9_r2-10.json',)
lm_h_edges = [11., 12., 13., 16.]

pdata = ProjPaths.sim_dir_of(sim_info) / 'G13_z0_all_postprocessed.hdf5'
wp_kw = {
    'pimax': 10., 'n_repeat': 50,
    'bootstrap_kw': {'keep_samples': True},
    'rs': np.array([2., 10.]),
}
for i, (m_s_range, ofile) in enumerate(zip(m_s_ranges, ofiles)):
    
    sman = SubSampleManager(sim_info, pdata, m_s_range_in_sol=m_s_range)
    sim_cross = SimCross(sman)
    
    #wp_tot = sim_cross.wp_of('s_dwarf', **wp_kw)

    wp_S = sim_cross.wps_of('s_dwarf', 'Sigma_star', 
        ps=None, qs=[-0.1, 7.,15., 25., 1.0e6], **wp_kw)

    #if i == 0:
    #    u_m = 1.0e10 / sman.sim_info.cosmology.hubble
    #    m_h_edges = 10.0**np.array(lm_h_edges) / u_m
    #    wp_h = sim_cross.wps_of(
    #        's_halo', 'm_h', 
    #        ps=None, qs=m_h_edges, **wp_kw)

    data = DataDict({
        'dwarfs': {
            'S': wp_S, 
            #'tot': wp_tot,    
        },
        #'halos': wp_h,
    })

    path = ProjPaths.sim_dir_of(sim_info) / ofile
    with open(path, 'w') as f:
        json.dump(data, f, default=json_default, indent=2)

In [ ]:
hubble = g13_all.sim_info.cosmology.hubble
m_scale = 1.0e10 / hubble
m_s_lb = 10**7.5 / m_scale
m_s_ub = 10**9.0 / m_scale

ssfr_scale = hubble / 1.0e9
ssfr_sep = 1.0e-11 / ssfr_scale

m_h_key = 'm_h'
S_key = 'S'

m_s_lb, m_s_ub, ssfr_sep

In [ ]:
m_h_edges = 10.0**np.array([11.5, 12.0, 12.5, 13.0, 16.0]) / m_scale

In [ ]:
s_ref = g13_all.subset_by_value('m_s', lo=m_s_lb)
cls_ref = clustering.Clustering(s_ref)

s_c = g13_all.subset_by_value('is_c', eq=True)
s_dwarf = s_c.subset_by_value('m_s', lo=m_s_lb, hi=m_s_ub)\
    .subset_by_value('ssfr', ssfr_sep)

s_ref.n_objs, s_dwarf.n_objs

In [ ]:
cls_kw = {'pimax': 10., 'n_repeat': 50}

In [ ]:
subs_ref = SubSample(s_ref)
subs_halos = SubSample.list_from_bins(s_c, m_h_key, edges=m_h_edges)
subs_list = SubSample.list_from_bins(
    s_dwarf, S_key, edges=[-0.1, 7.,15., 25., 1.0e6])

In [ ]:
out_g = {str(i): subs_ref.proj_cross_with(subs, **cls_kw)  
         for i, subs in enumerate(subs_list) }

In [ ]:
out_tot = subs_ref.proj_cross_with(SubSample(s_dwarf), **cls_kw)

In [ ]:
out_h = {str(i): subs_ref.proj_cross_with(subs, **cls_kw)  
         for i, subs in enumerate(subs_halos) }

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'wp-result.hdf5'
h5.File.dump_to(path, {'dwarfs': {
    'S': out_g, 
    'tot': out_tot                                  
}, 'halos': out_h}, f_flag='w')

In [ ]:
subs_list

In [ ]:
d_out = {
    str(i): {'S': subs.samp[S_key]} for i, subs in enumerate(subs_list) 
}
d_out = {
    'samples': d_out,
    'meta': [
        'S: stellar mass surface density in [M_sun/pc^2].',
    ]
}
with open(ProjPaths.sim_dir_of(sim_info) / 'wp-samples.json', 'w') as f:
    json.dump(d_out, f, indent=2, default=json_default)

In [ ]:
# old impl
g13_res_g = DataDict({
    'xis': get_cls_binned(s_dwarf, cls_ref, n_rep=50)    
})

g13_res_h = DataDict({
    'xis': get_cls_binned(s_c, cls_ref, 
        'm_h', 10.0**np.array([
            (11.5, 12.0),
            (12.0, 12.5),
            (12.5, 13.0),
            (13.0, 16.0),
        ]) / m_scale,
        n_rep=50,
    )
})

path = ProjPaths.sim_dir_of(sim_info) / 'wp-result.hdf5'
h5.File.dump_to(path, {'dwarfs': g13_res_g, 'halos': g13_res_h}, f_flag='w')

### For tests

In [ ]:
g13_all = sample.LGalaxiesGuo13SampleRaw.load(in_path='MS2_z0_all.hdf5', name='All-LGalaxies')
g13_ref = g13_all.subset_by_value('m_s', lo=10**-2.65, name='Ref-LGalaxies')
g13cls = clustering.Clustering(g13_ref)
# g13_dwarf = sample.LGalaxiesGuo13Sample('z0_dwarf_sample.hdf5')

g13_c = g13_all.subset_by_value('is_c', eq=True, name=r'All\ Central')
g13_dwarf = g13_c.subset_by_value('m_s', lo=10**-2.65, hi=10**-1.15)\
    .subset_by_value('f_d', lo=.75).subset_by_value('ssfr', 1.0e-2, name=r'All\ Dwarfs')

In [ ]:
g13_udg = sample.LGalaxiesGuo13SampleUdg.load(in_path='z0_UDG_sample.hdf5', name=r'All\ UDG')
g13_udg.find_cen_flag(g13_all)
g13_udg_c = g13_udg.subset_by_value('is_c', name=r'UDG\ Cen')

In [ ]:
g13_all, g13_c, g13_dwarf, g13_udg

In [ ]:
g13outs = DataDict()

In [ ]:
def get_g13cls_S_binned():
    S_ranges = [
        (0., 1.),
        (1., 7.),
        (7., 15.),
        (25., 37.),
        (50., 100.),
    ]
    samp_from = g13_dwarf
    cls_by = g13cls
    n_rep = 5
    
    outs = []
    for s1, s2 in S_ranges:
        name = r'\Sigma_*=[{%.0f}, {%.0f})'%(s1, s2)
        samp = samp_from.subset_by_value('S', lo=s1, hi=s2)
        print(samp.n_objs)
        out = cls_by.proj_cross(samp, n_repeat=n_rep) | {'S_range': (s1,s2)}
        outs.append(out)
    return outs
g13outs['S_binned'] = get_g13cls_S_binned()

In [ ]:
g13outs['all'] = g13cls.proj_cross(g13_dwarf, n_repeat=5) | {'name': r'\rm All\ dwarfs'}

In [ ]:
s = g13_c.subset_by_value('m_h', lo=10**2.0, hi=10**2.5, 
    name=r'{\rm Halo}\,(M_{\rm h} = 10^{[12.,12.5]})'); print(s.n_objs)
g13outs['h12'] = g13cls.proj_cross(s, n_repeat=5) | {'name': s.name}
s = g13_c.subset_by_value('m_h', lo=10**3., name=r'{\rm Halo}\,(M_{\rm h} \geqslant 10^{13})'); print(s.n_objs)
g13outs['h13'] = g13cls.proj_cross(s, n_repeat=5) | {'name': s.name}

In [ ]:
# g13_udg_c.subset_by_value('m_s', lo=10**-2.65, hi=10**-1.15) # not neccesary
s = g13_udg_c.subset_by_value('ssfr', 1.0e-1)
g13outs['udg_sf'] = g13cls.proj_cross(s, n_repeat=5) | {'name': r'{\rm UDG}\,({\rm SF})'}

s = g13_udg_c
g13outs['udg_all'] = g13cls.proj_cross(s, n_repeat=5) | {'name': r'{\rm UDG}\,({\rm all})'}

In [ ]:
s = g13_udg_c.subset_by_value('ssfr', hi=1.0e-1)
g13outs['udg_q'] = g13cls.proj_cross(s, n_repeat=5) | {'name': r'{\rm UDG}\,({\rm Q})'}

In [ ]:
lg_ssfr_bins = np.array([[-11., -2.], [-2.5, -1.5], [-1., 0.], [0.,1.], [1.0, 100.0]])
ssfr_bins = 10**lg_ssfr_bins
for i in range(5):
    s = g13_udg.subset_by_value('m_s', lo=10**-2.65, hi=10**-1.15)\
        .subset_by_value('is_c', eq=True)\
        .subset_by_value('ssfr', *ssfr_bins[i], 
            name=r'{\rm UDG}\ \log\,sSFR=[%.1f,%.1f]'%tuple(lg_ssfr_bins[i]))
    g13outs[f'udg_ssfr{i}'] = g13cls.proj_cross(s, n_repeat=50) | {'name': s.name}

In [ ]:
g13outs.keys()

In [ ]:
fig, axs = plot.subplots(
    (2, 1),
    figsize=(6., 6.5),
    share=(True, 'row'),
    space=0, extent=[.98, .98, .09, .125],
    layout='none', 
    ratios=(None, (3, 1.25)))
axs_f = axs.flat
axs.fmt_fill(lw=0)

cs3 = ColorSets.tab10[:3]
c_b = ColorSets.set1[1]

def plt_rat(d, d_all, kw):
    x, y, e = d['lg_rs_c', 'wp', 'wp_sd']
    ax.plot(x, y, **kw)
    
    y_all = d_all['wp']
    y, e = Num.safe_div(y, y_all), Num.safe_div(e, y_all)
    ax1.plot(x, y, **kw)

ax = axs_f[0]
ax1 = axs_f[1]
d_all = g13outs[f'all']
plt_rat(d_all, d_all, dict(c='k', ls='-', lw=2.5, zorder=10, label=r'$\rm All\ dwarfs$'))

for i, d in enumerate(g13outs['S_binned']):
    lab = r'$\Sigma_*=[%0.f,%0.f]$'%d['S_range']
    kw = dict(ls=(0,(i+2,(4-i)*.5)), c=c_b, label= lab) 
    plt_rat(d, d_all, kw)
    
labs = r'${\rm UDG}\,({\rm sSFR}>0.1\,h{\rm Gyr}^{-1})$', r'${\rm UDG}\,({\rm sSFR}<0.1\,h{\rm Gyr}^{-1})$', r'$\rm UDG\,(all)$'
for i, k in enumerate(['udg_sf', 'udg_q', 'udg_all']):
    kw = dict(ls=(0,(2*i+2,2-i)), c=cs3[1], lw=2.5, label=labs[i])
    plt_rat(g13outs[k], d_all, kw)

labs = r'${\rm Halo}\,(M_{\rm h}=10^{[12,12.5]})$', r'${\rm Halo}\,(M_{\rm h}>10^{13})$'
for i, k in enumerate(['h12', 'h13']):
    kw = dict(c='gray', lw=(i+1)*2, label=labs[i])
    plt_rat(g13outs[k], d_all, kw)

ax.scale(y='log').lim([-1.89, 1.1], [.5e0, 5.9e3])\
    .label(y=r'w_{\rm p}(r_{\rm p})\,[h^{-1}{\rm Mpc}]',)\
    .leg(loc='ll', fontsize=12.25, ncol=2, columnspacing=.01,
         labelspacing=-.05, title=r'$\bf L$-$\bf Galaxies\,(Guo13)$', handlelength=1.5)

ax = axs[1]
ax.scale(y='log').lim(y=[0.5, 11.5]).label(r'\log\, r_{\rm p}\,[h^{-1}{\rm Mpc}]',
                           r'\rm Ratio')
ProjPaths.save_fig('lgal_wp.pdf')

# TNG50-1

In [ ]:
%autoreload
gc.collect()

from obs.assembly_bias.clustering import SubSampleManager, SimCross

sim_info = sims.predefined['tng_50_1']
sim_dir = ProjPaths.man_sim_dir(sim_info)

## Datasets

In [ ]:
s_all = sample.SimSample.load(sim_info, 'z0_all.hdf5')\
    .requires_spin().requires_assembly().requires_dist_to_halos()

# store the result for simplicity
p = ProjPaths.sim_dir_of(sim_info) / 'z0_all_postprocessed.hdf5'
assembly_bias.sample_dumper.dump_sample_to(p, 
    s_all.objs, sim_info, sim_info.n_snapshots-1)

p = ProjPaths.sim_dir_of(sim_info) / 'z0_all_postprocessed.hdf5'
h5.File.ls_from(p)

## Relative bias

In [ ]:
pdata = sim_dir.base_dir / 'z0_all_postprocessed.hdf5'
man_kw = {
    'add_rsd': False,
    'dmo': False,
    'm_s_range_in_sol': [10**8., 10**9.],
    # 'm_s_range_in_sol': [10**8.5, 10**9.],
}
sman = SubSampleManager(sim_info, pdata, **man_kw)
sman.s_dwarf

rs = np.concatenate([
    np.logspace(-2., 0., 5),
    np.logspace(0.15, 1.2, 8)
])
wp_kw = {
    'pimax': 10., 'n_repeat': 100,
    'bootstrap_kw': {'keep_samples': True},
    'rs': rs
    # 'rs': np.array([2., 10.]),
}
sim_cross = SimCross(sman)

In [ ]:
o_path = sim_dir.base_dir / 'wp/dwarfs_lm8-9_by-S.json'
# o_path = sim_dir.base_dir / 'wp/dwarfs_lm8.5-9_by-S.json'

Sigma_star_lims = [-0.1, 7.,15., 25., 1.0e6]
wps = sim_cross.wps_of('s_dwarf', 'Sigma_star', ps=None, qs=Sigma_star_lims, **wp_kw)
wps['total'] = sim_cross.wp_of('s_dwarf', **wp_kw)
Json.dump_file(wps, o_path, indent=2)

In [ ]:
lm_h_edges = [11., 12., 13., 16.]
o_path = sim_dir.base_dir / 'wp/halos_by-M.json'

u_m = 1.0e10 / sman.sim_info.cosmology.hubble
m_h_edges = 10.0**np.array(lm_h_edges) / u_m
wps = sim_cross.wps_of('s_halo', 'm_mean200', ps=None, qs=m_h_edges, **wp_kw)

Json.dump_file(wps, o_path, indent=2)

In [ ]:
m_s_ranges = ([10**7.5, 10**9.],
    [10**8., 10**9.],
    [10**8.5, 10**9.],)
ofiles = ('wp-tng50-ms7.5-9.json',
    'wp-tng50-ms8-9.json',
    'wp-tng50-ms8.5-9.json',)
rs = np.concatenate([
    np.logspace(-2., 0., 5),
    np.logspace(0.15, 1.2, 8)
])
lm_h_edges = [11., 12., 13., 16.]

pdata = ProjPaths.sim_dir_of(sim_info) / 'z0_all_postprocessed.hdf5'
wp_kw = {
    'pimax': 10., 'n_repeat': 50, 'rs': rs,
    'bootstrap_kw': {'keep_samples': True},
}

for i, (m_s_range, ofile) in enumerate(zip(m_s_ranges, ofiles)):
    sman = SubSampleManager(sim_info, pdata, m_s_range_in_sol=m_s_range)
    sim_cross = SimCross(sman)
    
    
    wp_tot = sim_cross.wp_of('s_dwarf', **wp_kw)
    
    wp_S = sim_cross.wps_of('s_dwarf', 'Sigma_star', 
        ps=None, qs=[-0.1, 7.,15., 25., 1.0e6], **wp_kw)

    if i == 0:
        u_m = 1.0e10 / sman.sim_info.cosmology.hubble
        m_h_edges = 10.0**np.array(lm_h_edges) / u_m
        wp_h = sim_cross.wps_of(
            's_halo', 'm_mean200', 
            ps=None, qs=m_h_edges, **wp_kw)

    data = DataDict({
        'dwarfs': {
            'S': wp_S, 'tot': wp_tot,    
        },
        'halos': wp_h,
    })
    
    path = ProjPaths.sim_dir_of(sim_info) / ofile
    with open(path, 'w') as f:
        json.dump(data, f, default=json_default, indent=2)

# TNG

In [ ]:
%autoreload
from obs.assembly_bias.clustering import SubSampleManager, SimCross
from scipy.interpolate import interp1d
from obs.assembly_bias.utils import gal_wp

sim_info = sims.predefined['tng']
sim_dir = ProjPaths.man_sim_dir(sim_info)

## Datasets

In [ ]:
s_all = sample.SimSample.load(sim_info, 'tng_z0.hdf5').requires_spin()\
    .requires_assembly().require_c()  # .requires_dist_to_halos()
s_all

In [ ]:
# store the result for simplicity
p = sim_dir.base_dir / 'z0_all_postprocessed.hdf5'
assembly_bias.sample_dumper.dump_sample_to(p, 
    s_all.objs, sim_info, sim_info.n_snapshots-1)
h5.File.ls_from(p)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
p = sim_dir.base_dir / 'z0_all_postprocessed.hdf5'
d_in = h5.File.load_from(p)
m_200m, m_200c = d_in['objs']['m_mean200', 'm_crit200']
lm_200m = np.log10(m_200m)
lm_200c = np.log10(m_200c)

In [ ]:
fig, ax = plot.subplots(1, figsize=4.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

mod = LinearRegression().fit(lm_200c[:, None], lm_200m)
k, b = mod.coef_[0], mod.intercept_

x = np.linspace(-3., 4.5, 64)
y = k * x + b
print(k, b)
ax.scatter(lm_200c, lm_200m, s=1)
ax.plot(x, y, 'k--', c='r')

## Relative bias, z_f, etc.

In [ ]:
m_s_range = [10**8.5, 10**9.]
pdata = ProjPaths.sim_dir_of(sim_info) / 'z0_all_postprocessed.hdf5'
kw = {
    'sim_info': sim_info, 'data_file': pdata, 'm_s_range_in_sol': m_s_range, 'add_rsd': False, 'dmo': False
}
smans = {
    'by_ssfr': SubSampleManager(**kw),
    'by_noej': SubSampleManager(**kw, ssfr_lb_in_sol=-1.0, baryon_excl_ej=True),
    'all': SubSampleManager(**kw, ssfr_lb_in_sol=-1.0),
}
#ssfr_scale = sman.sim_info.cosmology.hubble / 1.0e9

In [ ]:
wp_kw = {
    'pimax': 10., 'n_repeat': 100,
    'bootstrap_kw': {'keep_samples': True},
    'rs': np.array([2., 10.]),
}
sim_crosses = {k: SimCross(v) for k, v in smans.items()}

In [ ]:
ps = np.array([0., 0.02, 0.05, 0.10, 0.15, 0.20, 0.40, 0.60, 0.80, 0.85, 0.90, 
               0.95, 0.98, 1.0])

In [ ]:
S_ranges = [-1., 7.], [7., 15.], [15., 100000.]
d_wps = DataDict()
for k, sim_cross in sim_crosses.items():
    d_wps[k] = []
    for i_s, (S_lo, S_hi) in enumerate(S_ranges):
        wps = sim_cross.wps_of('s_dwarf', 'z_half', ps=ps, **wp_kw,
                         subsamp_sel = [ {'key': 'Sigma_star', 'lo': S_lo, 'hi': S_hi} ]
            )
        d_wps[k].append(wps)

In [ ]:
sman_allssfr_noej = SubSampleManager(sim_info, pdata, m_s_range_in_sol=m_s_range,
                                add_rsd=False, ssfr_lb_in_sol=-1.0, baryon_excl_ej=True)

In [ ]:
sman.s_dwarf, sim_cross

In [ ]:
sman_allssfr = SubSampleManager(sim_info, pdata, m_s_range_in_sol=m_s_range,
                                add_rsd=False, ssfr_lb_in_sol=-1.0)

In [ ]:
sman_allssfr.s_dwarf

In [ ]:
sman_allssfr_noej.s_dwarf

In [ ]:
sman.s_dwarf

In [ ]:
ps = np.concatenate(
    [
        np.linspace(0., 0.9, 8),
        np.linspace(0.92, 0.98, 6),
        np.linspace(0.99, 1., 4),
    ]
)
print(ps)

In [ ]:
wp_kw = dict(sman=sman, ps=ps, n_reps=10, pimax=10.)
wps = [
    gal_wp.WpVsProp(samp_name='s_diffuse_dwarf', samp_kw={'S_ub': 7.}, **wp_kw),
    gal_wp.WpVsProp(samp_name='s_diffuse_dwarf', samp_kw={'S_lb': 15.}, **wp_kw),
    gal_wp.WpVsProp(**wp_kw),
]

In [ ]:
outs = []
for wp in wps:
    wp.run()
    wp.summary(inv_cum=True)
    outs.append(wp.wps)

In [ ]:
wp_kw = dict(sman=sman_allssfr, ps=ps, n_reps=10, pimax=10.)
wps = [
    gal_wp.WpVsProp(samp_name='s_diffuse_dwarf', samp_kw={'S_ub': 7.}, **wp_kw),
    gal_wp.WpVsProp(samp_name='s_diffuse_dwarf', samp_kw={'S_lb': 15.}, **wp_kw),
    gal_wp.WpVsProp(**wp_kw),
]

In [ ]:
outs_ssfr = []
for wp in wps:
    wp.run()
    wp.summary(inv_cum=True)
    outs_ssfr.append(wp.wps)

In [ ]:
d_in = Path('/Users/yangyaochen/work/science/work_forecast/mahgic/run_dir/bin/obs/assembly_bias/wp_vs_prop')
outss = [
    Json.load_file(d_in / 'tng_z_half.json', parse=False),
    Json.load_file(d_in / 'tng_z_halfallssfr_noej.json', parse=False),
    Json.load_file(d_in / 'tng_z_halfallssfr.json', parse=False),
]

In [ ]:
fig, axs = plot.subplots((2,3), share=(True, 'row'),
    space=0.03, subsize=(4.25, 3.0), 
    margin=[0.02, 0.01, 0.12, 0.065], layout='none',
    ratios=(None, [2, 1.]))
axs_f = axs.flat

cs = 'b', 'g', 'r', 'k'
labels = r'$\Sigma_* < 7\,{\rm M}_\odot\,{\rm pc}^{-2}$', \
    r'$\Sigma_* = 7$-$15\,{\rm M}_\odot\,{\rm pc}^{-2}$',\
    r'$\Sigma_*\geqslant 15\,{\rm M}_\odot\,{\rm pc}^{-2}$', r'$\rm All\ \Sigma_*$'

labs_samp = r'\rm Star$-$\rm forming\,(main\ texts)', r'\rm Backsplash$-$\rm excluded', r'\rm All'
for i_outs, _outs in enumerate(outss):
    axs_c = axs[:, i_outs]
    for i_out, out in enumerate(_outs):
        ax = axs_c[0]
        wps = np.array(out['wps']) 
        y, y_sd = wps.mean(0), wps.std(0)
        y, y_sd = y / y[-1], y_sd / y[-1]
        
        x = np.array(out['cum_ps'])
        x2 = np.array(out['qs'] )

        axs.c(cs[i_out])
        ax.errorfill(x, y, yerr=y_sd, label=labels[i_out], lw=3,
                        fill_between_kw={'lw': 0., 'alpha': .15})

        ax = axs_c[1]
        ax.c(cs[i_out])
        ax.plot(x, x2)
        
        ax = axs_c[0]
        ax.text(labs_samp[i_outs], (.92, .92), ha='right')
        
axs[0].scale().lim([.5e-2, 1.1], [1., 3.25])\
    .label(y=r'{\rm Relative\ bias}')

axs[1].scale('log').lim(y=[0., 7.5])\
    .label(r'{\rm Fraction\ of \ }z_{\rm f} \geqslant z_{\rm f,thres}', r'z_{\rm f,thres}')

axs_f[0].leg()

axs.label_outer()
ProjPaths.save_fig('fig_rel_bias_vs_zf_sub_binned.pdf')

In [ ]:
s_dwarf = sman_allssfr.s_dwarf
ssfr_scale = sman.sim_info.cosmology.hubble / 1.0e9
z_h, S, ssfr = s_dwarf['z_half', 'Sigma_star', 'ssfr']
ssfr = ssfr * ssfr_scale

In [ ]:
fig, axs = plot.subplots((1,2), share=False, space=0.25, subsize=(5.0, 5.0), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

ax = axs_f[0]

rg = [0., .8], [-0.2, 2.2]
x, y = Num.safe_lg(z_h + 1.) , Num.safe_lg(S) 
ax.scatter_2d(x, y, range=rg, 
              n_bins=(32, 32)).mesh(cmap='gnuplot').contour(colors='w')
ax.lim(*rg).label(r'\log\, (1+z_{\rm f})', r'\log\, \Sigma_*\, [{\rm M_\odot\,pc^{-2}}]')

ax = axs_f[1]
rg = [-10.5, -8.], [-0.2, 2.2]
x, y = Num.safe_lg(ssfr, lo=1.0e-11), Num.safe_lg(S)
ax.scatter_2d(x, y, range=rg, 
              n_bins=(32, 32)).mesh(cmap='gnuplot').contour(colors='w')
ax.lim(*rg).label(r'\log\, {\rm sSFR}', r'\log\, \Sigma_*\, [{\rm M_\odot\,pc^{-2}}]')


In [ ]:
s1 = s_dwarf.subset_by_value('Sigma_star', hi=7.)
s1.subset_by_value('z_half', lo=0.5).n_objs

## Relative bias

In [ ]:
pdata = sim_dir.base_dir / 'z0_all_postprocessed.hdf5'
man_kw = {
    'add_rsd': False,
    'dmo': False,
    # 'm_s_range_in_sol': [10**8., 10**9.],
    'm_s_range_in_sol': [10**8.5, 10**9.],
}
sman = SubSampleManager(sim_info, pdata, **man_kw)
sman.s_dwarf

wp_kw = {
    'pimax': 10., 'n_repeat': 100,
    'bootstrap_kw': {'keep_samples': True}
    # 'rs': np.array([2., 10.]),
}
sim_cross = SimCross(sman)

In [ ]:
# o_path = sim_dir.base_dir / 'wp/dwarfs_lm8-9_by-S.json'
o_path = sim_dir.base_dir / 'wp/dwarfs_lm8.5-9_by-S.json'

Sigma_star_lims = [-0.1, 7.,15., 25., 1.0e6]
wps = sim_cross.wps_of('s_dwarf', 'Sigma_star', ps=None, qs=Sigma_star_lims, **wp_kw)
wps['total'] = sim_cross.wp_of('s_dwarf', **wp_kw)
Json.dump_file(wps, o_path, indent=2)

In [ ]:
lm_h_edges = [11., 12., 13., 16.]
o_path = sim_dir.base_dir / 'wp/halos_by-M.json'

u_m = 1.0e10 / sman.sim_info.cosmology.hubble
m_h_edges = 10.0**np.array(lm_h_edges) / u_m
wps = sim_cross.wps_of('s_halo', 'm_mean200', ps=None, qs=m_h_edges, **wp_kw)

Json.dump_file(wps, o_path, indent=2)

## Property distributions 

In [ ]:
pdata = sim_dir.base_dir / 'z0_all_postprocessed.hdf5'
man_kw = {
    'add_rsd': False,
    'dmo': False,
    # 'm_s_range_in_sol': [10**8., 10**9.],
    'm_s_range_in_sol': [10**8.5, 10**9.],
}
sman = SubSampleManager(sim_info, pdata, **man_kw)
sman.s_dwarf

z_f Sigma correlation

In [ ]:
z_f, S = sman.s_dwarf['z_half', 'Sigma_star']
lS = Num.safe_lg(S)

z_p = np.linspace(0., 5., 256)
S_p, el, eh = stats.KernelRegression1D.by_local_kernel(z_f, S, z_p, max_dx=.25, reduce='errorbar')['y'][0].T

In [ ]:
Json.dump_file(
    {
        'z_half': z_p, 'Sigma_star': S_p, 'Sigma_star_err': (el, eh)
    },
    sim_dir.base_dir / 'stats' / 'z_half-Sigma_star.json'
)

In [ ]:
fig, axs = plot.subplots((1,2), share=False, space=0.25, 
                         subsize=(5.0, 5.0), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

ax = axs_f[0]
ax.scatter(S, z_f, s=1, alpha=.5, rasterized=True)
ax.c('r').errorbar(S_p, z_p, xerr=(el, eh), lw=2.5)
ax.scale('log').lim(y=[0., 4.5])

ax = axs_f[1]
ax.hist(np.log10(S), range=[-1., 3.], bins=32)

In [ ]:
z_f, S, spin, c = sman.s_dwarf['z_half', 'Sigma_star', 'spin_mean200', 'c_mean200']
lS = Num.safe_lg(S)
lspin = Num.safe_lg(spin)

In [ ]:
fig, axs = plot.subplots(
    (1, 3),
    share=False, space=0.25, subsize=(5.5, 5.0),
    margin=[0.1, 0.1, 0.1, 0.1],
    layout='none')
axs_f = axs.flat


axs.c('r').fmt_marker(fa=1)
lspin_rg = [-2.55, -0.91]
c_rg = [2., 40.]
lS_rg = [-0.4, 3.]
lspin_label = r'\log\, \lambda'
c_label = r'c'
lS_lab = r'\log\, \Sigma_*\,[{\rm M_\odot pc^{-2}}]'

ax = axs_f[0]
ax.scatter(lspin, lS, rasterized=True)
ax.lim(lspin_rg, lS_rg).label(lspin_label, lS_lab)

ax = axs_f[1]
ax.scatter(c, lS, rasterized=True)
ax.lim(c_rg, lS_rg).label(c_label, lS_lab)

ax = axs_f[2]
X = np.column_stack([lspin, c])
X_mean, X_sd = X.mean(0), np.std(X, axis=0)
X_sc = (X - X_mean) / X_sd
y_sc = lS
y_p = stats.KernelRegressionND.by_knn(X_sc, y_sc, X_sc, k=32)['y'][0]
ax.fmt_marker(elw=0)
art = ax.scatter(lspin, c, c=y_p, rasterized=True, s=20, zorder=20, 
                 cmap='gnuplot2', vmin=lS_rg[0], vmax=lS_rg[1]).last_draw
ax.scatter(lspin, c, c='gray', rasterized=True, s=40, zorder=19)
ax.label(lspin_label, c_label)
    
n = 32
k = 32
grid = GridMd(np.linspace(*lspin_rg, n), np.linspace(*c_rg, n))
y_p = grid.applied(
    lambda Xp: stats.KernelRegressionND.by_knn(
        X_sc, y_sc, (Xp-X_mean)/X_sd, k=k)['y'][0],
    xy_indexing=True).values
x1p, x2p = grid.locs_1d
ax._raw.contour(x1p, x2p, y_p, levels=16, colors='w', 
                linewidths=1., zorder=21)
fig.colorbar(art, ax=ax, location='right', label='$%s$'%lS_lab)

ax.lim(lspin_rg, c_rg)

z_f - sSFR / Sigma correlation

In [ ]:
x, y, z = s_dwarf2['z_half', 'ssfr', 'Sigma_star']
y = np.log10(y * ssfr_scale + 1.0e-12) 
x = np.log10(1.0 + x)
z = np.log10(z + 1.0e-12)

In [ ]:
s_dwarf2.z_dst

In [ ]:
fig, axs = plot.subplots((2,2), share=False, space=.03, 
                         subsize=(5., 5.), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

ranges = [[0., 1.], [-12.3, -7.5], [-0.99, 2.2]]

ax = axs[0,0]
rg = ranges[0], ranges[1]
ax.scatter_2d(x, y, range=rg, 
              n_bins=(32, 32)).mesh(cmap='gnuplot').contour(colors='w')
ax.lim(*rg).label(y=r'\log\,{\rm sSFR}\,[{\rm yr}^{-1}]')
a_plot.Axes(ax).add_secondary_axis_lgzp1_to_z(label=r'$z_{\rm f}$')

axs[0,1].axis_off().text(r'\rm TNG', (.5, .5), ha='center', va='center', fontsize=20)

ax = axs[1,0]
rg = ranges[0], ranges[2]
ax.scatter_2d(x, z, range=rg, 
              n_bins=(32, 32)).mesh(cmap='gnuplot').contour(colors='w')
ax.lim(*rg).label(r'\log\, z_{\rm f}', r'\log\, \Sigma_*\, [{\rm M_\odot\,pc^{-2}}]')

ax = axs[1,1]
rg = ranges[1], ranges[2]
ax.scatter_2d(y, z, range=rg, 
              n_bins=(32, 32)).mesh(cmap='gnuplot').contour(colors='w')

ax.lim(*rg).label(r'\log\,{\rm sSFR}\,[{\rm yr}^{-1}]', 
    r'\log\, \Sigma_*\, [{\rm M_\odot\,pc^{-2}}]')

axs.label_outer()

In [ ]:
fig, ax = plot.subplots(1, figsize=5., margin=[0.1, 0.1, 0.1, 0.1], layout='none')

ax.scatter_2d(x, y, range=[[0., 1.], [-12.1, -7.8]], 
              n_bins=(32, 32)).mesh(cmap='gnuplot').contour(colors='w')

ax.text(r'\rm TNG', (.92, .92), color='w', ha='right')
ax.label(r'\log\,(1+z_{\rm f})', r'{\rm sSFR}\,[{\rm yr}^{-1}]')

### For tests

In [ ]:
m_s_min = 10**(-2. - 0.15)
s_ref = s_all.subset_by_value('m_star', lo=m_s_min)

In [ ]:
tng_cls = clustering.Clustering(s_ref)

In [ ]:
s_c = s_all.subset_by_value('is_c', eq=True)
s_c2 = s_c.subset_by_value('m_star', lo=m_s_min, hi=10**-1.15)
s_dwarf = s_c2.subset_by_value('ssfr', 10**-1.5)

In [ ]:
s_dwarfq = s_c2.subset_by_value('ssfr', hi=10**-1.5)

In [ ]:
s_noej = s_c2.subset_by_value('last_sat_z', lo=15.)
s_ej = s_c2.subset_by_value('last_sat_z', hi=1.5)

In [ ]:
def get_tng_cls_S_binned(samp_from = s_dwarf):
    S_ranges = [
        (0., 3.),
        (3., 7.),
        (10., 15.),
        (15., 30.),
    ]
    cls_by = tng_cls
    n_rep = 5
    
    outs = []
    for s1, s2 in S_ranges:
        samp = samp_from.subset_by_value('Sigma_star', lo=s1, hi=s2)
        print(samp.n_objs)
        out = cls_by.proj_cross(samp, n_repeat=n_rep) | {'S_range': (s1,s2)}
        outs.append(out)
    return outs

In [ ]:
tng_outs = {}
tng_outs['S_binned'] = get_tng_cls_S_binned()

In [ ]:
tng_outs['Sq_binned'] = get_tng_cls_S_binned(samp_from=s_dwarfq)

In [ ]:
tng_outs['all'] = tng_cls.proj_cross(s_dwarf, n_repeat=5)

In [ ]:
tng_outs['ej'] = tng_cls.proj_cross(s_ej, n_repeat=5)
tng_outs['noej'] = tng_cls.proj_cross(s_noej, n_repeat=5)

In [ ]:
tng_outs['allq'] = tng_cls.proj_cross(s_dwarfq, n_repeat=5)

In [ ]:
s = s_c.subset_by_value('m_h', lo=10**2.0, hi=10**2.5); print(s.n_objs)
tng_outs['h12'] = tng_cls.proj_cross(s, n_repeat=5)
s = s_c.subset_by_value('m_h', lo=10**3.); print(s.n_objs)
tng_outs['h13'] = tng_cls.proj_cross(s, n_repeat=5)

In [ ]:
s = s_noej.subset_by_value('S_fR', 10., 12.); print(s.n_subhs)
outs['noej_lows'] = cls.proj_cross(s, n_repeat=1)

In [ ]:
s = s_noej.subset_by_value('z_half', lo=3.).subset_by_value('d_h13', 5.); print(s.n_subhs)
outs['large_d_h12'] = cls.proj_cross(s, n_repeat=1)

In [ ]:
fig, axs = plot.subplots(
    (2, 1),
    figsize=(6., 6.5),
    share=(True, 'row'),
    space=0, extent=[.98, .98, .09, .125],
    layout='none', 
    ratios=(None, (3, 1.25)))
axs_f = axs.flat
axs.fmt_fill(lw=0)

cs3 = ColorSets.set1[:3]
c_b = cs3[1]
c_r = cs3[0]

def plt_rat(d, d_all, kw):
    x, y, e = d['lg_rs_c', 'wp', 'wp_sd']
    ax.plot(x, y, **kw)
    
    y_all = d_all['wp']
    y, e = Num.safe_div(y, y_all), Num.safe_div(e, y_all)
    ax1.plot(x, y, **kw)

ax = axs_f[0]
ax1 = axs_f[1]
d_all = tng_outs[f'all']
plt_rat(d_all, d_all, dict(c=c_b, ls='-', lw=2.5, zorder=10, label=r'$\rm All\ SF\ dwarfs$'))

d_allq = tng_outs[f'allq']
plt_rat(d_allq, d_all, dict(c=c_r, ls='-', lw=2.5, zorder=10, label=r'$\rm All\ Q\ dwarfs$'))

ds = tng_outs['S_binned']
for i, d in enumerate(ds):
    lab = r'$\Sigma_*=[%0.1f,%0.1f]$'%d['S_range']
    kw = dict(ls=(0,(0.5*i+.5,(len(ds)-i)*.5)), c=c_b, label= lab) 
    plt_rat(d, d_all, kw)
    
dsq = tng_outs['Sq_binned'][:-2]
for i, d in enumerate(dsq):
    kw = dict(ls=(0,(0.5*i+.5,(len(ds)-1-i)*.5)), c=c_r) 
    plt_rat(d, d_all, kw)
    
plt_rat(tng_outs['ej'], d_all, dict(c='purple', ls='-', lw=2.5, zorder=10, label=r'$\rm ejected$'))
plt_rat(tng_outs['noej'], d_all, dict(c='darkblue', ls='-', lw=2.5, zorder=10, label=r'$\rm non-ejected$'))


labs = r'${\rm Halo}\,(M_{\rm h}=10^{[12,12.5]})$', r'${\rm Halo}\,(M_{\rm h}>10^{13})$'
for i, k in enumerate(['h12', 'h13']):
    kw = dict(c='gray', lw=(i+1)*2, label=labs[i])
    plt_rat(tng_outs[k], d_all, kw)

ax.scale(y='log').lim([-1.89, 1.1], [.5e0, 5.9e3])\
    .label(y=r'w_{\rm p}(r_{\rm p})\,[h^{-1}{\rm Mpc}]',)\
    .leg(loc='ll', fontsize=12.25, ncol=2, columnspacing=.01,
         labelspacing=-.05, title=r'$\bf TNG100$', handlelength=1.5)

ax = axs[1]
ax.scale(y='log').lim(y=[0.5, 11.5]).label(r'\log\, r_{\rm p}\,[h^{-1}{\rm Mpc}]',
                           r'\rm Ratio')

ProjPaths.save_fig('tng_wp.pdf')

# TNG-Dark300-1

In [ ]:
%autoreload
from obs.assembly_bias.clustering import SubSampleManager, SimCross
from obs.assembly_bias import sidm
from sklearn.neighbors import KDTree
from obs.assembly_bias.utils import gal_wp
from scipy.interpolate import interp1d
from astropy import units as U

sim_info = sims.predefined['tng_dark_300_1']
sim_dir = ProjPaths.man_sim_dir(sim_info)

## Datasets

### LGalaxies-H15

In [ ]:
s_all = sample.SimSample.load(sim_info, 'z0_all.hdf5')\
    .require_LGalaxies_H15()\
    .requires_spin().requires_assembly().require_c()\
    #.requires_dist_to_halos().requires_disk()
s_all

In [ ]:
p = sim_dir.base_dir / 'z0_all_postprocessed.hdf5.LGalaxies-H15'
assembly_bias.sample_dumper.dump_sample_to(p, 
    s_all.objs, sim_info, sim_info.n_snapshots-1)

### Full sample

In [ ]:
s_all = sample.SimSample.load(sim_info, 'z0_all.hdf5').requires_spin().requires_assembly()\
    .require_c()
p = ProjPaths.sim_dir_of(sim_info) / 'z0_all_postprocessed.hdf5'
assembly_bias.sample_dumper.dump_sample_to(p, 
    s_all.objs, sim_info, sim_info.n_snapshots-1)

In [ ]:
p = ProjPaths.sim_dir_of(sim_info) / 'z0_all_postprocessed.hdf5'
h5.File.ls_from(p)

### Sub sample

In [ ]:
sim_info = sims.predefined['tng_dark_300_1']
sim_dir = ProjPaths.man_sim_dir(sim_info)

pdata = ProjPaths.sim_dir_of(sim_info) / 'z0_all_postprocessed.hdf5'
samp = sample.SimSample.load(sim_info, pdata)

In [ ]:
u_m = u_m = sim_info.cosmology.unit_system.u_m_to_sol
s_h = samp.subset_by_value('is_c', eq=True)\
    .subset_by_value('m_mean200', lo=10**10.50/u_m)\
    .subset_by_value('last_sat_z', lo=15.)
s_h

In [ ]:
keys = [
    'c_h', 'c_mean200', 'is_c',
    'last_sat_snap', 'last_sat_z', 'leaf_offs', 'm_crit200',
    'm_crit200_form', 'm_h', 'm_mean200', 'm_mean200_form', 'm_peak',
    'm_peak_snap', 'm_peak_v', 'm_sub', 'm_tophat', 'r_h', 'r_mean200',
    'r_tophat', 'r_v_max', 'root_offs', 'snap_afterform', 'snap_form',
    'spin', 'spin_form', 'spin_form_h', 'spin_form_mean200', 'spin_h',
    'spin_mean200', 'subfind_id', 'subhalo_id', 'v', 'v_h', 'v_max',
    'v_max2h', 'v_mean200', 'v_peak', 'v_peak2max', 'v_peak_m',
    'v_peak_snap', 'x', 'x_t_half', 'z_half'
]
d_out = {
    'ctx': h5.File.load_from(pdata, 'ctx'),
    'header': h5.File.load_from(pdata, 'header'),
    'objs': {k: s_h[k] for k in keys}
}

In [ ]:
h5.File.dump_to(
    str(pdata) + '.c.lm-gt-10p5.noBS',
    d_out, f_flag='w')

## Abundance matching

In [ ]:
sim_info = sims.predefined['tng_dark_300_1']
sim_dir = ProjPaths.man_sim_dir(sim_info)

pdata = ProjPaths.sim_dir_of(sim_info) / 'z0_all_postprocessed.hdf5'
samp = sample.SimSample.load(sim_info, pdata)
samp.objs['hid'] = np.arange(samp.n_objs)

Export subsamples

In [ ]:
u_m = sim_info.cosmology.unit_system.u_m_to_sol
# m_lb, m_ub = 10**10.9 / u_m, 10**11.1 / u_m
# o_key = '_lm11'
# m_lb, m_ub = 10**10.745 / u_m, 10**10.755 / u_m
# o_key = 'lm10p75'
# m_lb, m_ub = 10**10.49 / u_m, 10**10.51 / u_m
m_lb, m_ub = 10**10.75 / u_m, 10**10.85 / u_m
o_key = 'lm10p75-10p85'

s_h = samp.subset_by_value('is_c', eq=True)\
    .subset_by_value('m_mean200', lo=m_lb, hi=m_ub)\
    .subset_by_value('last_sat_z', lo=15.)
print(s_h)

In [ ]:
out = {k: s_h[k]  for k in ['m_mean200', 'r_mean200', 'c_mean200', 
                            'spin_mean200', 'z_half', 'hid']}
sim_dir.dump_sample(out, f'z0{o_key}.hdf5', f_flag='w')

Load and reorganize external results

In [ ]:
s_h.objs |= sim_dir.load_sample(f'z0{o_key}.hdf5')

key_in = f'Sigma_star_SIDM'
x_in = s_h[key_in]
x_out = np.zeros(len(samp['hid']), dtype=x_in.dtype)
x_out[ s_h['hid'] ] = x_in
    
h5.File.dump_to(pdata, {
    f'{key_in}_{o_key}': x_out,
}, 'objs', g_flag='ac', dump_flag='ac')

In [ ]:
out = {k: s_h[k]  for k in ['m_mean200', 'r_mean200', 'c_mean200', 'spin_mean200', 'z_half', 'hid']}
h5.File.dump_to(ProjPaths.sim_dir_of(sim_info) / 'z0_m10p75.hdf5', 
    out, f_flag='ac', dump_flag='ac')

In [ ]:
p_ld = ProjPaths.sim_dir_of(sim_info) / 'z0_m10p75.hdf5'
d_ld = h5.File.load_from(p_ld)
for k, v in d_ld.items():
    if k[:4] != 'sidm':
        continue
    print(k)
    v_out = np.zeros_like(samp['hid'], dtype=v.dtype) 
    v_out[d_ld['hid']] = v
    h5.File.dump_to(pdata, v_out, f'objs/lm10p75_{k}', dump_flag='ac')

In [ ]:
out = {k: s_h[k]  for k in ['m_crit200', 'c', 'z_half']}
h5.File.dump_to(ProjPaths.sim_dir_of(sim_info) / 'z0_m11.hdf5', 
    out, f_flag='ac')

### Matching with z_f

In [ ]:
u_m = sim_info.cosmology.unit_system.u_m_to_sol
m_lb, m_ub = 10**10.5 / u_m, 10**11. / u_m
s_h = samp.subset_by_value('is_c', eq=True)\
    .subset_by_value('m_mean200', lo=m_lb, hi=m_ub)\
    .subset_by_value('last_sat_z', lo=15.)

In [ ]:
s_g = obs_catalogs.tot_dwarfs.s_massive
len(s_g['z'])

In [ ]:
rng = Rng(10086)
rn = stats.RandomNoise(.001, rng=rng)
lS =  rn.add_to(np.log10(s_g['Sigma_star'])) 
lz_f = rn.add_to(np.log10(s_h['z_half'] + 1.))

for rho in [1., 0.85]:
    lS_pred = u_stats.abudance_match_with_corr_coef(lz_f, lS, rho=-rho, rng=rng)
    S_pred = 10**lS_pred
    
    S_all = np.zeros(len(samp['hid']), dtype=S_pred.dtype)
    S_all[ s_h['hid'] ] = S_pred
    h5.File.dump_to(pdata, S_all, 
                    f'objs/Sigma_star_AM_lm10p5-11_rho{rho:.2f}', dump_flag='ac')

### SIDM relations

In [ ]:
from obs.assembly_bias.sidm.drivers import SIDMDriver

In [ ]:
u_m = sim_info.cosmology.unit_system.u_m_to_sol
m_lb, m_ub = 10**10.5 / u_m, 10**11. / u_m
s_h = samp.subset_by_value('is_c', eq=True)\
    .subset_by_value('m_mean200', lo=m_lb, hi=m_ub)\
    .subset_by_value('last_sat_z', lo=15.)
    
n = s_h.n_objs
rng = Rng(0)
sel = rng.choice(n, 4096*2, replace=False)
s_h = s_h.subset(sel)

In [ ]:
model = SIDMDriver(s_h)
model.require_rho0()

In [ ]:
model_noada = SIDMDriver(s_h, ada_model=None)
model_noada.require_rho0()

In [ ]:
model_no_sidmb = SIDMDriver(s_h, sidm_with_b=False)
model_no_sidmb.require_rho0()

In [ ]:
model_no_b = SIDMDriver(s_h, sidm_with_b=False, ada_model=None)
model_no_b.require_rho0()

In [ ]:
model_01 = SIDMDriver(s_h, sigma_in_cm2perg=0.1)
model_01.require_rho0()

In [ ]:
model_10 = SIDMDriver(s_h, sigma_in_cm2perg=1.0)
model_10.require_rho0()

In [ ]:
def _sum_1(x, y, z):
    X = np.column_stack([x, y])
    X_mean, X_sd = X.mean(0), np.std(X, axis=0)
    X_sc = (X - X_mean) / X_sd
    z_p = stats.KernelRegressionND.by_knn(X_sc, z, X_sc, k=64)['y'][0]
    
    y_lo, y_hi = np.quantile(y, [.01, .99])
    y_p = np.linspace(y_lo, y_hi, 64)
    x_p, el, eh = stats.KernelRegression1D.by_local_kernel(y, x, y_p, reduce='errorbar')['y'][0].T
    
    x_lo, x_hi = np.quantile(x, [.01, .99])
    x_p2 = np.linspace(x_lo, x_hi, 64)
    y_p2, el2, eh2 = stats.KernelRegression1D.by_local_kernel(x, y, x_p2, reduce='errorbar')['y'][0].T
    
    return DataDict({
        'x': x, 'y': y, 'z': z, 'z_p': z_p, 
        'x_p': x_p, 'y_p': y_p, 'el': el, 'eh': eh,
        'x_p2': x_p2, 'y_p2': y_p2, 'el2': el2, 'eh2': eh2,
    })
    
def sum_sidm(model: SIDMDriver):
    r1, R_eff, S, rho0 = model.data['r1', 'R_eff', 'S', 'rho0']
    z_f = s_h.objs['z_half']

    r1 = r1 * model.u_l * 1.0e-3
    R_eff = R_eff * model.u_l * 1.0e-3
    rho0 = rho0 * model.u_m / model.u_l**3 * 1.0e9
    lrho0 = np.log10(rho0)
    lS = np.log10(S)
    
    out = []
    for i_x, x in enumerate([z_f, R_eff, lS]):
        y, z = r1, lrho0
        out.append(_sum_1(x, y, z)) 
        
    return out

In [ ]:
d_sidms = {
    'all': sum_sidm(model),
    'no_adam': sum_sidm(model_noada),
    'no_sidmb': sum_sidm(model_no_sidmb),
    'no_b': sum_sidm(model_no_b),
    '0.1': sum_sidm(model_01),
    '1.0': sum_sidm(model_10),
}

In [ ]:
obs_catalogs.tot_dwarfs.s_massive

In [ ]:
fig, ax = plot.subplots(1, figsize=(6., 5.), margin=[0.1, 0.1, 0.1, 0.1], layout='none')


mods = model, model_no_b, model_noada, model_no_sidmb, model_01, model_10
lws = 3, 2,2,1.5,2,2
lss = '-', (0,(1,1)), (0,(3,2)), '-', (0,(1,1)), (0,(3,2))
cs = 'k', 'r', 'r', 'r', 'b', 'b'
for i_mod, mod in enumerate(mods):
    r1, R_eff = mod.data['r1', 'R_eff']
    r1 = r1 * model.u_l * 1.0e-3
    R_eff = R_eff * model.u_l * 1.0e-3
    
    lr1, lr_e = np.log10(r1), np.log10(R_eff)    
    h, e = np.histogram(lr1, range=(-0.75, 1.), bins=32, density=True)
    c = .5 * (e[:-1] + e[1:])
    ax.plot(c, h, lw=lws[i_mod], ls=lss[i_mod], c=cs[i_mod])
    
r_e = obs_catalogs.tot_dwarfs.s_massive['r_50']
lr_e = Num.safe_lg(r_e)

h, e = np.histogram(lr_e, range=(-0.75, 1.), bins=32, density=True)
c = .5 * (e[:-1] + e[1:])
ax.plot(c, h, lw=3, c='grey', label=r'$\rm Dwarfs\ r_{\rm e}\,(Complete)$')
    
ax.scale(y='log').lim([-0.75, 1.], [1.0e-2, 10.])\
    .label(r'\log\, r_{c}\,[{\rm kpc}]', r'\rm PDF')\
    .leg(loc='ul')

In [ ]:
fig, axs = plot.subplots((1,3), share=(False, True),
    space=0.03, subsize=(4.75, 4.5), 
    margin=[0.02, 0.02, 0.14, 0.035], layout='none')

axs_f = axs.flat

scat_kw = dict(rasterized=True)

axs.fmt_marker(elw=0, fa=1)
x_labs = r'z_{\rm f}', r'R_{\rm eff}\,[{\rm kpc}]', r'\log\,\Sigma_*\,[{\rm M_\odot\,pc^{-2}}]'
x_lims = [-0.1, 3.9], [-.3, 5.75], [.3, 2.75]

lws = 2,2,1.5
lss = (0,(1,1)), (0,(3,2)), '-'
labs = r'$\rm No\ baryon\ effect$', r'$\rm +\rho_{\rm b}\ in\ Poisson$', r'$\rm +Adiabatic\ contraction$'
for i_k, k in enumerate(['no_b', 'no_adam', 'no_sidmb']):
    for i_x, d in enumerate(d_sidms[k]):
        ax = axs_f[i_x]
        lab = labs[i_k] if i_x == 1 else None
        # x_p, y_p, el, eh = d['x_p', 'y_p', 'el', 'eh']
        # ax.plot(x_p, y_p, c='r', lw=lws[i_k], ls=lss[i_k], alpha=1, zorder=3, label=lab)
        x_p, y_p, el, eh = d['x_p2', 'y_p2', 'el2', 'eh2']
        ax.plot(x_p, y_p, c='r', lw=lws[i_k], ls=lss[i_k], alpha=1, zorder=3, label=lab)
        
lws = 2,2
lss = (0,(1,1)), (0,(3,2))
labs = r'$\sigma_{\rm m}=0.1\,{\rm cm^2/g}$', r'$\sigma_{\rm m}=1.0\,{\rm cm^2/g}$'
for i_k, k in enumerate(['0.1', '1.0']):
    for i_x, d in enumerate(d_sidms[k]):
        ax = axs_f[i_x]
        lab = labs[i_k] if i_x == 2 else None
        #x_p, y_p, el, eh = d['x_p', 'y_p', 'el', 'eh']
        #ax.plot(x_p, y_p, c='b', lw=lws[i_k], ls=lss[i_k], alpha=1, zorder=3, label=lab)
        x_p, y_p, el, eh = d['x_p2', 'y_p2', 'el2', 'eh2']
        ax.plot(x_p, y_p, c='b', lw=lws[i_k], ls=lss[i_k], alpha=1, zorder=3, label=lab)
    
for i_x, d in enumerate(d_sidms['all']):
    ax = axs_f[i_x]
    
    x, y, z_p = d['x', 'y', 'z_p']
    norm = plot.abc.mpl.colors.Normalize(7.8, 8.8)
    cb = ax.scatter(x, y, **scat_kw, zorder=-1, c=z_p, s=25,
                    norm=norm, cmap='Greens').last_draw
    ax.scatter(x, y, **scat_kw, zorder=-2, c='k', s=35)
    
    #x_p, y_p, el, eh = d['x_p', 'y_p', 'el', 'eh']
    #ax.c('k')
    #lab = r'${\rm SIDM}\,(\sigma_{\rm m}=0.5\,{\rm cm^2/g})$' if i_x == 0 else None
    #ax.plot(x_p, y_p, lw=4, zorder=4, label=lab)
    #ax.plot(x_p-el, y_p, lw=1, zorder=4)
    #ax.plot(x_p+eh, y_p, lw=1, zorder=4)
    #ax._raw.fill_betweenx(y_p, x_p-el, x_p+eh, alpha=.25, zorder=2)

    x_p, y_p, el, eh = d['x_p2', 'y_p2', 'el2', 'eh2']
    ax.c('k')
    lab = r'${\rm SIDM}\,(\sigma_{\rm m}=0.5\,{\rm cm^2/g})$' if i_x == 0 else None
    ax.plot(x_p, y_p, lw=4, zorder=4, label=lab)
    ax.plot(x_p, y_p-el, lw=1, zorder=4)
    ax.plot(x_p, y_p+eh, lw=1, zorder=4)
    ax._raw.fill_between(x_p, y_p-el, y_p+eh, alpha=.25, zorder=2)
    
    ax.lim(x_lims[i_x], [-.3, 5.75]).label(x_labs[i_x], r'r_{\rm c}\,[{\rm kpc}]')

fig.colorbar(cb, ax=axs, 
             label=r'$\log\, \rho_0\,[{\rm M_\odot\,kpc^{-3}}]$', 
             aspect=30, fraction=.05, pad=.003)
x = [-1, 20]
y = x 
axs_f[1].plot(x, y, c='k', lw=1., zorder=-10)
axs.label_outer()

kw = dict(labelcolor='linecolor', fontsize=12.5, handlelength=2.)
axs_f[0].leg(loc='lr', **kw)
axs_f[1].leg(loc='lr', **kw)
axs_f[2].leg(loc='ll', **kw)

# ProjPaths.save_fig('fig_sidm_pred.pdf')

### c - z_f relation

In [ ]:
sim_info = sims.predefined['tng_dark_300_1']
pdata = ProjPaths.sim_dir_of(sim_info) / 'z0_all_postprocessed.hdf5'
man_kw = {'add_rsd': True, 'dmo': True}

sman = SubSampleManager(sim_info, pdata, **man_kw)

In [ ]:
c, zh = sman.s_dwarf['c', 'z_half']

In [ ]:
fig, ax = plot.subplots(1, figsize=4.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

ax.scatter(zh, c)
zp = np.linspace(0., 5., 32)
cp = stats.KernelRegression1D.by_local_kernel(zh, c, zp, max_dx=.1, reduce='median')['y'][0]

ax.plot(zp, cp, c='r', lw=2.5)

In [ ]:
p_data = ProjPaths.sim_dir_of(sim_info) / 'data_tables' / 'z_half2c_dwarfs.json'
Json.dump_file({
    'z_half': zp, 'c': cp,
}, p_data, indent=2)

### Bias vs halo property

Some trials

In [ ]:
pdata = ProjPaths.sim_dir_of(sim_info) / 'z0_all_postprocessed.hdf5'
man_kw = {
    'add_rsd': True, 
    'dmo': True,
    'm_h_range_in_sol': [10**10.745, 10**10.755],
    'm_h_ref_lb': 10**10.5,
}
sman = SubSampleManager(sim_info, pdata, **man_kw)
sman.s_dwarf

In [ ]:
ps = 1.0 - np.logspace(0., -3., 32)
ps = np.concatenate([ps, [1.0]])
print(ps)
wp = gal_wp.WpVsProp(sman, ps=ps, n_reps=10)

In [ ]:
wp.run()
wp.summary()
wp.wps

In [ ]:
d_in = ProjPaths.sim_dir_of(sim_info) / 'wp_vs_prop'
outs = Json.load_file(d_in/'tng_dark_300_1_z_half.json', parse=False) + \
    Json.load_file(d_in/'tng_dark_300_1_z_half_mh-high.json', parse=False)

From Ziwen: 

所有样本的红移都是 < 0.025

然后用这些样本(p%)的成团性 / compact dwarf 的成团性，并在 2-10 Mpc/h 的范围内拟合出 relative bias。
p=5, 10, 20, 25, 100

在数据中，第一列代表是 relative bias 的值，第二列是向下的误差，第三列是向上的误差。

In [ ]:
# z < 0.025, all halos
#d_obs = np.array( [[0.05, 2.23642895469386, 0.2663679438783164, 0.2678282379958272,],
#[0.1, 1.6993961151798982, 0.19360213711139962, 0.19180111810986644,],
#[0.2, 1.567316629158932, 0.1784526765909622, 0.17588837697628823,],
#[0.25, 1.6042531792508226, 0.16490115899612268, 0.16770357652826884,],
#[1., 1.3240592698755504, 0.09927722755607515, 0.09779923960316461,]])

# z < 0.04, massive dwarf with M_* = 10^{8.5-9} Msun
d_obs = np.array(
[
[2.5/100.0, 2.423472572866024, 0.25756286102913784, 0.2599523742102381],
[10/100.0, 1.739430133386184, 0.1191584230966738, 0.12209993713811373],
#[20/100.0, 1.5781420528462617, 0.11252922681410715, 0.11011750841174694],
[25/100.0, 1.5360601548401411, 0.10454498808989388, 0.10554346878574306],
[50/100.0, 1.3570793923244775, 0.07830968863516885, 0.07771558495622499],
[100/100.0, 1.2036625675575139, 0.05912973032857782, 0.060003700546162],
])

In [ ]:
fig, axs = plot.subplots((2,1), share=(True, False),
    space=0.03, subsize=(6., 3.0), 
    margin=[0.02, 0.02, 0.09, 0.135], layout='none',
    ratios=(None, [2, 1.]))
axs_f = axs.flat

cs = 'k', 'grey'
labels = r'$\rm M_{\rm h}=10^{10.5-11}\,{\rm M}_\odot$',\
    r'$\rm M_{\rm h}=10^{10.9-11.1}\,{\rm M}_\odot$'

for i_out, out in enumerate(outs):
    ax = axs_f[0]
    wps = np.array(out['wps']) 
    y, y_sd = wps.mean(0), wps.std(0)
    y, y_sd = y / y[-1], y_sd / y[-1]
    
    x = out['cum_ps']
    x2 = out['qs'] 
    p2q = interp1d(x, x2, kind='slinear')

    ax.c(cs[i_out])
    ax.errorfill(x, y, yerr=y_sd, lw=2.5, fill_between_kw={'lw': 0., 'alpha': 0.5})

    ax = axs_f[1]
    ax.c(cs[i_out])
    ax.plot(x, x2, 
        label=labels[i_out])
    
ax = axs_f[0]

x, y, el, eh = d_obs.T
y = y / y[-1]
x2 = p2q(x)
axs.c('b').fmt_marker(elw=2, s=8)
ax.errorbar(x[[0]], y[[0]], yerr=[el[[0]], eh[[0]]], lw=0, capsize=1.5,
            marker='s', label=r'$\Sigma_{\rm *,thres} = 7\,{\rm M}_\odot\,pc^{-2}$')
axs.c('green')
ax.errorbar(x[1:], y[1:], yerr=[el[1:], eh[1:]], lw=0, capsize=1.5,
            marker='s', label=r'$\rm {\rm Varied\ } \Sigma_{\rm *,thres}$')

#axs.c('k')
#for i in range(len(x2)):
#    ax = axs_f[0]
#    ax.plot((x2[i],)*2, [-10., 2.], lw=1, ls='--', )
#    ax = axs_f[1]
#    ax.plot((x2[i],)*2, [0.01, 1.2], lw=1, ls='--', )

leg_kw = dict(labelcolor='linecolor', handlelength=1.0, numpoints=1, 
              fontsize=12, title_fontsize=12)
ax = axs_f[0]
ax.scale('log').lim([3.0e-3, 1.25], [0.9, 2.75])\
    .label(y=r'{\rm Relative\ bias}')\
    .leg(**leg_kw,
        title=r'$\rm Dwarfs\,(M_*=10^{8.5 - 9}\,{\rm M}_\odot\,,z \leqslant 0.04)$')

ax = axs_f[1]    
ax.scale('log').lim(y=[0.25, 3.75])\
    .label(r'{\rm Fraction\ of\ }z_{\rm f} > z_{\rm f, thres}\ {\rm or}\ \Sigma_* < \Sigma_{\rm *,thres}', r'z_{\rm f,thres}')\
    .leg(loc=(0.05, 1.1), **leg_kw, title=r'$\rm Simulated\ halos$')
    
#ProjPaths.save_fig('fig_rel_bias_vs_zf_vs_obs.pdf')

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
y, y_sd = out.mean(0), out.std(0)
y, y_sd = y / y[-1], y_sd / y[-1]
x = 1.0 - np.array(d_run['ps'])[:-1][::-1]
# x2 = -np.quantile(-sman.s_dwarf['z_half'], x) 

In [ ]:
fig, ax = plot.subplots(1, figsize=(6., 5.), margin=[0.1, 0.1, 0.1, 0.1], layout='none')

ax.errorbar(x, y, yerr=y_sd, capsize=0, elinewidth=1.5, lw=3.5)

ax.scale('log').lim([1e-3, 1.], [0.9, 3.0])\
    .label(r'p', r'{\rm Relative\ bias\ for\ }\,\mathcal{P}(z_{\rm f}) > 1-p')
    
fn = interp1d(x2[::-1], x[::-1], kind='linear', fill_value='extrapolate')
x_tks = fn([1, 1.5, 2, 2.5, 3, 3.5])
for i_x, x_tk in enumerate(x_tks):
    ax.plot([x_tk, x_tk], [0., 3.0], c=cs_set2[0], ls='-', lw=1.5, alpha=.75)

## 2PCF with abundance matching

In [ ]:
pdata = ProjPaths.sim_dir_of(sim_info) / 'z0_all_postprocessed.hdf5'
man_kw = {
    'add_rsd': True, 
    'dmo': True,
    'm_h_range_in_sol': [10**10.75, 10**10.85],
    #'m_h_range_in_sol': [10**10.745, 10**10.755],
    #'m_h_range_in_sol': (10**10.9, 10**11.1)
    #'m_h_range_in_sol': (10**10.49, 10**10.51),
    'm_h_ref_lb': 10**10.5,
}
sman = SubSampleManager(sim_info, pdata, **man_kw)
sman.s_dwarf

In [ ]:
def take_wps_rel(d_in: DataDict, d_rel: DataDict):
    wp_rel = d_in['wp_samples'] / d_rel['wp_samples']
    assert wp_rel.shape[1] == 1
    wp_rel = wp_rel[:, 0]
    y, yl, yh = np.median(wp_rel), np.quantile(wp_rel, 0.16), np.quantile(wp_rel, 0.84)
    yel, yeh = y - yl, yh - y
    x, (xl, xh) = d_in['sub_sample']['median', '1sigma']
    xel, xeh = x - xl, xh - x
    return x, y, yel, yeh

def take_wps_rels(d_ins: DataDict, d_rel: DataDict = None):
    n_ds = len(d_ins)
    if d_rel is None:
        d_rel = d_ins[str(n_ds-1)]
    wp_rel = []
    for i in range(n_ds):
        wp_rel.append(take_wps_rel(d_ins[str(i)], d_rel))
    wp_rel = np.array(wp_rel)
    return wp_rel

ps = np.cumsum([0.02467638, 0.23220065, 0.25970874, 0.48341424])
ps[-1] = 1.
ps = np.concatenate([[0.], ps])
ps_obs = ps

ps_fine = np.array([0.0, 0.02, 0.04, 0.06, 0.08, 0.14, 0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9, 1.0])

ps_list = {
    'obs_bin': ps_obs,
    'fine_bin': ps_fine,
}

In [ ]:
wp_kw = {'pimax': 40., 'n_repeat': 25, 'rs': np.array([2., 10.]),
         'bootstrap_kw': {'keep_samples': True}}
sim_cross = SimCross(sman)

In [ ]:
#prop_pref = 'Sigma_star_SIDM_lm10p75-10p85'
prop_pref = 'spin_mean200'

prop_key = f'{prop_pref}'
vals = sman.s_dwarf.objs[prop_key]
print(prop_key, vals.min(), vals.max())

o_data = {
    k: sim_cross.wps_of('s_dwarf', prop_key, ps = ps, **wp_kw) 
    for k, ps in ps_list.items()
}
o_path = ProjPaths.sim_dir_of(sim_info) / f'wp/am/{prop_key}.hdf5'
Json.dump_file(o_data, o_path, indent=2)

In [ ]:
wps = sim_cross.wps_of('s_dwarf', 'S_pred_massive', ps = ps, **wp_kw)
ofile = 'wp/S-matched.json'
ofile = 'wp/S-matched-obsbin.json'
path = ProjPaths.sim_dir_of(sim_info) / ofile
Json.dump_file(wps, path, indent=2)

In [ ]:
#samp_k = 'lm10p75'
samp_k = 'lm10p5'
for prop_k in 'z_half', 'c_mean200':
    comb_key = f'S_pred_{samp_k}_{prop_k}'
    wps = sim_cross.wps_of('s_dwarf', comb_key, ps = ps_obs, **wp_kw)
    Json.dump_file(wps, ProjPaths.sim_dir_of(sim_info) / f'wp/by_{comb_key}-obsbin.json', indent=2)

In [ ]:
for k in 'lm10p75_sidm_S0p5', 'lm10p75_sidm_S1', 'lm10p75_sidm_S2':
    wps = sim_cross.wps_of('s_dwarf', f'S_pred_{k}', 
                           ps = ps_obs, **wp_kw)
    Json.dump_file(wps, ProjPaths.sim_dir_of(sim_info) / f'wp/by_S_pred_{k}-obsbin.json', indent=2)

In [ ]:
for k in '0.1', '0.2', '0.3', '0.5', '1.0', '2.0':
    #wps = sim_cross.wps_of('s_dwarf', f'S_pred_massive_rn{k}', ps = ps, **wp_kw)
    #Json.dump_file(wps, ProjPaths.sim_dir_of(sim_info) / f'wp/S-matched_rn{k}.json', indent=2)
    
    wps = sim_cross.wps_of('s_dwarf', f'S_pred_massive_rn{k}', ps = ps_obs[-2:], **wp_kw)
    Json.dump_file(wps, ProjPaths.sim_dir_of(sim_info) / f'wp/S-matched_rn{k}-obsbin.json', indent=2)

In [ ]:
for k in '0.1', '0.2', '0.3', '0.5', '1.0', '2.0':
    r1_key = f'r1_sigma0.1_rn{k}'
    wps = sim_cross.wps_of('s_dwarf', f'S_pred_massive_{r1_key}', ps = ps, **wp_kw)
    Json.dump_file(wps, ProjPaths.sim_dir_of(sim_info) / f'wp/S-matched_{r1_key}.json', indent=2)
    
    wps = sim_cross.wps_of('s_dwarf', f'S_pred_massive_{r1_key}', ps = ps_obs[-2:], **wp_kw)
    Json.dump_file(wps, ProjPaths.sim_dir_of(sim_info) / f'wp/S-matched_{r1_key}-obsbin.json', indent=2)

In [ ]:
for sidm_s in [
    0.01, 
    0.1, 
    0.2, 1.0, 5.0, 10.0, 100.0
    ]:
    r1_key = f'r1_sigma{sidm_s:.1f}'
    key = f'S_pred_massive_{r1_key}'
    wps = sim_cross.wps_of('s_dwarf', key, ps = ps, **wp_kw)
    Json.dump_file(wps, ProjPaths.sim_dir_of(sim_info) / f'wp/S-matched_{r1_key}.json', indent=2)
    
    wps = sim_cross.wps_of('s_dwarf', key, ps = ps_obs, **wp_kw)
    Json.dump_file(wps, ProjPaths.sim_dir_of(sim_info) / f'wp/S-matched_{r1_key}-obsbin.json', indent=2)

In [ ]:
wps = DataDict()

for prop_key in 'Sigma_star_SIDM_lm10p75-10p85', 'spin_mean200':
    d_in = Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/am/{prop_key}.hdf5')
    wps[prop_key] = {
        'fine_bin': take_wps_rels(d_in['fine_bin'], d_in['obs_bin/3']),
        'obs_bin': take_wps_rels(d_in['obs_bin'])
    }

wps['tng'] = take_wps_rels(Json.load_file(ProjPaths.sim_dir_of(sims.predefined['tng']) / 'wp/ms8.5-9_r2-10.json')['dwarfs/S'])
wps['lgal'] = take_wps_rels(Json.load_file(ProjPaths.sim_dir_of(sims.predefined['millennium_2_scaled_wmap7']) / 'wp/ms8.5-9_r2-10.json')['dwarfs/S'])

wps['obs'] = np.array([
    [6.2,    2.4, 0.28, 0.28,],
    [11.37,  1.44, 0.1, 0.1,],
    [19.33,  1.29, 0.09, 0.09,],
    [42.37, 1.0, 0.0, 0.0,],
])

In [ ]:
fig, ax = plot.subplots((1,1), share=(True, False),
    space=0.03, subsize=(6.5, 5.), 
    margin=[0.02, 0.125, 0.1, 0.135], layout='none')


mss = 8,7,7
cs = cs_dark2[1], cs_dark2[0], cs_dark2[2]
lws = .75, 2., 2.
labs = r'$\rm Observation$', r'$\rm TNG$', r'$\rm L$-$\rm Galaxies$'
mks =   'o', 's', 's'
for i, k in enumerate(('obs', 'tng', 'lgal')):

    x, y, el, eh = wps[k].T
    ax.c(cs[i]).errorbar(x, y, yerr=(el, eh), lw=lws[i], marker=mks[i], 
                           mew=2, ms=mss[i], capsize=0, elinewidth=1.5, 
                           label=labs[i], zorder=10)
    
cs = 'k', 'grey'
labs = r'$\rm Disk\ model\ with\ SIDM$', r'\rm Spin'
for i, k in enumerate(('Sigma_star_SIDM_lm10p75-10p85', 'spin_mean200')):    
    ax.c(cs[i])
    if i == 0:
        x, y, el, eh = wps[f'{k}/obs_bin'].T
        ax.errorbar(x, y, yerr=(el, eh), lw=0, marker='s', 
                            mew=2, ms=9, capsize=0, elinewidth=.5,
                            label=labs[i], zorder=9)
        x, y, el, eh = wps[f'{k}/fine_bin'].T
        ax.errorfill(x, y, yerr=(el, eh), lw=1., ls='-',
                    fill_between_kw={'lw': 0., 'alpha': .2}, zorder=9
                    )
    
ax.scale(x='log').lim([10**0.35, 1.0e2], [0.55, 2.75])
ax.label(r'\Sigma_*\,[{\rm M}_\odot\,{\rm pc}^{-2}]', r'{\rm Relative\ bias}')
ax.leg(loc='ur', labelcolor='linecolor', handlelength=1.2, numpoints=1, fontsize=12.5)

In [ ]:
d_wps = Json.load_file(ProjPaths.sim_dir_of(sim_info) / 'wp/S-matched.json')
d_wps_obs_bin = Json.load_file(ProjPaths.sim_dir_of(sim_info) / 'wp/S-matched-obsbin.json')

d_wps_rns = [
    [
        Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/S-matched_rn{k}.json'),
        Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/S-matched_rn{k}-obsbin.json')
     ]
    for k in ('0.1', '0.2', '0.3', '0.5', '1.0')
]
d_wps_r1s = [
    [
        Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/S-matched_r1_sigma{sidm_s:.1f}.json'),
        Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/S-matched_r1_sigma{sidm_s:.1f}-obsbin.json')
     ]
    for sidm_s in [
    #0.01, 
    0.1, 
    0.2, 1.0, 5.0, 10.0
    #100.0
    ]    
]
d_wps_r1rns = [
    [
        Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/S-matched_r1_sigma0.1_rn{k}.json'),
        Json.load_file(ProjPaths.sim_dir_of(sim_info) / f'wp/S-matched_r1_sigma0.1_rn{k}-obsbin.json')
     ]
    for k in ('0.1', '0.2', '0.3', '0.5', '1.0')
]

d_wps_tng = Json.load_file(ProjPaths.sim_dir_of(sims.predefined['tng']) / 'wp/ms8.5-9_r2-10.json')['dwarfs/S']
d_wps_lg = Json.load_file(ProjPaths.sim_dir_of(sims.predefined['millennium_2_scaled_wmap7']) / 'wp/ms8.5-9_r2-10.json')['dwarfs/S']

#'lm10p75_sidm_S0p5', 'lm10p75_sidm_S1', 'lm10p75_sidm_S2'
d_wps_sidm_S1_obs_bin = Json.load_file(ProjPaths.sim_dir_of(sim_info) / 'wp/by_S_pred_lm10p75_sidm_S1-obsbin.json')

d_wps_lm10p5_z_half = Json.load_file(ProjPaths.sim_dir_of(sim_info) / 'wp/by_S_pred_lm10p5_z_half-obsbin.json')
d_wps_lm10p5_c_mean200 = Json.load_file(ProjPaths.sim_dir_of(sim_info) / 'wp/by_S_pred_lm10p5_c_mean200-obsbin.json')

In [ ]:
def take_wps_rel(d_in: DataDict, d_rel: DataDict):
    wp_rel = d_in['wp_samples'] / d_rel['wp_samples']
    assert wp_rel.shape[1] == 1
    wp_rel = wp_rel[:, 0]
    y, yl, yh = np.median(wp_rel), np.quantile(wp_rel, 0.16), np.quantile(wp_rel, 0.84)
    yel, yeh = y - yl, yh - y
    x, (xl, xh) = d_in['sub_sample']['median', '1sigma']
    xel, xeh = x - xl, xh - x
    return x, xel, xeh, y, yel, yeh

def take_wps_rels(d_ins: DataDict, d_rel: DataDict = None):
    n_ds = len(d_ins)
    if d_rel is None:
        d_rel = d_ins[str(n_ds-1)]
    wp_rel = []
    for i in range(n_ds):
        wp_rel.append(take_wps_rel(d_ins[str(i)], d_rel))
    wp_rel = np.array(wp_rel)
    return wp_rel
    
    
wps_rel = take_wps_rels(d_wps, d_wps_obs_bin['3'])
wps_rel_2 = take_wps_rels(d_wps_obs_bin)
wps_rel_tng = take_wps_rels(d_wps_tng)
wps_rel_lg = take_wps_rels(d_wps_lg)
wps_rel_rns = [take_wps_rels(a, b['0']) 
               for a, b in d_wps_rns]
wps_rel_r1s = [take_wps_rels(a, b['3']) 
               for a, b in d_wps_r1s]
wps_rel_r1s_obs = [take_wps_rels(b, b['3']) 
               for a, b in d_wps_r1s]
wps_rel_r1rns = [take_wps_rels(a, b['0']) 
               for a, b in d_wps_r1rns]

wps_rel_obs = np.array([
    [6.2,    2.4, 0.28, 0.28,],
    [11.37,  1.44, 0.1, 0.1,],
    [19.33,  1.29, 0.09, 0.09,],
    [42.37, 1.0, 0.0, 0.0,],
])

wps_rel_sidm_S1_obs_bin = take_wps_rels(d_wps_sidm_S1_obs_bin)

wps_rel_lm10p5_z_half = take_wps_rels(d_wps_lm10p5_z_half)
wps_rel_lm10p5_c_mean200 = take_wps_rels(d_wps_lm10p5_c_mean200)

In [ ]:
def interp_keys(k1='S_pred_massive', k2='z_half'):
    S, z_f = sman.s_dwarf[k1, k2]
    Sp = np.logspace(*np.quantile(np.log10(S), [0.001, 0.999]), 64) 
    z_f_p = interp1d(S, z_f)(Sp)
    
    rn = stats.RandomNoise(.001, noise_in_lg=True, rng=0)
    x = np.sort(rn.add_to(z_f)) 
    n = len(x)
    ps = np.arange(n) / (n-1.) * 100.
    zf2p = interp1d(x, ps, kind='linear', fill_value='extrapolate')
    p2zf = interp1d(ps, x, kind='linear', fill_value='extrapolate')
    
    return DataDict({
        'interp': (zf2p, p2zf),
        'x': Sp, 'y': z_f_p
    } )
interp_axis = interp_keys()
interp_axis_r1 = interp_keys('S_pred_massive_r1_sigma0.1', 'r1_sigma0.1')

In [ ]:
fig, ax = plot.subplots((1,1), share=(True, False),
    space=0.03, subsize=(6.5, 5.), 
    margin=[0.02, 0.125, 0.1, 0.135], layout='none')


x, y, el, eh = wps_rel_obs.T
ax.c(cs_dark2[1]).errorbar(x, y, yerr=(el, eh), lw=0.75, marker='o', 
                           mew=2, ms=8, capsize=0, elinewidth=1.5, 
                           label=r'$\rm Observation$')

x, xel, xeh, y, el, eh = wps_rel_r1s[1].T
ax.c('k').errorfill(x, y, yerr=(el, eh), lw=2, 
                    fill_between_kw={'lw': 0., 'alpha': .3})

x, xel, xeh, y, el, eh = wps_rel_r1s_obs[1].T
ax.c('k').errorbar(x, y, yerr=(el, eh), lw=0., marker='s', mew=2, ms=7, 
                   capsize=0, elinewidth=1.5,
                   label=r'${\rm Abundance\ matching}\,(r_{\rm c})$')

x, xel, xeh, y, el, eh = wps_rel_sidm_S1_obs_bin.T
ax.c('b').errorbar(x, y, yerr=(el, eh), lw=2., marker='s', 
                           mew=2, ms=7, capsize=0, elinewidth=1.5,
                           label='  ' +r'$(\Sigma_{\rm bary, 1kpc})$')

x, xel, xeh, y, el, eh = wps_rel_lm10p5_z_half.T
ax.c('r').errorbar(x, y, yerr=(el, eh), lw=2., marker='s', 
                           mew=2, ms=7, capsize=0, elinewidth=1.5,
                           label='  ' +r'$(z_{\rm half, 10.5})$')

x, xel, xeh, y, el, eh = wps_rel_lm10p5_c_mean200.T
ax.c('pink').errorbar(x, y, yerr=(el, eh), lw=2., marker='s', 
                           mew=2, ms=7, capsize=0, elinewidth=1.5,
                           label='  ' +r'$(c_{10.5})$')

x, xel, xeh, y, el, eh = wps_rel_tng.T
ax.c(cs_dark2[0]).errorbar(x, y, yerr=(el, eh), lw=2., marker='s', 
                           mew=2, ms=7, capsize=0, elinewidth=1.5,
                           label=r'$\rm TNG$')

x, xel, xeh, y, el, eh = wps_rel_lg.T
ax.c(cs_dark2[2]).errorbar(x, y, yerr=(el, eh), lw=2., marker='s', 
                           mew=2, ms=7, capsize=0, elinewidth=1.5,
                           label=r'$\rm L$-$\rm Galaxies$')

ax.c('k')
lws = np.linspace(0.5, 3.0, 7)
for i, sidm_s in enumerate([
    0.1, 
    0.2, 1.0, 5.0, 10.0
    ]):
    if i == 1:
        continue
    x, xel, xeh, y, el, eh = wps_rel_r1s[i].T
    ax.plot(x, y, lw=.75, zorder=-1, ls=(0,(2,2)))
    #ax.errorbar(x, y, yerr=(el, eh), capsize=0, 
    #            elinewidth=.5, marker='o', ms=2.5, lw=lws[i], zorder=-1)

ax.c('grey')
for i, k in enumerate(('0.1', '0.2', '0.3', '0.5', '1.0')):
    x, xel, xeh, y, el, eh = wps_rel_r1rns[i].T
    #ax.plot(x, y, lw=.75, alpha=.75, zorder=-1)
    ax.errorbar(x, y, yerr=(el, eh), lw=.75, capsize=0, 
                elinewidth=.5, marker='o', ms=2.5)

ax.scale(x='log').lim([10**0.35, 1.0e2], [0.75, 2.75])
ax.label(r'\Sigma_*\,[{\rm M}_\odot\,{\rm pc}^{-2}]', r'{\rm Relative\ bias}')
ax.leg(loc='ur', labelcolor='linecolor', handlelength=1.2, numpoints=1, fontsize=12.5)

## 2PCF vs spin and z_f

In [ ]:
pdata = ProjPaths.sim_dir_of(sim_info) / 'z0_all_postprocessed.hdf5'
man_kw = {
    'add_rsd': True, 
    'dmo': True,
    'm_h_range_in_sol': (10**10.5, 10**11.),
    'm_h_ref_lb': 10**10.5,
}
sman = SubSampleManager(sim_info, pdata, **man_kw)
sman.s_dwarf

In [ ]:
wp_kw = {
    'pimax': 10., 'n_repeat': 10,
    'bootstrap_kw': {'keep_samples': True},
}
sim_cross = SimCross(sman)

In [ ]:
dps = [0.02467638, 0.23220065, 0.25970874, 0.48341424]
ps = np.cumsum(dps[::-1])
ps[-1] = 1. 
ps = np.concatenate([[0.], ps])

wp_tot = sim_cross.wp_of('s_dwarf', **wp_kw)
for k_in in 'z_half', 'spin_h':
    o_path = sim_dir.base_dir / f'wp/dwarf_halos_lm10p5-11_by-{k_in}.json'
    wps = sim_cross.wps_of('s_dwarf', k_in, ps=ps, **wp_kw)
    wps['total'] = wp_tot
    Json.dump_file(wps, o_path, indent=2)

In [ ]:
lm_h_edges = [11., 12., 13., 16.]
o_path = sim_dir.base_dir / 'wp/halos_by-M.json'

u_m = 1.0e10 / sman.sim_info.cosmology.hubble
m_h_edges = 10.0**np.array(lm_h_edges) / u_m
wps = sim_cross.wps_of('s_halo', 'm_mean200', ps=None, qs=m_h_edges, **wp_kw)
Json.dump_file(wps, o_path, indent=2)

In [ ]:
s1 = sman.s_dwarf.subset_by_percentile('z_half', 0.00, 0.02)


In [ ]:
wp = clustering.n2wp_periodic(pc['n_pairs'], pc['n1'], pc['n2'], pc['rs'],
                              pc['l_box'])

In [ ]:
pc, wp

In [ ]:
ofile = 'wp-tng300-1-dark.json'
ps = [0.00, 0.02, 0.33, 0.67, 0.98, 1.00]
lm_h_edges = [11., 12., 13., 16.]

pdata = ProjPaths.sim_dir_of(sim_info) / 'z0_all_postprocessed.hdf5'
man_kw = {'add_rsd': True, 'dmo': True}
wp_kw = {'pimax': 40., 'n_repeat': 10, 
         'bootstrap_kw': {'keep_samples': True}
         }

sman = SubSampleManager(sim_info, pdata, **man_kw)
sim_cross = SimCross(sman)

wp_tot = sim_cross.wp_of('s_dwarf', **wp_kw)    
wp_z_f = sim_cross.wps_of('s_dwarf', 'z_half', ps = ps, **wp_kw)
wp_spin = sim_cross.wps_of('s_dwarf', 'spin', ps = ps, **wp_kw)

u_m = 1.0e10 / sman.sim_info.cosmology.hubble
m_h_edges = 10.0**np.array(lm_h_edges) / u_m
wp_h = sim_cross.wps_of('s_halo', 'm_mean200', ps=None, qs=m_h_edges, **wp_kw)

data = DataDict({
    'dwarfs': {
        'z_f': wp_z_f, 'spin': wp_spin, 'tot': wp_tot,    
    },
    'halos': wp_h,
})
    
path = ProjPaths.sim_dir_of(sim_info) / ofile
with open(path, 'w') as f:
    json.dump(data, f, default=json_default, indent=2)

## Old routines

In [ ]:
sim_info = sims.predefined['tng_dark_300_1']
s_all = sample.SimSample.load(sim_info, 'z0_all.hdf5').requires_spin().requires_assembly().require_c()
s_c = s_all.subset_by_value('is_c', eq=True)

In [ ]:
hubble = sim_info.cosmology.hubble
m_scale = 1.0e10 / hubble

m_h_lb = 10**10.5 / m_scale
m_h_ub = 10**11.0 / m_scale

z_lc_lb = 15.

m_h_key = 'm_mean200'

m_h_lb, m_h_ub, z_lc_lb

In [ ]:
m_h_edges = 10.0**np.array([11.5, 12.0, 12.5, 13.0, 16.0]) / m_scale

In [ ]:
s_ref = s_all.subset_by_value('m_peak', lo=m_h_lb)
cls_ref = clustering.Clustering(s_ref)
subs_ref = SubSample(s_ref, add_rsd=True)

s_dwarf = s_c.subset_by_value(m_h_key, lo=m_h_lb, hi=m_h_ub).subset_by_value('last_sat_z', lo=15.)
s_ref.n_objs, s_dwarf.n_objs

s_dwarf2 = s_c.subset_by_value(m_h_key, lo=m_h_lb, hi=m_h_ub)

### Output for Ziwen For SIDM

In [ ]:
keys = 'm_crit200', 'm_mean200', 'm_tophat', \
    'v_max', 'v_peak', 'v_peak_snap', 'c',\
    'last_sat_z', 'z_half', 'spin', 'spin_form'
    
assert s_dwarf['is_c'].all()

out_d = {k: s_dwarf[k] for k in keys}
out_meta = {
    'source': 'TNG300-1-Dark',
    'selection': 'central & m_mean200 in 10**{10.5-11.0} & last_sat_z >= 15',
    'quantities': {
        'm_crit200': 'virial mass, [10^10 M_sun/h]',
        'm_mean200': 'mean mass, [10^10 M_sun/h]',
        'm_tophat': 'tophat mass, [10^10 M_sun/h]',
        'v_max': 'max circular velocity, physical [km/s]',
        'v_peak': 'peak of v_max along the main branch, physical [km/s]',
        'v_peak_snap': 'snapshot when v_peak is reached',
        'c': 'concentration',
        'last_sat_z': 'last redshift when the halo is a satellite',
        'z_half': 'redshift of half mass',
        'spin': 'spin parameter',
        'spin_form': 'spin parameter at z_half',
    }
}
out = {
    'meta': out_meta,
    'data': out_d,
}

p = ProjPaths.sim_dir_of(sim_info) / 'sample-dwarfs-c-noej.json'
with open(p, 'w') as f:
    json.dump(out, f, indent=2, default=json_default)

In [ ]:
with open(p, 'r') as f:
    in_d = json.load(f)

In [ ]:
# Input from Ziwen's results

def join_txt(s: sample.SubhaloSet, file, col_names):
    d = np.loadtxt(file, dtype=float)
    assert d.shape == (len(s[s.keys[0]]), len(col_names))
    for col_name, val in zip(col_names, d.T):
        if col_name in s.objs:
            print(f'Overwriting {col_name}')
        s.objs[col_name] = val

for suf in '_10.0', '_40.0', '_0.1',: # '_0.2', '_0.5', '_1.0', '_2.0':
    p = ProjPaths.sim_dir_of(sim_info) / f'SIDM/DMO_info{suf}'
    join_txt(s_dwarf, p, [f'r1_DMO{suf}', 'halo_age', 
        'halo_concentration', f'rho_0{suf}'])

In [ ]:
np.quantile(np.log10(s_dwarf['r1_DMO_10.0']), 0.03)
np.quantile(np.log10(s_dwarf['rho_0_10.0']), 0.99)

In [ ]:
fig, ax = plot.subplots(1, figsize=4.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

ax.hist(np.log10(s_dwarf['rho_0_0.1']), bins=20)

In [ ]:
r1_s = s_dwarf[
    #'r1_DMO_0.1', 'rho_0_0.2', 'rho_0_0.5', 'rho_0_2.0'
    'r1_DMO_0.1', 'rho_0_10.0', 'rho_0_40.0', 'r1_DMO_10.0'
    #'r1_DMO_0.5', 'r1_DMO_1.0', 'r1_DMO_2.0'
    #, 'halo_concentration', 'c'
]

In [ ]:
fig, axs = plot.subplots((1,3), share=False, space=0.2, subsize=(4.5, 4.5), 
                         margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

labs = r'\rho_{0,0.2}', r'\rho_{0,0.5}', r'\rho_{0,2.0}'
n = len(r1_s[0]) // 1000
for i_ax, i_col in enumerate((1,2,3)):
    ax = axs[i_ax]
    ax.scatter(r1_s[0][::n], r1_s[i_col][::n])
    ax.label(y=labs[i_ax])
axs.label(r'r_{1,0.5}')
axs.scale(y='log')

### 2PCF

In [ ]:
cls_kw = {'pimax': 40., 'n_repeat': 5}

In [ ]:
# halos
subs_halos = SubSample.list_from_bins(s_c, m_h_key, edges=m_h_edges)

In [ ]:
out_h = {str(i): subs_ref.proj_cross_with(subs, **cls_kw)  
         for i, subs in enumerate(subs_halos) }

In [ ]:
out_tot = subs_ref.proj_cross_with(SubSample(s_dwarf), **cls_kw)


In [ ]:
out_tot2 = subs_ref.proj_cross_with(SubSample(s_dwarf2), **cls_kw)

In [ ]:
out_r1 = binned_proj_cross(subs_ref, s_dwarf, 'r1_DMO')

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'wp-result-pi40-resamp50-r1-bins.hdf5'
h5.File.dump_to(path, {
    'dwarfs': {
        'r1': out_r1
    },
}, f_flag='w')

In [ ]:
out_r1_list = {
  f'r1{suf}': binned_proj_cross(subs_ref, s_dwarf, f'r1_DMO{suf}')  
  for suf in ['_0.2', '_1.0', '_2.0']
} 

path = ProjPaths.sim_dir_of(sim_info) / 'wp-result-pi40-resamp50-r1-list.hdf5'
h5.File.dump_to(path, {
    'dwarfs': out_r1_list,
}, f_flag='w')

In [ ]:
out_r1_list_2 = {
  f'r1{suf}': binned_proj_cross(
    subs_ref, s_dwarf, f'r1_DMO{suf}', ps=[0.00, 0.03, 0.33, 0.67, 0.98, 1.],
  )  for suf in ['_10.0', '_40.0']
} 
path = ProjPaths.sim_dir_of(sim_info) / 'wp/r1_binned_2.hdf5'
h5.File.dump_to(path, {
    'dwarfs': out_r1_list_2,
}, f_flag='w')

Binned by $\rho_0$

In [ ]:
out_rho_0_list = {
    f'rho_0{suf}': binned_proj_cross(subs_ref, s_dwarf, f'rho_0{suf}')  
    for suf in ['_0.2', '_0.5', '_1.0', '_2.0']
}
path = ProjPaths.sim_dir_of(sim_info) / 'wp/rho_0_binned.hdf5'
h5.File.dump_to(path, {
    'dwarfs': out_rho_0_list,
}, f_flag='w')

In [ ]:
out_rho_0_list_2 = {
    f'rho_0{suf}': binned_proj_cross(subs_ref, s_dwarf, f'rho_0{suf}')  
    for suf in ['_10.0', '_40.0']
}
path = ProjPaths.sim_dir_of(sim_info) / 'wp/rho_0_binned_2.hdf5'
h5.File.dump_to(path, {
    'dwarfs': out_rho_0_list_2,
}, f_flag='w')

In [ ]:
subs_list = SubSample.list_from_percents(s_dwarf2, 
    'z_half', [0.00, 0.02, 
               0.33, 0.67, 0.98, 1.00
], add_rsd=True)
out_z2 = {str(i): subs_ref.proj_cross_with(subs, **cls_kw)  
         for i, subs in enumerate(subs_list) }

In [ ]:
subs_list = SubSample.list_from_percents(s_dwarf, 
    'spin', [0.00, 0.02, 
            0.33, 0.67, 0.98, 1.00
], add_rsd=True)
out_spin = {str(i): subs_ref.proj_cross_with(subs, **cls_kw)  
         for i, subs in enumerate(subs_list) }

In [ ]:
subs_list = SubSample.list_from_percents(s_dwarf2, 
    'spin', [0.00, 0.02, 
            0.33, 0.67, 0.98, 1.00
], add_rsd=True)
out_spin2 = {str(i): subs_ref.proj_cross_with(subs, **cls_kw)  
         for i, subs in enumerate(subs_list) }

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'wp-result-pi40-resamp50.hdf5'
h5.File.dump_to(path, {
    'dwarfs': {
        'z': out_z, 'spin': out_spin, 'tot': out_tot
    },
    'halos': out_h
}, f_flag='w')

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'wp-result-pi40-resamp50-addej.hdf5'
h5.File.dump_to(path, {
    'dwarfs': {
        'z': out_z2, 'spin': out_spin2, 'tot': out_tot2
    },
    'halos': out_h
}, f_flag='w')

In [ ]:
# rerun it
res_z = {
    'dwarfs': get_cls_by_quantiles(s_dwarf, cls_ref, key='z_half'),
    'halos': get_cls_binned(s_c, cls_ref, m_h_key, ranges=m_h_bins,n_rep=5)
}

In [ ]:
res_z2 = {
    'dwarfs': get_cls_by_quantiles(s_dwarf2, cls_ref, key='z_half'),
    'halos': res_z['halos']
}

In [ ]:
res_spin2 = {
    'dwarfs': get_cls_by_quantiles(s_dwarf2, cls_ref, key='spin'),
    'halos': res_z['halos']
}

In [ ]:
res_spin = {
    'dwarfs': get_cls_by_quantiles(s_dwarf, cls_ref, key='spin'),
    'halos': res_z['halos']
}

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'wp-result.hdf5'
h5.File.dump_to(path, {'z': res_z, 'z2': res_z2, 
    'spin': res_spin, 'spin2': res_spin2}, f_flag='w')

In [ ]:
sim_info = sims.predefined['tng_dark_300_1']
path = ProjPaths.sim_dir_of(sim_info) / 'wp-result.hdf5'
res = h5.File.load_from(path)
res_z, res_z2, res_spin, res_spin2 = res['z', 'z2', 'spin', 'spin2']

In [ ]:
qs = np.quantile(s_dwarf['spin'], [0.02, 0.33, 0.67, 0.98])
for q in qs:
    print(f'{q:.2f}')

### Distance to neighbors

In [ ]:
subs_h12 = SubSample(s_c, m_h_key, 1.0e12 / m_scale, add_rsd=True)
subs_h13 = SubSample(s_c, m_h_key, 1.0e13 / m_scale, add_rsd=True)

subs_tot = SubSample(s_dwarf)
subs_tot2 = SubSample(s_dwarf2)

subs_list = SubSample.list_from_percents(s_dwarf, 
    'z_half', [0.00, 0.02, 
               0.33, 0.67, 0.98, 1.00
], add_rsd=True)

In [ ]:
def get_min_ds(subs_list: list[SubSample], subs_ref: SubSample):
    out = DataDict()
    for i, subs in enumerate(subs_list):
        min_d = subs_ref.min_dist_3d_with(subs)
        assert len(subs.samp['x']) == min_d.size
        out[str(i)] = {'min_d': min_d}
        
    return out

def get_min_ds_2d(subs_list: list[SubSample], subs_ref: SubSample, **kw):
    out = DataDict()
    for i, subs in enumerate(subs_list):
        min_d, r_h = subs_ref.min_dist_2d_with(subs, **kw)
        z_half = subs.samp['z_half']
        assert len(z_half) == min_d.size == r_h.size
        out[str(i)] = {'min_d': min_d, 'z_half': z_half, 'r_h': r_h}
        
    return out

def dump_mind(res, path: Path):
    o = np.array([res['min_d'], res['z_half'], res['r_h']]).T
    sel = o[:, 0] < 1.0e5
    o = o[sel]
    np.savetxt(path, o, fmt=r'%.6g')

In [ ]:
res_mind_h12 = get_min_ds_2d([subs_tot], subs_h12)['0']
res_mind_h13 = get_min_ds_2d([subs_tot], subs_h13)['0']

path = ProjPaths.sim_dir_of(sim_info) / 'min-dist-h12.txt'
dump_mind(res_mind_h12, path)

path = ProjPaths.sim_dir_of(sim_info) / 'min-dist-h13.txt'
dump_mind(res_mind_h13, path)

In [ ]:
res_mind_h13 = get_min_ds_2d([subs_tot], subs_h13, max_dx_proj=50)['0']

path = ProjPaths.sim_dir_of(sim_info) / 'min-dist-h13-max50.txt'
dump_mind(res_mind_h13, path)

In [ ]:
res_mind_h12 = get_min_ds_2d([subs_tot2], subs_h12)['0']
res_mind_h13 = get_min_ds_2d([subs_tot2], subs_h13)['0']

path = ProjPaths.sim_dir_of(sim_info) / 'min-dist-h12-addej.txt'
dump_mind(res_mind_h12, path)

path = ProjPaths.sim_dir_of(sim_info) / 'min-dist-h13-addej.txt'
dump_mind(res_mind_h13, path)

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'min-dist-h13.txt'
dset1 = np.loadtxt(path)

path = ProjPaths.sim_dir_of(sim_info) / 'min-dist-h13-max50.txt'
dset2 = np.loadtxt(path)

In [ ]:
fig, ax = plot.subplots(1, figsize=4.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

cs = 'r', 'b'
for i, dset in enumerate([dset1, dset2]):
    d, z_f, r_h = dset.T
    d_scld = d / r_h
    h, e = np.histogram(d_scld, bins=50, range=[0., 30.], density=True)
    ax.stairs(h, e, ec=cs[i])

In [ ]:
x = res_mind_h12['min_d']
sel = x < 1.0e5
x = x[sel] / res_mind_h12['r_h'][sel]

fig, ax = plot.subplots(1, figsize=4.5, margin=[0.1, 0.1, 0.1, 0.1], layout='none')

ax.hist(np.log10(x), range=[-0.75, 1.])

In [ ]:
res_mind = {}

In [ ]:
res_mind |= {
    'h12': get_min_ds(subs_list, subs_h12),
    'h13': get_min_ds(subs_list, subs_h13),
} 

In [ ]:
res_mind |= {
    'h12_2d': get_min_ds_2d(subs_list, subs_h12),
    'h13_2d': get_min_ds_2d(subs_list, subs_h13),
}

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'min-dist-result.hdf5'
h5.File.dump_to(path, res_mind, f_flag='w')

In [ ]:
sim_info = sims.predefined['tng_dark_300_1']

path = ProjPaths.sim_dir_of(sim_info) / 'min-dist-result.hdf5'
res_mind = h5.File.load_from(path)

In [ ]:
h5.File.ls_from(path)

In [ ]:
fig, axs = plot.subplots((2,2), share=('col', True), space=0., 
    subsize=(5.5, 4.05), margin=[0.075, 0.02, 0.075, 0.065], layout='none')
axs_f = axs.flat

cs_g = 'b', 'g', 'orange', 'r', 'purple'
g_labs = [
    r'$0^{\rm th}\,$-$\,2^{\rm nd}$',
    r'$2^{\rm nd}\,$-$\,33^{\rm rd}$',
    r'$33^{\rm rd}\,$-$\,67^{\rm th}$',
    r'$67^{\rm th}\,$-$\,98^{\rm th}$',
    r'$98^{\rm th}\,$-$\,100^{\rm th}$',
]

for ih, h_lab in enumerate(['h12', 'h13']):
    res = res_mind[h_lab]
    ax = axs[ih, 0]
    for i in range(5):
        d = res[str(i)]['min_d']
        sel = d < 30.
        # print(sel.sum()/sel.size)
        d = d[sel]
        x = Num.safe_lg(d)
        x_med = np.median(x)
        x_mean = Num.safe_lg(np.mean(d)) 
        
        c = cs_g[i]
        ax.hist(x, range=[-.75, 1.5], bins=32, density=True, fc='none', 
                ec=c, lw=3.)
        ax.plot([x_med]*2, [1.8, 2.15], lw=2, c=c, label=g_labs[i])
        # ax.plot([x_mean]*2, [2.0, 2.15], lw=2, ls=(0,(2,1)), c=c)
        
    res = res_mind[h_lab + '_2d']
    ax = axs[ih, 1]
    for i in range(5):
        d = res[str(i)]['min_d']
        sel = d < 30.
        #print(sel.sum()/sel.size)
        d = d[sel]
        x = Num.safe_lg(d)
        x_med = np.median(x)
        x_mean = Num.safe_lg(np.mean(d)) 
        print(10**x_med)
        
        c = cs_g[i]
        ax.hist(x, range=[-.75, 1.5], bins=32, density=True, fc='none', 
                ec=c, lw=3.)
        ax.plot([x_med]*2, [1.8, 2.15], lw=2, c=c, label=g_labs[i])
        #ax.plot([x_mean]*2, [2.0, 2.15], lw=2, ls=(0,(2,1)), c=c)
    
labs = r'$R_{\rm sep, 3D}\,[h^{-1}{\rm Mpc}]$', r'$R_{\rm sep}\,[h^{-1}{\rm Mpc}]$'
for i, ax in enumerate(axs[0]):
    ax: plot.Axes
    fn = lambda lg_x: 10.0**lg_x, lambda x: np.log10(x+1.0e-10)
    sax = ax.secondary_xaxis('top', functions=fn)
    sax.fmt_frame(label={'x': labs[i]},
             ticks={'x': [0.1, 0.5, 1, 2, 3, 5, 7, 10, 15.]})
    ax.tick_params(x={
        'which': 'both', 'top': False,
    })
    
ax.tick_params(x={
   'which': 'both', 'top': False,
})
axs[0,0].leg(loc='ur', title=r'$z_{\rm f}\ {\rm percentiles}$',
    title_fontsize=13., labelspacing=0., numpoints=1, fontsize=13, 
    ncol=1, handlelength=1.)
axs[:, 0].label(y=r'{\rm PDF}').lim([-0.75, 1.4], [0.0, 2.55]).label(r'\log\,R_{\rm sep, 3D}\,[h^{-1}{\rm Mpc}]')
axs[:, 1].lim([-0.75, 1.4]).label(r'\log\,R_{\rm sep}\,[h^{-1}{\rm Mpc}]')

axs[0,0].text(r'{\rm To\ halos},\,M_{\rm h}\geqslant 10^{12}{\rm M}_\odot', (.08, .92))
axs[1,0].text(r'{\rm To\ halos},\,M_{\rm h}\geqslant 10^{13}{\rm M}_\odot', (.08, .92))

axs[0,0].text(r'\rm Median', (.275, .775), fontsize=13)
# axs[0,0].text(r'\rm Mean', (.275, .81), fontsize=13)
axs.label_outer()

# ProjPaths.save_fig('fig_d2halo_dmo.pdf')

## Density and Tidal Field

In [ ]:
from pyhipp.field import cubic_box
%autoreload
gc.collect()

sim_info = sims.predefined['tng_dark_300_1']

In [ ]:
# generate smoothed field

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'density_field.99.hdf5'
with h5.File(path) as f:
    rho_x = cubic_box.DensityField.load(f)

In [ ]:
for r_sm in 1,2,4, 5, 8, 10:
    sm = cubic_box.FourierSpaceSmoothing(n_workers=8, r_sm=float(r_sm))
    res = sm.run(rho_x)
    path = ProjPaths.sim_dir_of(sim_info) / f'density_field.99.sm{r_sm}.hdf5'
    with h5.File(path, 'w') as f:
        res.dump(f)

In [ ]:


tf = cubic_box.TidalField(n_workers=8, r_sm=1.0)
res = tf.run(rho_x)

path = ProjPaths.sim_dir_of(sim_info) / 'tidal_field.99.sm1.hdf5'
with h5.File(path, 'w') as f:
    res.dump(f)

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'tidal_field.99.sm2.hdf5'
delta_sm_k = h5.File.load_from(path, 'delta_sm_k')
fft = cubic_box.fft.NdRealFFT(n_workers=8, norm='ortho')
delta_sm_x = fft.backward(delta_sm_k)
h5.File.dump_to(path, delta_sm_x, key='delta_sm_x')

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'tidal_field.99.sm2.hdf5'
with h5.File(path) as f:
    clsf = cubic_box.TidalClassifier.new_from_file(f, lam_th=.2)
clsf

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'tidal_field.99.sm1.hdf5'
with h5.File(path) as f:
    f_interp = cubic_box.cosmic_web.FieldInterpolator.new_density_field_from_file(
        f)
f_interp

In [ ]:
fig, ax = plot.subplots(1, figsize=4.5, extent=[1., 1., 0., 0.], layout='none')

lams_f = lams.reshape(-1, 3)
cs = ColorSets.tab10[:3]
for i in range(3):
    x = lams_f[:, i]
    ax.c(cs[i]).hist(x, range=[-1., 1.], bins=20)

In [ ]:
fig, ax = plot.subplots(1, figsize=4.5, extent=[1., 1., 0., 0.], layout='none')

z0 = 40.0
z1 = z0 + 10.0

l_grid = sim_info.full_box_size / 256
zi0, zi1 = int(z0 / l_grid), int(z1 / l_grid)
im = clsf.is_filament[:, :, zi0:zi1].sum(-1) >= 1

ax._raw.imshow(im.T, origin='lower',)

In [ ]:
fig, ax = plot.subplots(1, figsize=4.5, extent=[1., 1., 0., 0.], layout='none')

z0 = 40.0
z1 = z0 + 10.0

l_grid = sim_info.full_box_size / 256
zi0, zi1 = int(z0 / l_grid), int(z1 / l_grid)
im = Num.safe_lg(f_interp._data[:, :, zi0:zi1].max(-1)) 

ax._raw.imshow(im.T, origin='lower',)

## Web - web correlation

In [ ]:
# load tidal data and cosmic web types

path = ProjPaths.sim_dir_of(sim_info) / 'tidal_field.99.sm1.hdf5'
with h5.File(path) as f:
    clsf = cubic_box.TidalClassifier.new_from_file(f, lam_th=0.2)
clsf

s_web_types = {
    t: sample.SubhaloSet.from_cosmic_web(clsf, sim_info, web_type=t) 
    for t in ['k', 'f', 's', 'v']
}
cls_web_types = {
    t: clustering.Clustering(s) for t, s in s_web_types.items()
}

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'tidal_field.99.sm1.hdf5'
with h5.File(path) as f:
    f_interp = cubic_box.cosmic_web.FieldInterpolator.new_density_field_from_file(
        f)
f_interp

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'density_field.99.sm5.hdf5'
with h5.File(path) as f:
    f_interp_sm5 = cubic_box.cosmic_web.FieldInterpolator.new_density_field_from_file(
        f, key='delta_sm_x')
f_interp_sm5

In [ ]:
def corr_with_web(samp: sample.SubhaloSet, rs=np.logspace(-.7, 1.2, 21), 
                  n_max_rand=2000, n_repeat=20, n_threads=8, 
                  use_proj=False, attach_sample = True):
    out = DataDict()
    for web_t in ['k', 'f', 's', 'v']:
        cls = cls_web_types[web_t]
        
        fn = cls.proj_cross if use_proj else cls.cross
        out[web_t] = fn(samp, rs=rs, n_max_rand=n_max_rand,
                        n_repeat=n_repeat, n_threads=n_threads)
    out['samp'] = samp
    return out

### Make dwarf sample

In [ ]:
def add_f_web_type(s: sample.SubhaloSet, cls_web_types: dict[str, clustering.Clustering],
                   r=10.0, key='frac_'):
    cnts = {}
    for t, cls in cls_web_types.items():
        cnt = cls.counts_in_r(s['x'], r=r)
        cnts[t] = cnt
    cnt_tot = np.array(list(cnts.values())).sum(0)
    for t, cnt in cnts.items():
        s.objs |= {
            f'{key}{t}': cnt / cnt_tot
        }

In [ ]:
s_all = sample.SimSample.load(sim_info, 'z0_cent.hdf5')\
    .requires_spin().requires_assembly().requires_cosmic_type(clsf)
s_c = s_all

In [ ]:
s_ref = s_all.subset_by_value('m_crit200', lo=10**0.6)
cls_ref = clustering.Clustering(s_ref)

In [ ]:
add_f_web_type(s_dwarf, cls_web_types)
s_dwarf = s_c.subset_by_value('m_crit200', lo=10**0.6, hi=10**0.8)

for k_web in ['k', 'f', 's', 'v']:
    s_dwarf.requires_dist_to(f'd2{k_web}', s_web_types[k_web]['x'])
    s_dwarf.requires_dist_to(f'd2{k_web}_2th', s_web_types[k_web]['x'], k_th=2)
path = ProjPaths.sim_dir_of(sim_info) / 'dwarf.c.frac_web.hdf5'
s_dwarf.to_h5_file(path, f_flag='w')

### Clusterings

In [ ]:
s_all = sample.SimSample.load(sim_info, 'z0_all.hdf5')
s_c = s_all.subset_by_value('is_c', eq=True)

s_ref = s_all.subset_by_value('v_peak_m', lo=10**0.6)
cls_ref = clustering.Clustering(s_ref)

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'dwarf.c.frac_web.hdf5'
s_dwarf = sample.SubhaloSet.from_h5_file(path)
s_dwarf = sample.SimSample.from_data(s_dwarf.objs, s_dwarf.header, s_dwarf.sim_info)

s_dwarf.objs['rho_5'] = f_interp_sm5.value_at(s_dwarf['x'])

s_dwarf

In [ ]:
s_typed = {
    t: s_dwarf.subset_by_value('cosmic_type', eq=clsf.web_types[t]) for t in s_web_types.keys()
} 
s_typed['all'] = s_dwarf
s_noej = s_dwarf.subset_by_value('last_sat_z', lo=15.)
s_ej = s_dwarf.subset_by_value('last_sat_z', hi=1.5)

In [ ]:
outs: DataDict[str, np.ndarray|sample.SubhaloSet] = DataDict()

In [ ]:
outs_with_ej: DataDict[str, np.ndarray|sample.SubhaloSet] = DataDict()
outs_with_ej['all'] = {'samp': s_dwarf}
outs_with_ej['dense'] = {'samp': outs_with_ej['all/samp'].subset_by_value('rho_5', 0.75)}
outs_with_ej['near_k'] = {'samp': outs_with_ej['dense/samp'].subset_by_value('d2k', 1.5, 5.)}
outs_with_ej['less_s'] = {'samp': outs_with_ej['near_k/samp'].subset_by_value('frac_s', hi=.53)}
outs_with_ej['low_zf'] = {'samp': outs_with_ej['less_s/samp'].subset_by_value('z_half', hi=1.)}

In [ ]:
s = s_noej
outs['noej'] = corr_with_web(s, use_proj=True)

In [ ]:
s = outs['noej/samp'].subset_by_value('rho_5', .75)
outs['dense'] = corr_with_web(s, use_proj=True)

s = outs['noej/samp'].subset_by_value('is_f', eq=True)
outs['on_f'] = corr_with_web(s, use_proj=True)

In [ ]:
s = outs['dense/samp'].subset_by_value('d2k', 1.5, 5.)
outs['near_k'] = corr_with_web(s, use_proj=True)

s = outs['dense/samp'].subset_by_value('d2k', 3.)
outs['lb_k'] = corr_with_web(s, use_proj=True)

In [ ]:
s = outs['near_k/samp'].subset_by_value('frac_s', hi=.53)
outs['less_s'] = corr_with_web(s, use_proj=True)

s = outs['near_k/samp'].subset_by_value('frac_s', hi=.5)
outs['little_s'] = corr_with_web(s, use_proj=True)

In [ ]:
s = outs['less_s/samp'].subset_by_value('z_half', hi=1.)
outs['low_zf'] = corr_with_web(s, use_proj=True)

s = outs['less_s/samp'].subset_by_value('z_half', lo=2.)
outs['high_zf'] = corr_with_web(s, use_proj=True)

In [ ]:
outs['low_zf/samp']

In [ ]:
fig, axs = plot.subplots((2,2), share=False, space=0.2, subsize=(5., 4.5), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

y_lims = [-10., 100.], [2., 40.], [-1., 5.], [-40., 0.]
cs = 'k', 'r', 'r', 'g', 'g', cs_set1[1], cs_set1[1], 'orange', 'orange'
lws = 3, 3, 1, 3, 1, 3, 1, 3, 1
lss = '-', '-', '-.', '-', '-.', '-', '-.', '-', '-.'
y_labs = r'\rm Galaxy$-$\rm Knot', r'\rm Galaxy$-$\rm Filament', r'\rm Galaxy$-$\rm Sheet', r'\rm Galaxy$-$\rm Void'
for i_web, web_t in enumerate(['k', 'f', 's', 'v']):
    ax = axs_f[i_web]
    for i_samp, samp in enumerate(['noej', 'dense', 'on_f',
                                   'near_k', 'lb_k',
                                   'less_s', 'little_s', 
                                   'low_zf', 'high_zf',
        ]):
        ax.c(cs[i_samp])
        
        out = outs[samp][web_t]
        x, y, e = out['lg_rs_c', 'wp', 'wp_sd']
        kw = dict(lw=lws[i_samp], ls=lss[i_samp], label=samp)
        if i_samp == 0 or i_samp == len(lws)-3:
            ax.errorbar(x, y, e, **kw)
        else:
            ax.errorbar(x, y, **kw)

    ax.scale(y='symlog').lim(y=y_lims[i_web])
    ax.text(y_labs[i_web], (.05, .12))
    
axs_f[-1].leg(loc='ul', fontsize=13, handlelength=1.75, ncol=2)
axs.lim([-0.25, 1.15]).label(r'\log\,r_{\rm p}[h^{-1}{\rm Mpc}]')
axs[:, 0].label(y=r'w_{\rm p,\, cross}(r_{\rm p})\,[h^{-1}{\rm Mpc}]')

In [ ]:
fig, axs = plot.subplots((2,3), share=(False,True), space=(0., .2), subsize=(4.5, 4.5), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

x_lims = [0.9, 1.7], [0., 4.5], [-3., 0.], [-3., 0.], [1., 1.5], [-1., 1.] #[1.3, 2.2]
logged = False, False, True, True, False, True
cs = 'k', 'r', 'g', cs_set1[1], 'orange', 'grey', 'grey'

lab_names = r'V_{\rm max}/V_{\rm vir}', r'z_{\rm h,1/2}', r'\log\,\lambda_{\rm h}', \
    r'\log\,\lambda_{\rm h,1/2}', r'V_{\rm peak}/V_{\rm max}', r'\rho_5' #r'V_{\rm max}'

for i_quant, quant in enumerate(['v_max2h', 'z_half', 'spin', 'spin_form', 'v_peak2max', 'rho_5']):
    ax = axs_f[i_quant]
    for i_samp, samp in enumerate(['noej', 'dense', 'near_k', 'less_s', 'low_zf']):
        x = outs[samp]['samp'][quant]
        if logged[i_quant]:
            x = Num.safe_lg(x)
        ax.c(cs[i_samp])
        ax.hist(x, range=x_lims[i_quant], bins=32, log=True, density=True, lw=2)
        med, std = np.median(x), np.std(x)
        ax.text(r'($%s$)\ %.2f\pm%.2f'%(samp, med, std), (.95, .95-i_samp*.06), color=cs[i_samp], ha='right', fontsize=12)
        
    ax.label(lab_names[i_quant])
    
axs_f.lim(y=[0.01, 100])
axs[:, 0].label(y=r'\rm PDF')

In [ ]:
fig, axs = plot.subplots((2,3), share=(False,True), space=(0., .2), subsize=(4.5, 4.5), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

x_lims = [0.9, 1.7], [0., 4.5], [-3., 0.], [-3., 0.], [1., 1.5], [-1., 1.]
logged = False, False, True, True, False, True
cs = 'k', 'r', 'g', cs_set1[1], 'orange', 'grey', 'grey'

lab_names = r'V_{\rm max}/V_{\rm vir}', r'z_{\rm h,1/2}', r'\log\,\lambda_{\rm h}', \
    r'\log\,\lambda_{\rm h,1/2}', r'V_{\rm peak}/V_{\rm max}', r'V_{\rm max}'

for i_quant, quant in enumerate(['v_max2h', 'z_half', 'spin', 'spin_form', 'v_peak2max', 'rho_5']):
    ax = axs_f[i_quant]
    for i_samp, samp in enumerate(['all', 'dense', 'near_k', 'less_s', 'low_zf']):
        x = outs_with_ej[samp]['samp'][quant]
        if logged[i_quant]:
            x = Num.safe_lg(x)
        ax.c(cs[i_samp])
        ax.hist(x, range=x_lims[i_quant], bins=32, log=True, density=True, lw=2)
        med, std = np.median(x), np.std(x)
        ax.text(r'($%s$)\ %.2f\pm%.2f'%(samp, med, std), (.95, .95-i_samp*.06), color=cs[i_samp], ha='right', fontsize=12)
        
    ax.label(lab_names[i_quant])
    
axs_f.lim(y=[0.01, 100])
axs[:, 0].label(y=r'\rm PDF')

In [ ]:
h = sim_info.cosmology.hubble
s_hs =  [
    s_all.subset_by_value('m_crit200', 10**1.5 * h, 10**2.5 * h),
    s_c.subset_by_value('m_crit200', 10**3. * h, 10**10. * h)
]

In [ ]:
outs_wp_gal = DataDict() 
proj_cross_kw = dict(rs=np.logspace(-2., 1.2, 26), n_max_rand=None, n_repeat=20, n_threads=8)

In [ ]:
for k in 'low_zf',: #,'noej', 'dense', 'near_k', 'less_s':
    outs_wp_gal[k] = cls_ref.proj_cross(outs[f'{k}/samp'], **proj_cross_kw)

In [ ]:
k = 'h12'
outs_wp_gal[k] = cls_ref.proj_cross(s_hs[0], **proj_cross_kw)

k = 'h13'
outs_wp_gal[k] = cls_ref.proj_cross(s_hs[1], **proj_cross_kw)

In [ ]:
fig, ax = plot.subplots(1, figsize=5.5, 
                        margin=[0.1, 0.1, 0.1, 0.1], layout='none')

cs = 'k', 'r', 'g', cs_set1[1], 'orange', 'grey', 'grey'
lws = 3, 3, 3, 3, 3, 3, 1
for i_samp, samp in enumerate(['noej', 'dense', 'near_k', 'less_s', 'low_zf', 
                               'h12', 'h13']):
    out = outs_wp_gal[samp]
    x, y, e = out['lg_rs_c', 'wp', 'wp_sd']
    kw = dict(lw=lws[i_samp], label=samp)
    ax.c(cs[i_samp])
    ax.errorbar(x, y, e, **kw)
    
ax.scale(y='log').leg(loc='ll', fontsize=13, handlelength=1.75, ncol=2)\
    .label(r'\log\, r_{\rm p}\,[h^{-1}{\rm Mpc}]',r'w_{\rm p}(r_{\rm p})\,[h^{-1}{\rm Mpc}]')

In [ ]:
fig, axs = plot.subplots((2,2), share=False, 
                         space=0.2, subsize=(5., 4.5), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

cs = 'k', 'r', 'g', cs_set1[1], 'orange', 'grey', 'grey'
lws = 3, 3, 3, 3, 3, 3, 1
for i_samp, samp in enumerate(['noej', 'dense', 'near_k', 'less_s', 'low_zf']):
    s = outs[samp]['samp']
    for i_web, web_t in enumerate(['k', 'f', 's', 'v']):
        ax = axs_f[i_web]
        x = s[f'frac_{web_t}']  
        ax.c(cs[i_samp]).hist(x, bins=30, log=True, fc='none', lw=3,
                              label=samp)
        ax.label(r'f_{\rm %s}(<10\,h^{-1}{\rm Mpc})'%web_t)
        
axs_f[0].leg(loc='ur', fontsize=13)
axs.lim(y=[1.0, 1.0e5])
axs[:, 0].label(y=r'\rm Count')

In [ ]:
def show_map(samps = ['noej', 'dense', 'near_k', 'less_s', 'low_zf', 'high_zf'], map = f_interp._data, 
             field_type='density'):
    fig, axs = plot.subplots((2, 3), share=True, space=0, 
        subsize=(6.25, 6.25), margin=[.03, .01, .04, .06], layout='none')
    axs_f = axs.flat

    l_grid = clsf.mesh.l_grid
    l_box = sim_info.box_size

    z0 = 100.
    z1 = z0 + 10.
    dz = 1.

    zi0, zi1 = int((z0-dz) / l_grid), int((z1+dz) / l_grid)
    if field_type == 'density':
        im = Num.safe_lg(map[:, :, zi0:zi1].max(-1))
        im_1, im_2 = np.quantile(im, [0.01, .99])
    else:
        im = map[:, :, zi0:zi1].sum(-1) >= 1
        im_1, im_2 = 0., 2.

    cs = 'k', 'r', 'g', cs_set1[1], 'orange', 'grey', 'grey'
    lws = 3, 3, 3, 3, 3, 3, 1
    for i_samp, samp in enumerate(samps):
        s = outs[samp]['samp']
        X = s['x']
        x, y, z = X.T
        sel = (z >= z0) & (z <= z1)
        x, y = x[sel], y[sel]
        n_x = x.size
        print(x.size)
        
        ax = axs_f[i_samp]
        ax._raw.imshow(im.T, origin='lower', 
                        extent=[0., l_box, 0., l_box], 
                        cmap='Greys', vmin=im_1, vmax=im_2, aspect='auto')
        
        ax.scatter(x, y, s=(20 if n_x > 2000 else 100), fc='none', ec='b', marker='*', alpha=1, label=samp, 
                    rasterized=True, linewidths=1.)
            
    axs.lim([0., l_box], [0., l_box])
    axs.label(x=r'x\,[h^{-1}{\rm Mpc}]', y=r'y\,[h^{-1}{\rm Mpc}]').label_outer()
    axs.leg(loc='ur', fontsize=20, frameon=True, framealpha=0.9)
    
show_map()
show_map(map=clsf.is_filament, field_type='tidal')

## Web - z_form correlation

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'tidal_field.99.sm2.hdf5'
with h5.File(path) as f:
    clsf = cubic_box.TidalClassifier.new_from_file(f, lam_th=0.)

In [ ]:
sim_info = sims.predefined['tng_dark_300_1']
s_all = sample.SimSample.load(sim_info, 'z0_cent.hdf5').requires_spin()\
    .requires_assembly().requires_dist_to_halos().requires_disk().requires_cosmic_type(clsf)
s_c = s_all
s_all

In [ ]:
s_filament = sample.SubhaloSet.from_cosmic_web(clsf, sim_info, 'filament')
cls_filament = clustering.Clustering(s_filament)

In [ ]:
s_knot = sample.SubhaloSet.from_cosmic_web(clsf, sim_info, 'knot')
cls_knot = clustering.Clustering(s_knot)

In [ ]:
s_c = s_all

m_s_min = 10**(-2.5 - 0.15)
s_dwarf = s_c.subset_by_value('m_s_pred', lo=m_s_min, hi=10**-1.15)

s_noej = s_dwarf.subset_by_value('last_sat_z', lo=15.)
s_ej = s_dwarf.subset_by_value('last_sat_z', hi=1.5)

In [ ]:
s_noejhizf = s_noej.subset_by_value('z_half', 3., 15.)

In [ ]:
s_noejlozf = s_noej.subset_by_value('z_half', .9 , 1.)

In [ ]:
out_corr = DataDict()
cls_kw = dict(n_threads=4, rs=np.logspace(-0.3, 1.0, 15), n_repeat=10)

In [ ]:
cls = cls_filament.proj_cross

s = s_noejhizf
out_corr['noej_hizf_f'] = cls(s, **cls_kw)

s = s_noejlozf
out_corr['noej_lozf_f'] = cls(s, **cls_kw)

s = s_ej
out_corr['ej_f'] = cls(s, **cls_kw)

In [ ]:
cls = cls_knot.proj_cross

s = s_noejhizf
out_corr['noej_hizf_k'] = cls(s, **cls_kw)

s = s_noejlozf
out_corr['noej_lozf_k'] = cls(s, **cls_kw)

s = s_ej
out_corr['ej_k'] = cls(s, **cls_kw)

In [ ]:
fig, axs = plot.subplots((2,1), share=(True, False), 
    figsize=(6., 7.), space=0,
    margin=[.02, .02, .08, .14], layout='none')
axs_f = axs.flat


webs = 'f', 'k'

samps = 'ej', 'noej_hizf', 'noej_lozf'
cs = 'k', cs_set1[1], cs_set1[0]
labs = r'${\rm Ejected}$', r'${\rm NoEj}\,(z_{\rm h,1/2} > 3.)$', r'${\rm NoEj}\,(0.9 < z_{\rm h,1/2} < 1.0)$'

for i_web, web in enumerate(webs):
    ax = axs_f[i_web]
    for i_samp, samp in enumerate(samps):
        ax.c(cs[i_samp])
        k = f'{samp}_{web}'
        #x, y, e = out_corr[k]['lg_rs_c', 'xi', 'xi_sd']
        x, y, e = out_corr[k]['lg_rs_c', 'wp', 'wp_sd']
        ax.errorfill(x, y, yerr=e, label=labs[i_samp], 
                     fill_between_kw={'lw': 1.})
        
axs.scale(y='log')
#axs[0].lim([-0.2, 1.15], [0.1, 2.]).label(y=r'\xi_{\rm g, filament}(r)').leg(loc='ll')
#axs[1].lim(y=[0.1, 30.]).label(r'\log\, r\,[h^{-1}{\rm Mpc}]',
#                         r'\xi_{\rm g, knot}(r)')

# ProjPaths.save_fig('tng_dark_300_xi_to_cosmic_web.pdf')
#ProjPaths.save_fig('tng_dark_300_wp_to_cosmic_web.pdf')

In [ ]:
def regr(x: np.ndarray, xp: np.ndarray, stats_kw: dict):
    return stats.KernelRegression1D.by_local_kernel(x, x, xp, reduce='count', **stats_kw)['y'][0]

def get_fsel(x: np.ndarray, sel: np.ndarray, x_range=[0., 1.], max_dx=.2, kernel=.1):
    x_sel = x[sel]
    xp = np.linspace(*x_range, 64)
    stats_kw = dict(max_dx=max_dx, kernel=kernel)
    y = regr(x, xp, stats_kw=stats_kw)
    y_sel = regr(x_sel, xp, stats_kw=stats_kw)
    f = Num.safe_div(y_sel, y)
    return DataDict({'x': xp, 'f': f})

In [ ]:
outs = DataDict()

z, is_f, is_n = s_noej['z_half', 'is_filament', 'is_knot']
x_t = np.log10(1. + z)

outs['noej_f'] = get_fsel(x_t, is_f, max_dx=.3, kernel=.15)
outs['noej_k'] = get_fsel(x_t, is_n, max_dx=.3, kernel=.15)

z, is_f, is_n = s_ej['z_half', 'is_filament', 'is_knot']
x_t = np.log10(1. + z)

outs['ej_f'] = get_fsel(x_t, is_f, max_dx=.3, kernel=.15)
outs['ej_k'] = get_fsel(x_t, is_n, max_dx=.3, kernel=.15)

In [ ]:
fig, ax = plot.subplots(1, figsize=(6., 5.0), margin=[.1, .03, .12, .12], layout='none')

labs_i = r'Filaments', r'Knots'
labs_j = r'NoEj', r'Ejected'

ax.fmt_line(lw=5)

iks = 'f', 'k'
lss = '-', (0,(1,1))
lws = 1.5, 6

jks = 'noej', 'ej'
cs = cs_set1[1], cs_set1[0]

for i in range(2):
    ik, ilab = iks[i], labs_i[i]
    for j in range(2):
        jk, jlab = jks[j], labs_j[j]
        x, f = outs[f'{jk}_{ik}']['x', 'f']
        ax.plot(x, f, label=r'$\rm %s\, (%s)$'%(ilab, jlab), 
                lw=lws[i], ls=lss[i], c=cs[j])

ax._raw.fill_betweenx([-1., 2.], [0.6, 0.6], [1., 1.], color=cs[0], 
                      zorder=-10, alpha=.2, lw=0)
ax.text(r'\rm Diffuse\ dwarf\ candidate', 
        (.99, .92), ha='right', color=cs[0], fontsize=14)

astro.plot.Axes(ax).add_secondary_axis_lgzp1_to_z(label=r'$z_{\rm h,1/2}$')
ax.lim([0., 1.], [0., 1.]).label(r'\log\,(1+z_{\rm h,1/2})', r'f_{\rm web\ type}')
ax.leg(loc='ul', title=r'$\bf Web\ type$', labelcolor='linecolor')

ProjPaths.save_fig('f_web_type_vs_z_f.pdf')

## Halo views

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'tidal_field.99.sm2.hdf5'
with h5.File(path) as f:
    clsf = cubic_box.TidalClassifier.new_from_file(f, lam_th=0.)

In [ ]:
path = ProjPaths.sim_dir_of(sim_info) / 'tidal_field.99.sm1.hdf5'
with h5.File(path) as f:
    f_interp = cubic_box.cosmic_web.FieldInterpolator.new_density_field_from_file(
        f)
f_interp

In [ ]:
sim_info = sims.predefined['tng_dark_300_1']
s_all = sample.SimSample.load(sim_info, 'z0_cent.hdf5').requires_spin().requires_assembly().requires_dist_to_halos().requires_disk()
s_all

In [ ]:
s_c = s_all

In [ ]:
m_s_min = 10**(-2.5 - 0.15)
s_dwarf = s_c.subset_by_value('m_s_pred', lo=m_s_min, hi=10**-1.15)
s_noej = s_dwarf.subset_by_value('last_sat_z', lo=15.)

In [ ]:
s_noej_zf3 = s_noej.subset_by_value('z_half', 2.75, 15.)
s_noej_zfl2 = s_noej.subset_by_value('z_half', 0.9, 1.)

s_noej_zf3, s_noej_zfl2

In [ ]:
h = sim_info.cosmology.hubble
s_hs =  [
    s_c.subset_by_value('m_h', 10**1.5 * h, 10**2.5 * h),
    s_c.subset_by_value('m_h', 10**3. * h, 10**10. * h)
]

In [ ]:
fig, axs = plot.subplots((3,2), share=True, space=0, 
    subsize=(6.25, 6.25), margin=[.03, .01, .04, .06], layout='none')
axs_f = axs.flat

l_grid = f_interp._shape_fn.mesh.l_grid
l_box = sim_info.box_size
    
z0s = [20., 100., 180.]
cs = 'b', 'r'
labs = r'$z_{\rm h,1/2} > 2.75$', r'$ 0.9 \leqslant z_{\rm h,1/2} < 1$'
for i_z, z0 in enumerate(z0s):
    z1 = z0 + 10.0
    dz = 0.    
    zi0, zi1 = int((z0-dz) / l_grid), int((z1+dz) / l_grid)
    im = Num.safe_lg(f_interp._data[:, :, zi0:zi1].max(-1))
    im_1, im_2 = np.quantile(im, [0.01, 0.99])

    for i, s_g in enumerate([s_noej_zf3, s_noej_zfl2]):
        x, y, z = s_g['x'].T
        sel = (z >= z0) & (z <= z1)
        x, y = x[sel], y[sel]

        s = (s_g['r_h'][sel])**1. * 2200.
        
        ax = axs[i_z, i]
                
        ax._raw.imshow(im.T, origin='lower', 
                    extent=[0., l_box, 0., l_box], 
                    cmap='Greys', vmin=im_1, vmax=im_2, aspect='auto')
        
        ax.scatter(x, y, s=s, fc='none', ec=cs[i], marker='*', alpha=1, label=labs[i], 
                rasterized=True, zorder=100, linewidths=1.)
        
        if i_z == 0:
            ax._raw.set_title(labs[i], color=cs[i])        

axs.lim([0., l_box], [0., l_box])
axs.label(x=r'x\,[h^{-1}{\rm Mpc}]', y=r'y\,[h^{-1}{\rm Mpc}]').label_outer();

ProjPaths.save_fig('tng_dark_300_map_with_density_field.pdf')

In [ ]:
fig, axs = plot.subplots((3,2), share=True, space=0, 
    subsize=(6.25, 6.25), margin=[.03, .01, .04, .06], layout='none')
axs_f = axs.flat

l_grid = f_interp._shape_fn.mesh.l_grid
l_box = sim_info.box_size
    
z0s = [20., 100., 180.]
cs = 'b', 'r'
labs = r'$z_{\rm h,1/2} > 2.75$', r'$ 0.9 \leqslant z_{\rm h,1/2} < 1$'
for i_z, z0 in enumerate(z0s):
    z1 = z0 + 10.0
    dz = 0.    
    zi0, zi1 = int((z0-dz) / l_grid), int((z1+dz) / l_grid)
    im = Num.safe_lg(f_interp._data[:, :, zi0:zi1].max(-1))
    im_1, im_2 = np.quantile(im, [0.01, 0.99])

    for i, s_g in enumerate([s_highzf, s_dwarf]):
        x, y, z = s_g['x'].T
        sel = (z >= z0) & (z <= z1)
        x, y = x[sel], y[sel]

        s = (s_g['r_h'][sel])**1. * 2200.
        
        ax = axs[i_z, i]
                
        ax._raw.imshow(im.T, origin='lower', 
                    extent=[0., l_box, 0., l_box], 
                    cmap='Greys', vmin=im_1, vmax=im_2, aspect='auto')
        
        ax.scatter(x, y, s=s, fc='none', ec=cs[i], marker='*', alpha=1, label=labs[i], 
                rasterized=True, zorder=100, linewidths=1.)
        
        if i_z == 0:
            ax._raw.set_title(labs[i], color=cs[i])        

axs.lim([0., l_box], [0., l_box])
axs.label(x=r'x\,[h^{-1}{\rm Mpc}]', y=r'y\,[h^{-1}{\rm Mpc}]').label_outer();

#ProjPaths.save_fig('tng_dark_300_map_with_density_field.pdf???')

In [ ]:
fig, axs = plot.subplots((3,2), share=True, space=0, 
    subsize=(6.25, 6.25), margin=[.03, .01, .04, .06], layout='none')
axs_f = axs.flat

l_grid = clsf.mesh.l_grid
l_box = sim_info.box_size
    
z0s = [20., 100., 180.]
cs = 'b', 'r'
labs = r'$z_{\rm h,1/2} > 2.75$', r'$ 0.9 \leqslant z_{\rm h,1/2} < 1$'
for i_z, z0 in enumerate(z0s):
    z1 = z0 + 10.0
    dz = 5.    
    zi0, zi1 = int((z0-dz) / l_grid), int((z1+dz) / l_grid)
    im = clsf.is_filament[:, :, zi0:zi1].sum(-1) >= 1
    #im = clsf.is_filament[:, :, zi0:zi1]
    #im = im.sum(-1) / im.shape[-1]

    for i, s_g in enumerate([s_highzf, s_dwarf]):
        x, y, z = s_g['x'].T
        sel = (z >= z0) & (z <= z1)
        x, y = x[sel], y[sel]

        s = (s_g['r_h'][sel])**1. * 800.
        
        ax = axs[i_z, i]
                
        ax._raw.imshow(im.T, origin='lower', 
                    extent=[0., l_box, 0., l_box], 
                    cmap='Greys', vmin=0., vmax=2., aspect='auto')
        
        ax.scatter(x, y, s=s, fc=cs[i], ec=cs[i], marker='*', alpha=1, label=labs[i], 
                rasterized=True, zorder=100)
        
        if i_z == 0:
            ax._raw.set_title(labs[i], color=cs[i])        

axs.lim([0., l_box], [0., l_box])
axs.label(x=r'x\,[h^{-1}{\rm Mpc}]', y=r'y\,[h^{-1}{\rm Mpc}]').label_outer();

#ProjPaths.save_fig('tng_dark_300_map_with_cosmic_web.pdf??')

In [ ]:
fig, axs = plot.subplots((3,2), share=True, space=0, 
    subsize=(6.25, 6.25), margin=[.03, .01, .04, .06], layout='none')
axs_f = axs.flat

l_grid = clsf.mesh.l_grid
l_box = sim_info.box_size
    
z0s = [20., 100., 180.]
cs = 'b', 'r'
labs = r'$z_{\rm h,1/2} > 2.75$', r'$ 0.9 \leqslant z_{\rm h,1/2} < 1$'
for i_z, z0 in enumerate(z0s):
    z1 = z0 + 10.0
    dz = 2.    
    zi0, zi1 = int((z0-dz) / l_grid), int((z1+dz) / l_grid)
    im = clsf.is_sheet[:, :, zi0:zi1].sum(-1) >= 1

    for i, s_g in enumerate([s_cand, s_dwarf]):
        x, y, z = s_g['x'].T
        sel = (z >= z0) & (z <= z1)
        x, y = x[sel], y[sel]

        s = (s_g['r_h'][sel])**1. * 800.
        
        ax = axs[i_z, i]
                
        ax._raw.imshow(im.T, origin='lower', 
                    extent=[0., l_box, 0., l_box], 
                    cmap='Greys', vmin=0., vmax=2., aspect='auto')
        
        ax.scatter(x, y, s=s, fc=cs[i], ec=cs[i], marker='*', alpha=1, label=labs[i], 
                rasterized=True, zorder=100)
        
        if i_z == 0:
            ax._raw.set_title(labs[i], color=cs[i])        

axs.lim([0., l_box], [0., l_box])
axs.label(x=r'x\,[h^{-1}{\rm Mpc}]', y=r'y\,[h^{-1}{\rm Mpc}]').label_outer();

#ProjPaths.save_fig('tng_dark_300_map_with_cosmic_web.pdf??')

In [ ]:
fig, axs = plot.subplots((3,2), share=True, space=0, 
    subsize=(6.25, 6.25), margin=[.03, .01, .04, .06], layout='none')
axs_f = axs.flat

l_grid = clsf.mesh.l_grid
l_box = sim_info.box_size
    
z0s = [20., 100., 180.]
cs = 'b', 'r'
labs = r'$z_{\rm h,1/2} > 2.75$', r'$ 0.9 \leqslant z_{\rm h,1/2} < 1$'
for i_z, z0 in enumerate(z0s):
    z1 = z0 + 10.0
    dz = 0.    
    zi0, zi1 = int((z0-dz) / l_grid), int((z1+dz) / l_grid)
    im = clsf.is_filament[:, :, zi0:zi1].sum(-1) >= 1

    for i, s_g in enumerate([s_noej_zf3, s_noej_zfl2]):
        x, y, z = s_g['x'].T
        sel = (z >= z0) & (z <= z1)
        x, y = x[sel], y[sel]

        s = (s_g['r_h'][sel])**1. * 800.
        
        ax = axs[i_z, i]
                
        ax._raw.imshow(im.T, origin='lower', 
                    extent=[0., l_box, 0., l_box], 
                    cmap='Greys', vmin=0., vmax=2., aspect='auto')
        
        ax.scatter(x, y, s=s, fc=cs[i], ec=cs[i], marker='*', alpha=1, label=labs[i], 
                rasterized=True, zorder=100)
        
        if i_z == 0:
            ax._raw.set_title(labs[i], color=cs[i])        

axs.lim([0., l_box], [0., l_box])
axs.label(x=r'x\,[h^{-1}{\rm Mpc}]', y=r'y\,[h^{-1}{\rm Mpc}]').label_outer();

ProjPaths.save_fig('tng_dark_300_map_with_cosmic_web.pdf')

In [ ]:
fig, axs = plot.subplots((2,2), share=True, space=0.02, subsize=8, margin=[.025, .025, .045, .045], layout='none')
axs_f = axs.flat

z0, z1 = 20., 50.

for i, s_h in enumerate(s_hs):
    
    x, y, z = s_h['x'].T
    sel = (z >= z0 - 5.) & (z <= z1 + 5.)
    x, y = x[sel], y[sel]

    s = (s_h['r_h'][sel])**1. * 300.
    for j in range(2):
        ax = axs[j,i]
        ax.scatter(x, y, s=s, c='grey', alpha=.65, ec='none', rasterized=True)
    
cs = 'b', 'r'
labs = r'$z_{\rm h,1/2} > 2.75$', r'$ 0.9 \leqslant z_{\rm h,1/2} < 1$'
for i, s_g in enumerate([s_noej_zf3, s_noej_zfl2]):
    x, y, z = s_g['x'].T
    sel = (z >= z0) & (z <= z1)
    x, y = x[sel], y[sel]

    s = (s_g['r_h'][sel])**1. * 800.
    for j in range(2):
        ax = axs[i, j]
        ax.scatter(x, y, s=s, fc=cs[i], ec=cs[i], marker='*', alpha=1, label=labs[i], rasterized=True)
        
l_box = sim_info.box_size
ax.lim([0., l_box], [0., l_box])

axs[0,0]._raw.set_title(r'${\rm Halos},\,11.5 \leqslant \log\,M_{\rm h}/M_\odot<12.5$')
axs[0,1]._raw.set_title(r'${\rm Halos},\,13 \leqslant \log\,M_{\rm h}/M_\odot $')
axs.label(x=r'x\,[h^{-1}{\rm Mpc}]', y=r'y\,[h^{-1}{\rm Mpc}]').label_outer()
for ax in axs[:, 0]:
    ax.leg(loc='ul', labelcolor='mec', scatterpoints=1, 
           markerscale=2.5, fontsize=18, facecolor='white', frameon=True)
ProjPaths.save_fig('tng_dark_300_map.pdf')

### Output catalogs

In [ ]:
s_diffuse_candidate = s_noej.subset_by_value('z_half', 3., 15.)
out_keys = 'subhalo_id', 'subfind_id', 'm_crit200', 'm_s_pred', 'z_half'
out_dict = {k: s_diffuse_candidate[k] for k in out_keys}
out_df = pd.DataFrame(out_dict)
out_path = ProjPaths.sim_dir_of(sim_info) / 'diffuse_candidate.txt'
out_df.to_csv(out_path, index=False, header=False)
out_df.columns

# TNG-Dark

In [ ]:
%autoreload
from obs.assembly_bias.clustering import SubSampleManager, SimCross
from sklearn.neighbors import KDTree
from obs.assembly_bias.utils import gal_wp
from scipy.interpolate import interp1d

In [ ]:
sim_info = sims.predefined['tng_dark']
sim_dir = ProjPaths.man_sim_dir(sim_info)

## Datasets

### Full sample

In [ ]:
s_all = sample.SimSample.load(sim_info, 'tng_dark_z0.hdf5')\
    .requires_spin().requires_assembly().require_c()
    #.requires_dist_to_halos().requires_disk()
s_all

In [ ]:
p = sim_dir.base_dir / 'z0_all_postprocessed.hdf5'
assembly_bias.sample_dumper.dump_sample_to(p, 
    s_all.objs, sim_info, sim_info.n_snapshots-1)

In [ ]:
h5.File.ls_from(p)

### LGalaxies-H15

In [ ]:
s_all = sample.SimSample.load(sim_info, 'z0_all.hdf5')\
    .require_LGalaxies_H15()\
    .requires_spin().requires_assembly().require_c()\
    #.requires_dist_to_halos().requires_disk()
s_all

In [ ]:
p = sim_dir.base_dir / 'z0_all_postprocessed.hdf5.LGalaxies-H15'
assembly_bias.sample_dumper.dump_sample_to(p, 
    s_all.objs, sim_info, sim_info.n_snapshots-1)

### SIDM sample

In [ ]:
pdata = sim_dir.base_dir / 'z0_all_postprocessed.hdf5'
samp = sample.SimSample.load(sim_info, pdata)
samp.objs['hid'] = np.arange(samp.n_objs)

In [ ]:
u_m = sim_info.cosmology.unit_system.u_m_to_sol
m_lb, m_ub = 10**10.7 / u_m, 10**10.9 / u_m
o_key = 'lm10p7-10p9'

s_h = samp.subset_by_value('is_c', eq=True)\
    .subset_by_value('m_mean200', lo=m_lb, hi=m_ub)\
    .subset_by_value('last_sat_z', lo=15.)
print(s_h)

In [ ]:
out = {k: s_h[k]  for k in ['m_mean200', 'r_mean200', 'c_mean200', 
                            'spin_mean200', 'z_half', 'hid']}
sim_dir.dump_sample(out, f'z0{o_key}.hdf5', f_flag='w')

## + LGalaxies-H15

In [ ]:
pdata = sim_dir.base_dir / 'z0_all_postprocessed.hdf5.LGalaxies-H15'
man_kw = {
    'add_rsd': False,
    'dmo': False,
    # 'm_s_range_in_sol': [10**8., 10**9.],
    'm_s_range_in_sol': [10**8.5, 10**9.],
}
sman = SubSampleManager(sim_info, pdata, **man_kw)
sman.s_dwarf

In [ ]:
wp_kw = {
    'pimax': 10., 'n_repeat': 100,
    'bootstrap_kw': {'keep_samples': True},
    # 'rs': np.array([2., 10.]),
}
sim_cross = SimCross(sman)

In [ ]:
o_path = sim_dir.base_dir / 'wp/dwarfs_H15_lm8.5-9_r2-10_by-S.json'

Sigma_star_lims = [-0.1, 7.,15., 25., 1.0e6]
wps = sim_cross.wps_of('s_dwarf', 'Sigma_star', 
                       ps=None, qs=Sigma_star_lims, **wp_kw)
Json.dump_file(wps, o_path, indent=2)

In [ ]:
# o_path = sim_dir.base_dir / 'wp/dwarfs_lm8-9_by-S.json'
o_path = sim_dir.base_dir / 'wp/dwarfs_H15_lm8.5-9_by-S.json'

Sigma_star_lims = [-0.1, 7.,15., 25., 1.0e6]
wps = sim_cross.wps_of('s_dwarf', 'Sigma_star', 
                       ps=None, qs=Sigma_star_lims, **wp_kw)
wps['total'] = sim_cross.wp_of('s_dwarf', **wp_kw)
Json.dump_file(wps, o_path, indent=2)

In [ ]:
lm_h_edges = [11., 12., 13., 16.]
o_path = sim_dir.base_dir / 'wp/halos_by-M.json'

u_m = 1.0e10 / sman.sim_info.cosmology.hubble
m_h_edges = 10.0**np.array(lm_h_edges) / u_m
wps = sim_cross.wps_of('s_halo', 'm_mean200', ps=None, qs=m_h_edges, **wp_kw)

Json.dump_file(wps, o_path, indent=2)

### 2PCF

### Property distributions

In [ ]:
m_h, m_s, ssfr = sman.s_c['m_mean200', 'm_star', 'ssfr']

fig, axs = plot.subplots((1,2), share=False, space=0.25, subsize=(5.0, 5.0), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

ax = axs_f[0]
x, y = Num.safe_lg(m_h, m_s)
ax.scatter(x, y, s=1, alpha=.5, rasterized=True)

ax = axs_f[1]
x, y = Num.safe_lg(m_s, ssfr)
ax.scatter(x, y.clip(-2.2), s=1, alpha=.5, rasterized=True)
ax.lim([-2.5, 2.5], [-2.5, 2.5])

In [ ]:
z_f, S = sman.s_dwarf['z_half', 'Sigma_star']
lS = Num.safe_lg(S)

In [ ]:
z_p = np.linspace(0., 5., 256)
S_p, el, eh = stats.KernelRegression1D.by_local_kernel(z_f, S, z_p, max_dx=.25, reduce='errorbar')['y'][0].T

In [ ]:
Json.dump_file(
    {
        'z_half': z_p, 'Sigma_star': S_p, 'Sigma_star_err': (el, eh)
    },
    sim_dir.base_dir / 'stats' / 'z_half-Sigma_star.json'
)

In [ ]:
fig, axs = plot.subplots((1,2), share=False, space=0.25, 
                         subsize=(5.0, 5.0), margin=[0.1, 0.1, 0.1, 0.1], layout='none')
axs_f = axs.flat

ax = axs_f[0]
ax.scatter(S, z_f, s=1, alpha=.5, rasterized=True)
ax.c('r').errorbar(S_p, z_p, xerr=(el, eh), lw=2.5)
ax.scale('log').lim(y=[0., 4.5])

ax = axs_f[1]
ax.hist(np.log10(S), range=[-1., 3.], bins=32)

In [ ]:
z_f, S, spin, c = sman.s_dwarf['z_half', 'Sigma_star', 'spin_mean200', 'c_mean200']
lS = Num.safe_lg(S)
lspin = Num.safe_lg(spin)

In [ ]:
fig, axs = plot.subplots(
    (1, 3),
    share=False, space=0.25, subsize=(5.5, 5.0),
    margin=[0.1, 0.1, 0.1, 0.1],
    layout='none')
axs_f = axs.flat


axs.c('r').fmt_marker(fa=1)
lspin_rg = [-2.55, -0.91]
c_rg = [2., 40.]
lS_rg = [-0.4, 3.]
lspin_label = r'\log\, \lambda'
c_label = r'c'
lS_lab = r'\log\, \Sigma_*\,[{\rm M_\odot pc^{-2}}]'

ax = axs_f[0]
ax.scatter(lspin, lS, rasterized=True)
ax.lim(lspin_rg, lS_rg).label(lspin_label, lS_lab)

ax = axs_f[1]
ax.scatter(c, lS, rasterized=True)
ax.lim(c_rg, lS_rg).label(c_label, lS_lab)

ax = axs_f[2]
X = np.column_stack([lspin, c])
X_mean, X_sd = X.mean(0), np.std(X, axis=0)
X_sc = (X - X_mean) / X_sd
y_sc = lS
y_p = stats.KernelRegressionND.by_knn(X_sc, y_sc, X_sc, k=32)['y'][0]
ax.fmt_marker(elw=0)
art = ax.scatter(lspin, c, c=y_p, rasterized=True, s=20, zorder=20, 
                 cmap='gnuplot2', vmin=lS_rg[0], vmax=lS_rg[1]).last_draw
ax.scatter(lspin, c, c='gray', rasterized=True, s=40, zorder=19)
ax.label(lspin_label, c_label)
    
n = 32
k = 32
grid = GridMd(np.linspace(*lspin_rg, n), np.linspace(*c_rg, n))
y_p = grid.applied(
    lambda Xp: stats.KernelRegressionND.by_knn(
        X_sc, y_sc, (Xp-X_mean)/X_sd, k=k)['y'][0],
    xy_indexing=True).values
x1p, x2p = grid.locs_1d
ax._raw.contour(x1p, x2p, y_p, levels=16, colors='w', 
                linewidths=1., zorder=21)
fig.colorbar(art, ax=ax, location='right', label='$%s$'%lS_lab)

ax.lim(lspin_rg, c_rg)

### Sample Inspection

In [ ]:
class Sample2D(abc.HasLog):
    def __init__(self, x, y, z, verbose=True) -> None:
        super().__init__(verbose=verbose)

        self.x, self.y, self.z = x, y, z
        self.X = np.column_stack([x, y])

    def plot(self, ax: plot.Axes, x_lim, y_lim, n=32, levels=16,
             regr_f=0.05, regr_k=256, reduce='mean', regr_kernel=None, alpha=.65):
        grid = GridMd(np.linspace(*x_lim, n), np.linspace(*y_lim, n))
        X, x, y, z = self.X, self.x, self.y, self.z
        k = min(regr_k, len(self.X) * regr_f)
        self.log(f'{k=}')
        kernel = ('gaussian', {'sigma': regr_kernel}
                  ) if regr_kernel is not None else None
        z_pred = grid.applied(
            lambda Xp: stats.KernelRegressionND.by_knn(
                X, z, Xp, k=k, kernel=kernel, reduce=reduce)['y'][0],
            xy_indexing=True).values
        x_pred, y_pred = grid.locs_1d

        art = ax._raw.contour(x_pred, y_pred, z_pred,
                              levels=levels, colors='red', linewidths=3.)
        ax._raw.clabel(art, inline=True, fontsize=15)

        k2d = plot.KNearestNeighbor2D(k=k//2)
        s2d = ax.scatter_2d(x, y, range=[x_lim, y_lim], n_bins=n,
                            density_estimator=k2d)
        s2d.mesh(
            cmap='gnuplot2', alpha=alpha).contour(
            ps=[.99, .95, .68, .5],
            colors='w')

In [ ]:
# info = sims.predefined['tng']
sim_info = sims.predefined['tng_dark']
spin_samp = SpinSample(sim_info, suffix='.only_c', sham_key='m_peak')
s_ref = spin_samp.subsample_by_value('v_peak_m', lo=10**0.0)
s_ref_h13 = spin_samp.subsample_by_value('m_h', lo=1.0e3)
s_ref_h12 = spin_samp.subsample_by_value('m_h', lo=1.0e2)
s_dst = s_ref.subsample_by_value('m_s_pred', lo=10**-2.65, hi=10**-1.15)
# lowm_samp = spin_samp.subsample_by_value('m_star', lo=1.0e-2, hi=1.0e-1)
# ref_samp = spin_samp.subsample_by_value('m_star', lo=1.0e-2, hi=1.0e5)
#lowm_samp = spin_samp.subsample_by_value('m_tophat', lo=10**0.16, hi=10**1.)
#ref_samp = spin_samp.subsample_by_value('m_star', lo=10**0.16, hi=10**5.)

#lowm_nobs_samp = lowm_samp.subsample_by_value('last_sat_z', 1., 100.)
#highm_samp = spin_samp.subsample_by_value('m_star', lo=1.0e0, hi=1.0e3)
#lowm_samp

In [ ]:
_cls = spin_clustering._Clustering(sim_info)

In [ ]:
samp_h = spin_samp.subsample_by_value('m_h', 10**3.*.7)
samp_s = s_dst.subsample_by_value('z_half', -1., 100.)\
    .subsample_by_value('last_sat_z', -1., 17.5); 

x_s, x_h = samp_s['x'], samp_h['x']
d_min = _cls.min_dist_3d(x_s, x_h)

x, y = samp_s['x_t_half', 'last_sat_z']
y = Num.safe_lg(1.+y)

In [ ]:
fig, ax = plot.subplots(1, figsize=5., extent=[1., 1., 0., 0.], layout='none')

Sample2D(x, y, d_min).plot(ax, [0., 1.], [0., 1.], 
    n=32, levels=16, regr_k=256, regr_kernel=0.1, alpha=.75)
ax.label(r'\log (1+z_{\rm half})', r'\log (1+z_{\rm ej})')
a_plot.Axes(ax).add_secondary_axis_lgzp1_to_z(label=r'$z_{\rm half}$')
a_plot.Axes(ax).add_secondary_axis_lgzp1_to_z('right',label=r'$z_{\rm eject}$')

In [ ]:
samp_h = spin_samp.subsample_by_value('m_h', 10**3.*.7)
samp_s = s_dst.subsample_by_value('z_half', -1., 100.)

x_s, x_h = samp_s['x'], samp_h['x']
d_min = cls.min_dist_3d(x_s, x_h)

x1, x2, y = samp_s['S', 'S_fR', 'x_t_half']
x1, x2 = Num.safe_lg(x1, x2)

In [ ]:
fig, axs = plot.subplots((1,2), share=True, space=0.02, extent=[1., 1., 0., 0.], layout='none')
axs_f = axs.flat

for i_x, x in enumerate([x1, x2]):
    ax = axs[i_x]
    Sample2D(x, y, d_min).plot(ax, [-.51, 2.49], [0., .85], 
        n=32, levels=16, regr_kernel=0.25, regr_f=.1, regr_k=512)
    ax.label(r'\log \Sigma_*\ [M_\odot{\rm pc}^{-2}]', r'\log (1+z_{\rm half})')

a_plot.Axes(axs[1]).add_secondary_axis_lgzp1_to_z('right', label=r'$z_{\rm half}$')
axs.label_outer()

In [ ]:
x1, x2, y, z = samp_s['S', 'S_fR', 'x_t_half', 'm_s_pred']
x1, x2, z = Num.safe_lg(x1, x2, z*1.0e10/.7)

In [ ]:
fig, axs = plot.subplots((1,2), share=True, space=0.02, extent=[1., 1., 0., 0.], layout='none')
axs_f = axs.flat

for i_x, x in enumerate([x1, x2]):
    ax = axs[i_x]
    Sample2D(x, y, z).plot(ax, [-.51, 2.49], [0., .85], 
        n=32, levels=12, regr_kernel=0.25, regr_f=.1, regr_k=512, alpha=.75)
    ax.label(r'\log \Sigma_*\ [M_\odot{\rm pc}^{-2}]', r'\log (1+z_{\rm half})')

a_plot.Axes(axs[1]).add_secondary_axis_lgzp1_to_z('right', label=r'$z_{\rm half}$')
axs.label_outer()

In [ ]:
cls = spin_clustering._Clustering(sim_info)

samp_h = spin_samp.subsample_by_value('m_h', 10**3.*.7)
samp_s = s_dst.subsample_by_value('z_half', -1., 100.)

x_s, x_h = samp_s['x'], samp_h['x']
d_min = cls.min_dist_3d(x_s, x_h)

z, y, x = samp_s['z_half', 'S_fR', 'm_s_pred']
x, y = Num.safe_lg(x * 1.0e10 / .7, y)

In [ ]:
fig, ax = plot.subplots(1, figsize=5., extent=[1., 1., 0., 0.], layout='none')

Sample2D(x, y, z).plot(ax, [7.55, 9.], [-.5, 2.5], 
    n=32, levels=24, regr_kernel=0.25, regr_f=.1, regr_k=512)
ax.label(r'\log M_*\,[M_\odot]', r'\log \Sigma_*\ [M_\odot{\rm pc}^{-2}]')

### Bias Map

In [ ]:
samp = lowm_samp
xlim, ylim = (-0.02, 0.77), (-2.5, -.7)

#samp = lowm_samp
#xlim, ylim = (-0.02, 1.), (-2.5, -.7)

cls = Clustering(samp)
res = cls.count_r([2.,10.])
xis = res.xis_betweens()
x, y = samp['x_t_half', 'spin']
y = Num.safe_lg(y)
z = xis + 1.

In [ ]:
X = np.column_stack([x,y])
sd = stats.preprocessing.Standardize(X)
X = sd(X)
k = len(X) // 12
knn = stats._KnnRegression().fit(X, z)
query_kw = {'k': k, 'kernel': ('gaussian', {'sigma': .5})}
z_pred = knn(X, **query_kw)['y_mean']

In [ ]:
grid = GridMd.at[xlim[0]:xlim[1]:64j, ylim[0]:ylim[1]:64j]
def f(Xp):
    Xp = sd(Xp)
    z_pred = knn(Xp, 'mean', **query_kw)['y_mean']
    cnt = knn(Xp, 'count', max_dx=.3, **query_kw)['y_count']
    return np.ma.array(z_pred, mask=cnt < 8.)
x_g, y_g = grid.locs_1d
z_g = grid.applied(f, xy_indexing=True).values

In [ ]:
xp = np.linspace(*xlim, 32)
kw = dict()
yp, yc = stats.KernelRegression1D.by_knn(x, y, xp, k=len(x)//32, max_dx=.1,
    reduce=[('quantile', {'ps': [.25, .5, .75]}), 'count'],
    kernel=('gaussian', {'sigma': .5}), 
    **kw)['y']
xp, yp = Mask.on_arrays(yc>64, xp, yp)

In [ ]:
fig, axs = plot.subplots((2,2), share=('col', 'row'), space=0., ratios=([0.25,1], [0.25, 1]),
        figsize=(6,5.25),
        extent=[1., 1., 0., 0.], layout='none')
axs_f = axs.flat

zlims = np.quantile(z_pred, (.001, .999)) 

ax = axs[1,1]
lt = ax.scatter(x, y, c='k', ec='none', s=125, alpha=1).last_draw
lt = ax.scatter(x, y, c=z_pred, cmap='gnuplot2', ec='none', 
    s=75, alpha=1, vmin=zlims[0], vmax=zlims[1]).last_draw

ax._raw.contour(x_g, y_g, z_g, levels=np.linspace(zlims[0]+.025, zlims[1]-.025, 24), colors='k', linewidths=1.5)
fig.colorbar(lt, ax=axs, location='right', pad=.02, anchor=(0., 0.), shrink=4./5.,
    label=r'$1+\xi_{\rm p,S}(r=2\,$-$\,10\,h^{-1}{\rm Mpc})$')

lws = [3, 7, 3]
lss = '-', '-', '-'
for i, _yp in enumerate(yp.T):
    ax.plot(xp, _yp, c='gray', lw=lws[i], ls=lss[i], alpha=.75)

ax.label(r'\log\,(1+z_{\rm h,1/2})').lim(xlim, ylim)

axs[0,0].axis_off()

ax = axs[0,1]
a_plot.Axes(ax).add_secondary_axis_lgzp1_to_z(label=r'$z_{\rm h,1/2}$')
ax.hist(x, range=xlim, bins=32, density=True).lim(y=[0, 6.5])

ax = axs[1,0]
ax.label(y=r'\log\,\lambda_{\rm h}')
ax.hist(y, range=ylim, bins=32, orientation='horizontal', density=True)
ax.lim(x=[0,3.19])